[ScaDaMaLe, Scalable Data Science and Distributed Machine Learning](https://lamastex.github.io/scalable-data-science/sds/3/x/)
==============================================================================================================================

Old Bailey Online Data Analysis in Apache Spark
===============================================

2016, by Raaz Sainudiin and James Smithies is licensed under [Creative
Commons Attribution-NonCommercial 4.0 International
License](http://creativecommons.org/licenses/by-nc/4.0/).

#### Old Bailey, London's Central Criminal Court, 1674 to 1913

-   with Full XML Data for another great project. This is a starting
    point for ETL of Old Bailey Online Data from
    <http://lamastex.org/datasets/public/OldBailey/index.html>.

This work merely builds on [Old Bailey Online by Clive Emsley, Tim
Hitchcock and Robert Shoemaker](https://www.oldbaileyonline.org/) that
is licensed under a Creative Commons Attribution-NonCommercial 4.0
International License. Permissions beyond the scope of this license may
be available at https://www.oldbaileyonline.org/static/Legal-info.jsp.

  

### This exciting dataset is here for a course project in digital humanities

#### To understand the extraction job we are about to do here:

-   see [Jasper Mackenzie, Raazesh Sainudiin, James Smithies and Heather
    Wolffram, A nonparametric view of the civilizing process in London's
    Old Bailey, Research Report UCDMS2015/1, 32 pages,
    2015](http://lamastex.org/preprints/20150828_civilizingProcOBO.pdf).

The data is already loaded in dbfs (see dowloading and loading section
below for these details).

Analysing the Full Old Bailey Online Sessions Papers Dataset
============================================================

First **Step 0: Dowloading and Loading Data (The Full Dataset)** below
should have been done on the shard.  
This currently cannot be done in Community Edition as the dataset is not
loaded into the dbfs available in CE yet. But the datset is in the
academic shard and this is a walkthorugh of the Old Bailey Online data
in the academic shard.

Let's first check that the datasets are there in the distributed file
system.

In [ ]:
display(dbutils.fs.ls("dbfs:/datasets/obo/tei/")) // full data if you have it - not in CE!!

  

[TABLE]

In [ ]:
display(dbutils.fs.ls("dbfs:/datasets/obo/tei/ordinarysAccounts"))

  

[TABLE]

Truncated to 30 rows

In [ ]:
display(dbutils.fs.ls("dbfs:/datasets/obo/tei/sessionsPapers"))

  

[TABLE]

Truncated to 30 rows

  

  

Step 1: Exploring data first: xml parsing in scala
--------------------------------------------------

But, first let's understand the data and its structure.

**Step 0: Dowloading and Loading Data (The Full Dataset)** should have
been done already with data in dbfs alread.

In [ ]:
val raw = sc.wholeTextFiles("dbfs:/datasets/obo/tei/ordinarysAccounts/OA17070912.xml")

  

>     raw: org.apache.spark.rdd.RDD[(String, String)] = dbfs:/datasets/obo/tei/ordinarysAccounts/OA17070912.xml MapPartitionsRDD[192] at wholeTextFiles at command-753740454082156:1

In [ ]:
val raw = sc.wholeTextFiles("dbfs:/datasets/obo/tei/sessionsPapers/17280717.xml") // has data on crimes and punishments

  

>     raw: org.apache.spark.rdd.RDD[(String, String)] = dbfs:/datasets/obo/tei/sessionsPapers/17280717.xml MapPartitionsRDD[194] at wholeTextFiles at command-753740454082157:1

In [ ]:
//val oboTest = sc.wholeTextFiles("dbfs:/datasets/obo/tei/ordinaryAccounts/OA1693072*.xml")
val xml = raw.map( x => x._2 )
val x = xml.take(1)(0) // getting content of xml file as a string

  

>     xml: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[195] at map at command-753740454082158:2
>     x: String =
>     <?xml version="1.0" encoding="UTF-8"?>
>     <TEI.2>
>        <text>
>           <body>
>              <div0 type="sessionsPaper" id="17280717">
>                 <interp inst="17280717" type="collection" value="BAILEY"/>
>                 <interp inst="17280717" type="year" value="1728"/>
>                 <interp inst="17280717" type="uri" value="sessionsPapers/17280717"/>
>                 <interp inst="17280717" type="date" value="17280717"/>
>                 <xptr type="transcription" doc="17280717"/>
>
>                 <div1 type="frontMatter" id="f17280717-1">
>                    <interp inst="f17280717-1" type="collection" value="BAILEY"/>
>                    <interp inst="f17280717-1" type="year" value="1728"/>
>                    <interp inst="f17280717-1" type="uri" value="sessionsPapers/17280717"/>
>                    <interp inst="f17280717-1" type="date" value="17280717"/>
>
>                    <xptr type="pageFacsimile" doc="172807170001"/>THE PROCEEDINGS AT THE Sessions of the Peace, and Oyer and Terminer for the City of LONDON: AND
>              <p>On the King's Commission of Goal-Delivery of Newgate, held at Justice-Hall in the Old Baily, for the CITY of LONDON and COUNTY of MIDDLESEX.</p>
>                    <p>On Wednesday, Thursday, and Friday, being the 17th, 18th, and 19th of July, 1728, in the Second Year of His MAJESTY's Reign.</p>
>                    <p>(Price Six Pence)</p>
>                    <p>BEFORE the Right Honourable Sir
>                       <persName id="f17280717-1-person1" type="judiciaryName">
>                          EDWARD
>                          BECHER
>                       <interp inst="f17280717-1-person1" type="surname" value="BECHER"/>
>                          <interp inst="f17280717-1-person1" type="given" value="EDWARD"/>
>                          <interp inst="f17280717-1-person1" type="gender" value="male"/>
>                       </persName>
>                    , Lord Mayor of the City of London; the Right Honourable the Lord Chief
>                       <persName id="f17280717-1-person2" type="judiciaryName">
>                          Baron
>                          Pengelly
>                       <interp inst="f17280717-1-person2" type="surname" value="Pengelly"/>
>                          <interp inst="f17280717-1-person2" type="given" value="Baron"/>
>                          <interp inst="f17280717-1-person2" type="gender" value="male"/>
>                       </persName>
>                    ; Mr. Justice Reynolds; Mr.
>                       <persName id="f17280717-1-person3" type="judiciaryName">
>                          Baron
>                          Thompson
>                       <interp inst="f17280717-1-person3" type="surname" value="Thompson"/>
>                          <interp inst="f17280717-1-person3" type="given" value="Baron"/>
>                          <interp inst="f17280717-1-person3" type="gender" value="male"/>
>                       </persName>
>                    , Recorder of the City of London; and
>                       <persName id="f17280717-1-person4" type="judiciaryName">
>                          John
>                          Raby
>                       <interp inst="f17280717-1-person4" type="surname" value="Raby"/>
>                          <interp inst="f17280717-1-person4" type="given" value="John"/>
>                          <interp inst="f17280717-1-person4" type="gender" value="male"/>
>                       </persName>
>                    , Esq; Serjeant at Law; and other His Majesty's Justices of Goal-Delivery, and Oyer and Terminer aforesaid; Together with several of His Majesty's Justices of the Peace for the said City of London.</p>
>
>                    <p>London Jury.</p>
>                    <p>
>
>                       <persName id="f17280717-1-person5" type="jurorName">
>                          John
>                          Land
>                       <interp inst="f17280717-1-person5" type="surname" value="Land"/>
>                          <interp inst="f17280717-1-person5" type="given" value="John"/>
>                          <interp inst="f17280717-1-person5" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person6" type="jurorName">
>                          Nathaniel
>                          Mason
>                       <interp inst="f17280717-1-person6" type="surname" value="Mason"/>
>                          <interp inst="f17280717-1-person6" type="given" value="Nathaniel"/>
>                          <interp inst="f17280717-1-person6" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person7" type="jurorName">
>                          Benjamin
>                          Allibone
>                       <interp inst="f17280717-1-person7" type="surname" value="Allibone"/>
>                          <interp inst="f17280717-1-person7" type="given" value="Benjamin"/>
>                          <interp inst="f17280717-1-person7" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person8" type="jurorName">
>                          Joseph
>                          Westwood
>                       <interp inst="f17280717-1-person8" type="surname" value="Westwood"/>
>                          <interp inst="f17280717-1-person8" type="given" value="Joseph"/>
>                          <interp inst="f17280717-1-person8" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person9" type="jurorName">
>                          Gabriel
>                          Wittacre
>                       <interp inst="f17280717-1-person9" type="surname" value="Wittacre"/>
>                          <interp inst="f17280717-1-person9" type="given" value="Gabriel"/>
>                          <interp inst="f17280717-1-person9" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person10" type="jurorName">
>                          Samuel
>                          Tilley
>                       <interp inst="f17280717-1-person10" type="surname" value="Tilley"/>
>                          <interp inst="f17280717-1-person10" type="given" value="Samuel"/>
>                          <interp inst="f17280717-1-person10" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person11" type="jurorName">
>                          Robert
>                          Lathwell
>                       <interp inst="f17280717-1-person11" type="surname" value="Lathwell"/>
>                          <interp inst="f17280717-1-person11" type="given" value="Robert"/>
>                          <interp inst="f17280717-1-person11" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person12" type="jurorName">
>                          Edward
>                          Newman
>                       <interp inst="f17280717-1-person12" type="surname" value="Newman"/>
>                          <interp inst="f17280717-1-person12" type="given" value="Edward"/>
>                          <interp inst="f17280717-1-person12" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person13" type="jurorName">
>                          Nathaniel
>                          Pickering
>                       <interp inst="f17280717-1-person13" type="surname" value="Pickering"/>
>                          <interp inst="f17280717-1-person13" type="given" value="Nathaniel"/>
>                          <interp inst="f17280717-1-person13" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person14" type="jurorName">
>                          Simon
>                          Tunks
>                       <interp inst="f17280717-1-person14" type="surname" value="Tunks"/>
>                          <interp inst="f17280717-1-person14" type="given" value="Simon"/>
>                          <interp inst="f17280717-1-person14" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person15" type="jurorName">
>                          Thomas
>                          Maud
>                       <interp inst="f17280717-1-person15" type="surname" value="Maud"/>
>                          <interp inst="f17280717-1-person15" type="given" value="Thomas"/>
>                          <interp inst="f17280717-1-person15" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person16" type="jurorName">
>                          John
>                          Bond
>                       <interp inst="f17280717-1-person16" type="surname" value="Bond"/>
>                          <interp inst="f17280717-1-person16" type="given" value="John"/>
>                          <interp inst="f17280717-1-person16" type="gender" value="male"/>
>                       </persName>
>                    .</p>
>
>
>                    <p>Middlesex Jury.</p>
>                    <p>
>
>                       <persName id="f17280717-1-person17" type="jurorName">
>                          Elisha
>                          Impey
>                       <interp inst="f17280717-1-person17" type="surname" value="Impey"/>
>                          <interp inst="f17280717-1-person17" type="given" value="Elisha"/>
>                          <interp inst="f17280717-1-person17" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person18" type="jurorName">
>                          Christopher
>                          Harris
>                       <interp inst="f17280717-1-person18" type="surname" value="Harris"/>
>                          <interp inst="f17280717-1-person18" type="given" value="Christopher"/>
>                          <interp inst="f17280717-1-person18" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person19" type="jurorName">
>                          William
>                          Perkins
>                       <interp inst="f17280717-1-person19" type="surname" value="Perkins"/>
>                          <interp inst="f17280717-1-person19" type="given" value="William"/>
>                          <interp inst="f17280717-1-person19" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person20" type="jurorName">
>                          Gilbert
>                          Watson
>                       <interp inst="f17280717-1-person20" type="surname" value="Watson"/>
>                          <interp inst="f17280717-1-person20" type="given" value="Gilbert"/>
>                          <interp inst="f17280717-1-person20" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person21" type="jurorName">
>                          John
>                          Wells
>                       <interp inst="f17280717-1-person21" type="surname" value="Wells"/>
>                          <interp inst="f17280717-1-person21" type="given" value="John"/>
>                          <interp inst="f17280717-1-person21" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person22" type="jurorName">
>                          William
>                          Carpenter
>                       <interp inst="f17280717-1-person22" type="surname" value="Carpenter"/>
>                          <interp inst="f17280717-1-person22" type="given" value="William"/>
>                          <interp inst="f17280717-1-person22" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person23" type="jurorName">
>                          Allen
>                          Evans
>                       <interp inst="f17280717-1-person23" type="surname" value="Evans"/>
>                          <interp inst="f17280717-1-person23" type="given" value="Allen"/>
>                          <interp inst="f17280717-1-person23" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person24" type="jurorName">
>                          Henry
>                          Cowmbe
>                       <interp inst="f17280717-1-person24" type="surname" value="Cowmbe"/>
>                          <interp inst="f17280717-1-person24" type="given" value="Henry"/>
>                          <interp inst="f17280717-1-person24" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person25" type="jurorName">
>                          Simon
>                          Parsons
>                       <interp inst="f17280717-1-person25" type="surname" value="Parsons"/>
>                          <interp inst="f17280717-1-person25" type="given" value="Simon"/>
>                          <interp inst="f17280717-1-person25" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person26" type="jurorName">
>                          George
>                          Gilbert
>                       <interp inst="f17280717-1-person26" type="surname" value="Gilbert"/>
>                          <interp inst="f17280717-1-person26" type="given" value="George"/>
>                          <interp inst="f17280717-1-person26" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person27" type="jurorName">
>                          Nicholas
>                          Gardner
>                       <interp inst="f17280717-1-person27" type="surname" value="Gardner"/>
>                          <interp inst="f17280717-1-person27" type="given" value="Nicholas"/>
>                          <interp inst="f17280717-1-person27" type="gender" value="male"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName id="f17280717-1-person28" type="jurorName">
>                          Thomas
>                          Ireland
>                       <interp inst="f17280717-1-person28" type="surname" value="Ireland"/>
>                          <interp inst="f17280717-1-person28" type="given" value="Thomas"/>
>                          <interp inst="f17280717-1-person28" type="gender" value="male"/>
>                       </persName>
>                    .</p>
>
>                 </div1>
>
>
>                 <div1 type="trialAccount" id="t17280717-1">
>                    <interp inst="t17280717-1" type="collection" value="BAILEY"/>
>                    <interp inst="t17280717-1" type="year" value="1728"/>
>                    <interp inst="t17280717-1" type="uri" value="sessionsPapers/17280717"/>
>                    <interp inst="t17280717-1" type="date" value="17280717"/>
>                    <join result="criminalCharge" id="t17280717-1-off2-c29" targOrder="Y"
>                          targets="t17280717-1-defend29 t17280717-1-off2 t17280717-1-verdict5"/>
>
>                    <p>
>
>                       <persName id="t17280717-1-defend29" type="defendantName">
>                       James
>                       Haddock
>                    <interp inst="t17280717-1-defend29" type="surname" value="Haddock"/>
>                          <interp inst="t17280717-1-defend29" type="given" value="James"/>
>                          <interp inst="t17280717-1-defend29" type="gender" value="male"/>
>                       </persName>
>                 , of <placeName id="t17280717-1-defloc1">St. Bennet's Paul's Wharf</placeName>
>                       <interp inst="t17280717-1-defloc1" type="placeName" value="St. Bennet's Paul's Wharf"/>
>                       <interp inst="t17280717-1-defloc1" type="type" value="defendantHome"/>
>                       <join result="persNamePlace" targOrder="Y"
>                             targets="t17280717-1-defend29 t17280717-1-defloc1"/>, was indicted for <rs id="t17280717-1-off2" type="offenceDescription">
>                          <interp inst="t17280717-1-off2" type="offenceCategory" value="theft"/>
>                          <interp inst="t17280717-1-off2" type="offenceSubcategory" value="theftFromPlace"/>
>                    feloniously stealing 2 Guineas, and 5 l. 16 s. in Silver, a Silver Cup, a Silver Cork Screw, 2 Silver Spoons, and a Nutmeg-grater, in the Dwelling-House of
>
>                             <persName id="t17280717-1-victim31" type="victimName">
>                                James
>                                Reeves
>                             <interp inst="t17280717-1-victim31" type="surname" value="Reeves"/>
>                             <interp inst="t17280717-1-victim31" type="given" value="James"/>
>                             <interp inst="t17280717-1-victim31" type="gender" value="male"/>
>                             <join result="offenceVictim" targOrder="Y"
>                                   targets="t17280717-1-off2 t17280717-1-victim31"/>
>                          </persName>
>
>
>
>                       </rs>, on the <rs id="t17280717-1-cd3" type="crimeDate">9th of April, and in the 13th Year of his late Majesty King George the First</rs>
>                       <join result="offenceCrimeDate" targOrder="Y"
>                             targets="t17280717-1-off2 t17280717-1-cd3"/>, the Property of
>                    <persName id="t17280717-1-person32">
>                       James
>                       Reeves
>                    <interp inst="t17280717-1-person32" type="surname" value="Reeves"/>
>                          <interp inst="t17280717-1-person32" type="given" value="James"/>
>                          <interp inst="t17280717-1-person32" type="gender" value="male"/>
>                       </persName>
>                  aforesaid.</p>
>                    <p>
>
>                       <persName id="t17280717-1-person33">
>                       Elizabeth
>                       Reeves
>                    <interp inst="t17280717-1-person33" type="surname" value="Reeves"/>
>                          <interp inst="t17280717-1-person33" type="given" value="Elizabeth"/>
>                          <interp inst="t17280717-1-person33" type="gender" value="female"/>
>                       </persName>
>                  depos'd, That the Prisoner was a Lodger at her House on <placeName id="t17280717-1-crimeloc4">Addle-Hill</placeName>
>                       <interp inst="t17280717-1-crimeloc4" type="placeName" value="Addle-Hill"/>
>                       <interp inst="t17280717-1-crimeloc4" type="type" value="crimeLocation"/>
>                       <join result="offencePlace" targOrder="Y"
>                             targets="t17280717-1-off2 t17280717-1-crimeloc4"/>, near Doctors Commons, when this Robbery was committed, and that it being on the Sabbath Day, she desired the Prisoner, if he did not go abroad, to have an Eye to her Room, which she locked up, and which, he promised to have an Eye to; but when she came home, the Chamber Door and the Corner-Cupboard had been forced open, which appeared by the Mark of an Instrument, the Hinges tore off and the Money gone, the Drawers rifled, and the Plate taken out, though some of the Drawers, out of which the Plate was taken, she left lock'd when she went from Home; upon which she cried out, saying, she was robb'd, and the Prisoner's Wife being above Stairs, came down, and said, her Husband was gone out, that he had been guilty of Failings, and desired her to be easy and she would Work early and late to make Satisfaction, though she did not know he had taken the Things, but his not coming home again confirmed them the more in this Suspicion; and there were other Witnesses, who depos'd, That the Prosecutor left the Prisoner in Care of her Door, and that he promised to look after it.</p>
>                    <p>The Prisoner said in his Defence, That Mrs. Reeves had lost a pair of Silver Buckles out of her Drawers a Fortnight before this Robbery, and she said, she did believe it was done by a Char-woman that she employ'd, which Mrs. Reeves acknowledging, and the Prisoner telling a plausible Story of his being in bad Circumstances, and that Day the Robbery was committed, one Diston pittying his Case, lent him Money to go down to Bristol, to Trade there, if possible to retrieve himself, and he not daring to go out of Doors in the Week Days, went out of the House unfortunately on that Afternoon: There being no positive Evidence against him, the Jury <rs id="t17280717-1-verdict5" type="verdictDescription">
>                          <interp inst="t17280717-1-verdict5" type="verdictCategory" value="notGuilty"/>
>                    acquitted
>                 </rs> him.</p>
>                 </div1>
>
>
>                 <div1 type="trialAccount" id="t17280717-2">
>                    <interp inst="t17280717-2" type="collection" value="BAILEY"/>
>                    <interp inst="t17280717-2" type="year" value="1728"/>
>                    <interp inst="t17280717-2" type="uri" value="sessionsPapers/17280717"/>
>                    <interp inst="t17280717-2" type="date" value="17280717"/>
>                    <join result="criminalCharge" id="t17280717-2-off6-c33" targOrder="Y"
>                          targets="t17280717-2-defend35 t17280717-2-off6 t17280717-2-verdict7"/>
>
>                    <p>
>
>
>                       <persName id="t17280717-2-defend35" type="defendantName">
>                          David
>                          Ball
>                       <interp inst="t17280717-2-defend35" type="surname" value="Ball"/>
>                          <interp inst="t17280717-2-defend35" type="given" value="David"/>
>                          <interp inst="t17280717-2-defend35" type="gender" value="male"/>
>                       </persName>
>
>                  was indicted for <rs id="t17280717-2-off6" type="offenceDescription">
>                          <interp inst="t17280717-2-off6" type="offenceCategory" value="theft"/>
>                          <interp inst="t17280717-2-off6" type="offenceSubcategory" value="pettyLarceny"/>
>                    a Petty Larceny, in stealing a Yard and a Quarter of Linnen Cloth, value 11 d.
>                 </rs> the Goods of
>
>                       <persName id="t17280717-2-victim37" type="victimName">
>                          John
>                          Williams
>                       <interp inst="t17280717-2-victim37" type="surname" value="Williams"/>
>                          <interp inst="t17280717-2-victim37" type="given" value="John"/>
>                          <interp inst="t17280717-2-victim37" type="gender" value="male"/>
>                       </persName>
>
>                  and
>
>                       <persName id="t17280717-2-victim39" type="victimName">
>                          William
>                          Williams
>                       <interp inst="t17280717-2-victim39" type="surname" value="Williams"/>
>                          <interp inst="t17280717-2-victim39" type="given" value="William"/>
>                          <interp inst="t17280717-2-victim39" type="gender" value="male"/>
>                       </persName>
>
>                 ; to which Indictment he <rs id="t17280717-2-verdict7" type="verdictDescription">
>                          <interp inst="t17280717-2-verdict7" type="verdictCategory" value="guilty"/>
>                          <interp inst="t17280717-2-verdict7" type="verdictSubcategory" value="pleadedGuilty"/>
>                    pleaded Guilty
>                 </rs>.</p>
>                    <p>
>                       <rs id="t17280717-2-punish8" type="punishmentDescription">
>                          <interp inst="t17280717-2-punish8" type="punishmentCategory" value="transport"/>
>                          <join result="defendantPunishment" targOrder="Y"
>                                targets="t17280717-2-defend35 t17280717-2-punish8"/>
>
>                          <note>[Transportation. See summary.]</note>
>
>                       </rs>
>                    </p>
>                 </div1>
>
>
>                 <div1 type="trialAccount" id="t17280717-3">
>                    <interp inst="t17280717-3" type="collection" value="BAILEY"/>
>                    <interp inst="t17280717-3" type="year" value="1728"/>
>                    <interp inst="t17280717-3" type="uri" value="sessionsPapers/17280717"/>
>                    <interp inst="t17280717-3" type="date" value="17280717"/>
>                    <join result="criminalCharge" id="t17280717-3-off10-c36" targOrder="Y"
>                          targets="t17280717-3-defend40 t17280717-3-off10 t17280717-3-verdict11"/>
>
>                    <p>
>                       <xptr type="pageFacsimile" doc="172807170002"/>
>
>                       <persName id="t17280717-3-defend40" type="defendantName">
>                       Nathaniel
>                       Mercy
>                    <interp inst="t17280717-3-defend40" type="surname" value="Mercy"/>
>                          <interp inst="t17280717-3-defend40" type="given" value="Nathaniel"/>
>                          <interp inst="t17280717-3-defend40" type="gender" value="male"/>
>                       </persName>
>                 , of <placeName id="t17280717-3-defloc9">St. James's Westminster</placeName>
>                       <interp inst="t17280717-3-defloc9" type="placeName" value="St. James's Westminster"/>
>                       <interp inst="t17280717-3-defloc9" type="type" value="defendantHome"/>
>                       <join result="persNamePlace" targOrder="Y"
>                             targets="t17280717-3-defend40 t17280717-3-defloc9"/>, was indicted for <rs id="t17280717-3-off10" type="offenceDescription">
>                          <interp inst="t17280717-3-off10" type="offenceCategory" value="theft"/>
>                          <interp inst="t17280717-3-off10" type="offenceSubcategory" value="grandLarceny"/>
>                    stealing a Coach Wheel, value 12 Shillings
>                 </rs>, the Goods of
>                    <persName id="t17280717-3-victim41" type="victimName">
>                       Thomas
>                       West
>                    <interp inst="t17280717-3-victim41" type="surname" value="West"/>
>                          <interp inst="t17280717-3-victim41" type="given" value="Thomas"/>
>                          <interp inst="t17280717-3-victim41" type="gender" value="male"/>
>                          <join result="offenceVictim" targOrder="Y"
>                                targets="t17280717-3-off10 t17280717-3-victim41"/>
>                       </persName>
>                  , on the 9th of this Instant, to which Indictment he <rs id="t17280717-3-verdict11" type="verdictDescription">
>                          <interp inst="t17280717-3-verdict11" type="verdictCategory" value="guilty"/>
>                          <interp inst="t17280717-3-verdict11" type="verdictSubcategory" value="pleadedGuilty"/>
>                    pleaded guilty
>                 </rs>.</p>
>                    <p>
>                       <rs id="t17280717-3-punish12" type="punishmentDescription">
>                          <interp inst="t17280717-3-punish12" type="punishmentCategory" value="transport"/>
>                          <join result="defendantPunishment" targOrder="Y"
>                                targets="t17280717-3-defend40 t17280717-3-punish12"/>
>
>                          <note>[Transportation. See summary.]</note>
>
>                       </rs>
>                    </p>
>                 </div1>
>
>
>                 <div1 type="trialAccount" id="t17280717-4">
>                    <interp inst="t17280717-4" type="collection" value="BAILEY"/>
>                    <interp inst="t17280717-4" type="year" value="1728"/>
>                    <interp inst="t17280717-4" type="uri" value="sessionsPapers/17280717"/>
>                    <interp inst="t17280717-4" type="date" value="17280717"/>
>                    <join result="criminalCharge" id="t17280717-4-off14-c38" targOrder="Y"
>                          targets="t17280717-4-defend42 t17280717-4-off14 t17280717-4-verdict17"/>
>
>                    <p>
>
>                       <persName id="t17280717-4-defend42" type="defendantName">
>                       Margaret
>                       King
>                    <interp inst="t17280717-4-defend42" type="surname" value="King"/>
>                          <interp inst="t17280717-4-defend42" type="given" value="Margaret"/>
>                          <interp inst="t17280717-4-defend42" type="gender" value="female"/>
>                       </persName>
>                 , of <placeName id="t17280717-4-defloc13">St. Ann's Westminster</placeName>
>                       <interp inst="t17280717-4-defloc13" type="placeName" value="St. Ann's Westminster"/>
>                       <interp inst="t17280717-4-defloc13" type="type" value="defendantHome"/>
>                       <join result="persNamePlace" targOrder="Y"
>                             targets="t17280717-4-defend42 t17280717-4-defloc13"/>, was indicted for <rs id="t17280717-4-off14" type="offenceDescription">
>                          <interp inst="t17280717-4-off14" type="offenceCategory" value="theft"/>
>                          <interp inst="t17280717-4-off14" type="offenceSubcategory" value="grandLarceny"/>
>                    privately stealing a Gold Watch, value 16 l.
>                 </rs> on the <rs id="t17280717-4-cd15" type="crimeDate">first of May</rs>
>                       <join result="offenceCrimeDate" targOrder="Y"
>                             targets="t17280717-4-off14 t17280717-4-cd15"/> last, the Property of
>                    <persName id="t17280717-4-victim43" type="victimName">
>                       Timothy
>                       Conner
>                    <interp inst="t17280717-4-victim43" type="surname" value="Conner"/>
>                          <interp inst="t17280717-4-victim43" type="given" value="Timothy"/>
>                          <interp inst="t17280717-4-victim43" type="gender" value="male"/>
>                          <join result="offenceVictim" targOrder="Y"
>                                targets="t17280717-4-off14 t17280717-4-victim43"/>
>                       </persName>
>                  .</p>
>                    <p>The Prosecutor depos'd, That he met the Prisoner in the Street, and ask'd her to drink a Glass of Wine, to which she consented, and they went to the <placeName id="t17280717-4-crimeloc16">Swan Tavern in Newport Market</placeName>
>                       <interp inst="t17280717-4-crimeloc16" type="placeName"
>                               value="Swan Tavern in Newport Market"/>
>                       <interp inst="t17280717-4-crimeloc16" type="type" value="crimeLocation"/>
>                       <join result="offencePlace" targOrder="Y"
>                             targets="t17280717-4-off14 t17280717-4-crimeloc16"/>, and lovingly drank 4 Pints, the Prisoner asking him what it was a Clock, he pulled out his Gold Watch, and bid her look; that she took it in her Hand, but could not remember that she returned it again, neither could she say positively, that she had it, but as he mis'd it as soon as he parted with her he thought he had occasion of Suspicion; yet, said he, I had been drinking before, (tho' it was but Nine in the Morning)and can't tell directly how the Matter stood; which being all the Evidence he could give against her, she was <rs id="t17280717-4-verdict17" type="verdictDescription">
>                          <interp inst="t17280717-4-verdict17" type="verdictCategory" value="notGuilty"/>
>                    acquitted
>                 </rs>.</p>
>                 </div1>
>
>
>                 <div1 type="trialAccount" id="t17280717-5">
>                    <interp inst="t17280717-5" type="collection" value="BAILEY"/>
>                    <interp inst="t17280717-5" type="year" value="1728"/>
>                    <interp inst="t17280717-5" type="uri" value="sessionsPapers/17280717"/>
>                    <interp inst="t17280717-5" type="date" value="17280717"/>
>                    <join result="criminalCharge" id="t17280717-5-off19-c40" targOrder="Y"
>                          targets="t17280717-5-defend44 t17280717-5-off19 t17280717-5-verdict22"/>
>
>                    <p>
>
>                       <persName id="t17280717-5-defend44" type="defendantName">
>                       Elizabeth
>                       Mould
>                    <interp inst="t17280717-5-defend44" type="surname" value="Mould"/>
>                          <interp inst="t17280717-5-defend44" type="given" value="Elizabeth"/>
>                          <interp inst="t17280717-5-defend44" type="gender" value="female"/>
>                       </persName>
>                 , of <placeName id="t17280717-5-defloc18">St. Martins in the Fields</placeName>
>                       <interp inst="t17280717-5-defloc18" type="placeName" value="St. Martins in the Fields"/>
>                       <interp inst="t17280717-5-defloc18" type="type" value="defendantHome"/>
>                       <join result="persNamePlace" targOrder="Y"
>                             targets="t17280717-5-defend44 t17280717-5-defloc18"/>, was indicted for <rs id="t17280717-5-off19" type="offenceDescription">
>                          <interp inst="t17280717-5-off19" type="offenceCategory" value="theft"/>
>                          <interp inst="t17280717-5-off19" type="offenceSubcategory" value="pocketpicking"/>
>                    privately stealing Fifty Pounds and eight Shillings, from the Person of
>                          <persName id="t17280717-5-victim45" type="victimName">
>                             John
>                             Coxall
>                          <interp inst="t17280717-5-victim45" type="surname" value="Coxall"/>
>                             <interp inst="t17280717-5-victim45" type="given" value="John"/>
>                             <interp inst="t17280717-5-victim45" type="gender" value="male"/>
>                             <join result="offenceVictim" targOrder="Y"
>                                   targets="t17280717-5-off19 t17280717-5-victim45"/>
>                          </persName>
>
>
>                       </rs>, on the <rs id="t17280717-5-cd20" type="crimeDate">24th of June</rs>
>                       <join result="offenceCrimeDate" targOrder="Y"
>                             targets="t17280717-5-off19 t17280717-5-cd20"/> last</p>
>                    <p>The Prosecutor depos'd, That he being a <rs id="t17280717-5-viclabel21" type="occupation">Bricklayer</rs>
>                       <join result="persNameOccupation" targOrder="Y"
>                             targets="t17280717-5-victim45 t17280717-5-viclabel21"/>, was endeavouring to get some Business to do, at the House where the Prisoner lived, and it being a Chandler's Shop, to obtain the Good-Will of the People, he call'd for several Drams, and treated the Mistress of the House and one Mrs. Greaves, who was his Customer, with Cyder, Brandy, &amp;c. that whilst they were drinking the Prisoner came into the Room, and he likewise treated her, and she, in return, wip'd him over the Face, and seem'd very fond of him, saying, she would give him a fine Nosegay, and something to cheer his Heart, if he would go with her to Covent-Garden Market, and then (as his Expression was) she weagled him down into the Cellar, and there kept him lock'd up in a back Passage, that he was very much in Liquor, and scarce sensible of what he did, but he found what she had done to his Sorrow, for she had taken all his Money, which he missing, made a Noise, and the People of the House hearing him, let him out of his Place of Confinement, by conducting him up the Back Stairs; that he got Officers and search'd the Cellar, but could not find the Prisoner. The Prisoner desiring he might be ask'd if they did not drink together in the Cellar, the Prosecutor answer'd, No, she would fetch no Drink, saying, she did not care to be seen by the Publicans; that she was all for dry Money, and she, and None but she, had his Fifty Pounds, for save only him and her, there were neither Man, Woman, nor Child, nor Dog nor Cat in the Cellar.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person46">
>                       Elizabeth
>                       Harper
>                    <interp inst="t17280717-5-person46" type="surname" value="Harper"/>
>                          <interp inst="t17280717-5-person46" type="given" value="Elizabeth"/>
>                          <interp inst="t17280717-5-person46" type="gender" value="female"/>
>                       </persName>
>                  depos'd, That the Prosecutor came into her Shop, and drank Cyder, Usquebaugh and Brandy, and being disguis'd in Liquor he pull'd out eight or ten Guineas and said, he was no Scoundrel; that she and Mrs. Greaves, whom he brought in to treat, begg'd he would put up his Money and take Care of it, and about that Time the Prisoner came into the Room, and familiarly stroaking his cheeks, persuaded him to go into the Cellar, saying, he should pay his Footing, that they staid half an Hour below, and this Deponent looking down, saw the Prisoner's Mother there, that he came up with her again, and treated her with Usquebaugh, and at 11 o' Clock, which was several Hours after, they heard him in a back Passage, where the House was supplied with Water, and letting him into their House, he said he was robb'd, at which this Deponent's Husband thrust him out of doors.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person47">
>                       Isabella
>                       Greaves
>                    <interp inst="t17280717-5-person47" type="surname" value="Greaves"/>
>                          <interp inst="t17280717-5-person47" type="given" value="Isabella"/>
>                          <interp inst="t17280717-5-person47" type="gender" value="female"/>
>                       </persName>
>                  likewise confirm'd every Part of this Deposition, adding, That the Prisoner was not to be seen that Night after the Robbery.</p>
>                    <p>The Prisoner said in her Defence, That the Prosecutor went down into her Cellar, and behaved himself so rudely, that she was forced to threaten to send for an Officer, that she knew nothing of his Money, and had not gone away that Night, but as she was oblig'd by her Husband, and that she came next Morning and set her Greens out.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person48">
>                       Anthony
>                       Dyer
>                    <interp inst="t17280717-5-person48" type="surname" value="Dyer"/>
>                          <interp inst="t17280717-5-person48" type="given" value="Anthony"/>
>                          <interp inst="t17280717-5-person48" type="gender" value="male"/>
>                       </persName>
>                  depos'd, That the Prosecutor told him he had lost his Money being Drunk, that he had been in a Cellar and in a Vault, where he fell asleep, and gave a very odd account of the Adventure.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person49">
>                       George
>                       Smith
>                    <interp inst="t17280717-5-person49" type="surname" value="Smith"/>
>                          <interp inst="t17280717-5-person49" type="given" value="George"/>
>                          <interp inst="t17280717-5-person49" type="gender" value="male"/>
>                       </persName>
>                  depos'd, That the Prosecutor said he fell asleep upon a Vault, and could charge no Body.</p>
>                    <p>The Constable, the Watchman, and others severally depos'd, That the Morning after this happened, he said he could charge no particular person, but he would indict the House. The Prisoner having a very good Character from several reputable Witnesses, the Jury <rs id="t17280717-5-verdict22" type="verdictDescription">
>                          <interp inst="t17280717-5-verdict22" type="verdictCategory" value="notGuilty"/>
>                    acquitted
>                 </rs> her.</p>
>                 </div1>
>
>
>                 <div1 type="trialAccount" id="t17280717-6">
>                    <interp inst="t17280717-6" type="collection" value="BAILEY"/>
>                    <interp inst="t17280717-6" type="year" value="1728"/>
>                    <interp inst="t17280717-6" type="uri" value="sessionsPapers/17280717"/>
>                    <interp inst="t17280717-6" type="date" value="17280717"/>
>                    <join result="criminalCharge" id="t17280717-6-off24-c46" targOrder="Y"
>                          targets="t17280717-6-defend50 t17280717-6-off24 t17280717-6-verdict26"/>
>
>                    <p>
>
>                       <persName id="t17280717-6-defend50" type="defendantName">
>                       Phillip
>                       Hilliard
>                    <interp inst="t17280717-6-defend50" type="surname" value="Hilliard"/>
>                          <interp inst="t17280717-6-defend50" type="given" value="Phillip"/>
>                          <interp inst="t17280717-6-defend50" type="gender" value="male"/>
>                       </persName>
>                 , of <placeName id="t17280717-6-defloc23">St. James's Westminster</placeName>
>                       <interp inst="t17280717-6-defloc23" type="placeName" value="St. James's Westminster"/>
>                       <interp inst="t17280717-6-defloc23" type="type" value="defendantHome"/>
>                       <join result="persNamePlace" targOrder="Y"
>                             targets="t17280717-6-defend50 t17280717-6-defloc23"/>, was indicted for <rs id="t17280717-6-off24" type="offenceDescription">
>                          <interp inst="t17280717-6-off24" type="offenceCategory" value="theft"/>
>                          <interp inst="t17280717-6-off24" type="offenceSubcategory" value="grandLarceny"/>
>                    feloniously stealing a Silver Spoon, value eleven Shillings, the Property of
>                          <persName id="t17280717-6-victim51" type="victimName">
>                             Abraham
>                             Mannio
>                          <interp inst="t17280717-6-victim51" type="surname" value="Mannio"/>
>                             <interp inst="t17280717-6-victim51" type="given" value="Abraham"/>
>                             <interp inst="t17280717-6-victim51" type="gender" value="male"/>
>                             <join result="offenceVictim" targOrder="Y"
>                                   targets="t17280717-6-off24 t17280717-6-victim51"/>
>                          </persName>
>                        , on the <rs id="t17280717-6-cd25" type="crimeDate">27th of June</rs>
>                          <join result="offenceCrimeDate" targOrder="Y"
>                                targets="t17280717-6-off24 t17280717-6-cd25"/> last.
>                 </rs>
>                    </p>
>                    <p>Mr. Hayden depos'd, That the Prisoner brought the Spoon to him to pawn, and he suspecting it to be stolen, stopp'd him and carried him to the Round-House, where he confess'd he stole it at the Prosecutor's, he being invited there to Dinner with some Gentlemans Servants.</p>
>                    <p>
>
>                       <persName id="t17280717-6-person52">
>                       Robert
>                       Amey
>                    <interp inst="t17280717-6-person52" type="surname" value="Amey"/>
>                          <interp inst="t17280717-6-person52" type="given" value="Robert"/>
>                          <interp inst="t17280717-6-person52" type="gender" value="male"/>
>                       </persName>
>                  depos'd, That he attended the Gentlemens Servants at Dinner, and afterwards they miss'd a Spoon, that Mr. Haydon sent them word of the Spoon brought to him by the Prisoner, and he going to match it by the others, found it to be the same which they had lost, it being of the same Make and Mark: The Fact being thus plainly proved upon him, the Jury found him <rs id="t17280717-6-verdict26" type="verdictDescription">
>                          <interp inst="t17280717-6-verdict26" type="verdictCategory" value="guilty"/>
>                          <interp inst="t17280717-6-verdict26" type="verdictSubcategory" value="theftunder1s"/>
>                    guilty to the value of 10d.
>                 </rs>
>                    </p>
>                    <p>
>                       <rs id="t17280717-6-punish27" type="punishmentDescription">
>                          <interp inst="t17280717-6-punish27" type="punishmentCategory" value="transport"/>
>                          <join result="defendantPunishment" targOrder="Y"
>                                targets="t17280717-6-defend50 t17280717-6-punish27"/>
>
>                          <note>[Transportation. See summary.]</note>
>
>                       </rs>
>                    </p>
>                 </div1>
>
>
>                 <div1 type="trialAccount" id="t17280717-7">
>                    <interp inst="t17280717-7" type="collection" value="BAILEY"/>
>                    <interp inst="t17280717-7" type="year" value="1728"/>
>                    <interp inst="t17280717-7" type="uri" value="sessionsPapers/17280717"/>
>                    <interp inst="t17280717-7" type="date" value="17280717"/>
>                    <join result="criminalCharge" id="t17280717-7-off29-c49" targOrder="Y"
>                          targets="t17280717-7-defend53 t17280717-7-off29 t17280717-7-verdict31"/>
>
>                    <p>
>
>                       <persName id="t17280717-7-defend53" type="defendantName">
>                       Robert
>                       Ashby
>                    <interp inst="t17280717-7-defend53" type="surname" value="Ashby"/>
>                          <interp inst="t17280717-7-defend53" type="given" value="Robert"/>
>                          <interp inst="t17280717-7-defend53" type="gender" value="male"/>
>                       </persName>
>                 , of <rs id="t17280717-7-deflabel28" type="occupation">St. Andrew's Holborn</rs>
>                       <join result="persNameOccupation" targOrder="Y"
>                             targets="t17280717-7-defend53 t17280717-7-deflabel28"/>, was indicted for <rs id="t17280717-7-off29" type="offenceDescription">
>                          <interp inst="t17280717-7-off29" type="offenceCategory" value="theft"/>
>                          <interp inst="t17280717-7-off29" type="offenceSubcategory" value="grandLarceny"/>
>                    stealing a Gold Watch, value 16 l. a Chain, value 4 l. a Cornelian Seal set in Gold, value 10s. the Property of
>                          <persName id="t17280717-7-victim54" type="victimName">
>                             Nicholas
>                             Roper
>                          <interp inst="t17280717-7-victim54" type="surname" value="Roper"/>
>                             <interp inst="t17280717-7-victim54" type="given" value="Nicholas"/>
>                             <interp inst="t17280717-7-victim54" type="gender" value="male"/>
>                          </persName>
>                         from the Person of
>                          <persName id="t17280717-7-victim55" type="victimName">
>                             Phoebe
>                             Thickpenny
>                          <interp inst="t17280717-7-victim55" type="surname" value="Thickpenny"/>
>                             <interp inst="t17280717-7-victim55" type="given" value="Phoebe"/>
>                             <interp inst="t17280717-7-victim55" type="gender" value="female"/>
>                          </persName>
>
>
>                       </rs>, who depos'd, That her Mistress being at Little Chelsea, sent her to their House in Lad-lane, for the Watch and other Things she had occasion for, that she call'd at the Prisoner's in Castle Yard, Chick-Lane, as she was going towards Chelsea with the Watch and the Bundle, where she drank part of a Pint of Beer, some Tea 2 Quarterns of Brandy, and a Bottle of Cyder; that the Prisoner would go part of the Way home with her, and in <placeName id="t17280717-7-crimeloc30">Leather-Lane</placeName>
>                       <interp inst="t17280717-7-crimeloc30" type="placeName" value="Leather-Lane"/>
>                       <interp inst="t17280717-7-crimeloc30" type="type" value="crimeLocation"/>
>                       <join result="offencePlace" targOrder="Y"
>                             targets="t17280717-7-off29 t17280717-7-crimeloc30"/>, he said, faith they would not part Dry lips, and accordingly they went into a publick House and drank a Pint of Twopenny, and two Quarterns of Brandy, that she had the Watch then, and at the Door the Prisoner kiss'd her, and gave her a shilling for a Coach, she having out-staid her Time; that when he kiss'd her, he put one Hand around her Waist, but what he did with the other she could not tell, that she then cross'd the Way to another House, and immediately miss'd the Watch, and she was sure she had pinn'd it so to her Side, that she could not drop it.</p>
>                    <p>
>
>                       <persName id="t17280717-7-person56">
>                       Margaret
>                       Nelson
>                    <interp inst="t17280717-7-person56" type="surname" value="Nelson"/>
>                          <interp inst="t17280717-7-person56" type="given" value="Margaret"/>
>                          <interp inst="t17280717-7-person56" type="gender" value="female"/>
>                       </persName>
>                  depos'd, That the Prisoner and Phoebe Thickpenny, came to her House in Leather-Lane, and he call'd for a private Room, to which the Girl would not consent, that the Girl wanted to go to, &amp;c. and she went with her, when the Girl said, she had her Mistress's Gold Watch, and seeming to look on it, told her it was 7 o' Clock, but this Deponent did not see the Watch, yet she said, she did verily believe she heard it beat, that they soon parted, and in a Quarter of an Hour the maid returned, and said she had lost the Watch.</p>
>                    <p>The Prisoner said in his Defence, That he knew nothing of it, any further than she said it was her Mistress's, that they parted very good Friends, and she having been an old Sweetheart of his, laid her Head upon his Shoulder, and said, She could live and die there, which would have been no little Aggravation to his Crime, had he wrong'd so good Natur'd a Creature: But the Evidence against him being weak, and several appearing to his Character, the Jury <rs id="t17280717-7-verdict31" type="verdictDescription">
>                          <interp inst="t17280717-7-verdict31" type="verdictCategory" value="notGuilty"/>
>                    acquitted
>                 </rs> him. </p>
>                 </div1>
>
>
>                 <div1 type="trialAccount" id="t17280717-8">
>                    <interp inst="t17280717-8" type="collection" value="BAILEY"/>
>                    <interp inst="t17280717-8" type="year" value="1728"/>
>                    <interp inst="t17280717-8" type="uri" value="sessionsPapers/17280717"/>
>                    <interp inst="t17280717-8" type="date" value="17280717"/>
>                    <join result="criminalCharge" id="t17280717-8-off33-c53" targOrder="Y"
>                          targets="t17280717-8-defend57 t17280717-8-off33 t17280717-8-verdict35"/>
>                    <join result="criminalCharge" id="t17280717-8-off33-c54" targOrder="Y"
>                          targets="t17280717-8-defend58 t17280717-8-off33 t17280717-8-verdict35"/>
>
>                    <p>
>
>                       <persName id="t17280717-8-defend57...

In [ ]:
val elem = scala.xml.XML.loadString(x)

  

>     elem: scala.xml.Elem =
>     <TEI.2>
>        <text>
>           <body>
>              <div0 id="17280717" type="sessionsPaper">
>                 <interp value="BAILEY" type="collection" inst="17280717"/>
>                 <interp value="1728" type="year" inst="17280717"/>
>                 <interp value="sessionsPapers/17280717" type="uri" inst="17280717"/>
>                 <interp value="17280717" type="date" inst="17280717"/>
>                 <xptr doc="17280717" type="transcription"/>
>
>                 <div1 id="f17280717-1" type="frontMatter">
>                    <interp value="BAILEY" type="collection" inst="f17280717-1"/>
>                    <interp value="1728" type="year" inst="f17280717-1"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="f17280717-1"/>
>                    <interp value="17280717" type="date" inst="f17280717-1"/>
>
>                    <xptr doc="172807170001" type="pageFacsimile"/>THE PROCEEDINGS AT THE Sessions of the Peace, and Oyer and Terminer for the City of LONDON: AND
>              <p>On the King's Commission of Goal-Delivery of Newgate, held at Justice-Hall in the Old Baily, for the CITY of LONDON and COUNTY of MIDDLESEX.</p>
>                    <p>On Wednesday, Thursday, and Friday, being the 17th, 18th, and 19th of July, 1728, in the Second Year of His MAJESTY's Reign.</p>
>                    <p>(Price Six Pence)</p>
>                    <p>BEFORE the Right Honourable Sir
>                       <persName type="judiciaryName" id="f17280717-1-person1">
>                          EDWARD
>                          BECHER
>                       <interp value="BECHER" type="surname" inst="f17280717-1-person1"/>
>                          <interp value="EDWARD" type="given" inst="f17280717-1-person1"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person1"/>
>                       </persName>
>                    , Lord Mayor of the City of London; the Right Honourable the Lord Chief
>                       <persName type="judiciaryName" id="f17280717-1-person2">
>                          Baron
>                          Pengelly
>                       <interp value="Pengelly" type="surname" inst="f17280717-1-person2"/>
>                          <interp value="Baron" type="given" inst="f17280717-1-person2"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person2"/>
>                       </persName>
>                    ; Mr. Justice Reynolds; Mr.
>                       <persName type="judiciaryName" id="f17280717-1-person3">
>                          Baron
>                          Thompson
>                       <interp value="Thompson" type="surname" inst="f17280717-1-person3"/>
>                          <interp value="Baron" type="given" inst="f17280717-1-person3"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person3"/>
>                       </persName>
>                    , Recorder of the City of London; and
>                       <persName type="judiciaryName" id="f17280717-1-person4">
>                          John
>                          Raby
>                       <interp value="Raby" type="surname" inst="f17280717-1-person4"/>
>                          <interp value="John" type="given" inst="f17280717-1-person4"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person4"/>
>                       </persName>
>                    , Esq; Serjeant at Law; and other His Majesty's Justices of Goal-Delivery, and Oyer and Terminer aforesaid; Together with several of His Majesty's Justices of the Peace for the said City of London.</p>
>
>                    <p>London Jury.</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person5">
>                          John
>                          Land
>                       <interp value="Land" type="surname" inst="f17280717-1-person5"/>
>                          <interp value="John" type="given" inst="f17280717-1-person5"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person5"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person6">
>                          Nathaniel
>                          Mason
>                       <interp value="Mason" type="surname" inst="f17280717-1-person6"/>
>                          <interp value="Nathaniel" type="given" inst="f17280717-1-person6"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person6"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person7">
>                          Benjamin
>                          Allibone
>                       <interp value="Allibone" type="surname" inst="f17280717-1-person7"/>
>                          <interp value="Benjamin" type="given" inst="f17280717-1-person7"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person7"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person8">
>                          Joseph
>                          Westwood
>                       <interp value="Westwood" type="surname" inst="f17280717-1-person8"/>
>                          <interp value="Joseph" type="given" inst="f17280717-1-person8"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person8"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person9">
>                          Gabriel
>                          Wittacre
>                       <interp value="Wittacre" type="surname" inst="f17280717-1-person9"/>
>                          <interp value="Gabriel" type="given" inst="f17280717-1-person9"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person9"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person10">
>                          Samuel
>                          Tilley
>                       <interp value="Tilley" type="surname" inst="f17280717-1-person10"/>
>                          <interp value="Samuel" type="given" inst="f17280717-1-person10"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person10"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person11">
>                          Robert
>                          Lathwell
>                       <interp value="Lathwell" type="surname" inst="f17280717-1-person11"/>
>                          <interp value="Robert" type="given" inst="f17280717-1-person11"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person11"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person12">
>                          Edward
>                          Newman
>                       <interp value="Newman" type="surname" inst="f17280717-1-person12"/>
>                          <interp value="Edward" type="given" inst="f17280717-1-person12"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person12"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person13">
>                          Nathaniel
>                          Pickering
>                       <interp value="Pickering" type="surname" inst="f17280717-1-person13"/>
>                          <interp value="Nathaniel" type="given" inst="f17280717-1-person13"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person13"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person14">
>                          Simon
>                          Tunks
>                       <interp value="Tunks" type="surname" inst="f17280717-1-person14"/>
>                          <interp value="Simon" type="given" inst="f17280717-1-person14"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person14"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person15">
>                          Thomas
>                          Maud
>                       <interp value="Maud" type="surname" inst="f17280717-1-person15"/>
>                          <interp value="Thomas" type="given" inst="f17280717-1-person15"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person15"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person16">
>                          John
>                          Bond
>                       <interp value="Bond" type="surname" inst="f17280717-1-person16"/>
>                          <interp value="John" type="given" inst="f17280717-1-person16"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person16"/>
>                       </persName>
>                    .</p>
>
>
>                    <p>Middlesex Jury.</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person17">
>                          Elisha
>                          Impey
>                       <interp value="Impey" type="surname" inst="f17280717-1-person17"/>
>                          <interp value="Elisha" type="given" inst="f17280717-1-person17"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person17"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person18">
>                          Christopher
>                          Harris
>                       <interp value="Harris" type="surname" inst="f17280717-1-person18"/>
>                          <interp value="Christopher" type="given" inst="f17280717-1-person18"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person18"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person19">
>                          William
>                          Perkins
>                       <interp value="Perkins" type="surname" inst="f17280717-1-person19"/>
>                          <interp value="William" type="given" inst="f17280717-1-person19"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person19"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person20">
>                          Gilbert
>                          Watson
>                       <interp value="Watson" type="surname" inst="f17280717-1-person20"/>
>                          <interp value="Gilbert" type="given" inst="f17280717-1-person20"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person20"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person21">
>                          John
>                          Wells
>                       <interp value="Wells" type="surname" inst="f17280717-1-person21"/>
>                          <interp value="John" type="given" inst="f17280717-1-person21"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person21"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person22">
>                          William
>                          Carpenter
>                       <interp value="Carpenter" type="surname" inst="f17280717-1-person22"/>
>                          <interp value="William" type="given" inst="f17280717-1-person22"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person22"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person23">
>                          Allen
>                          Evans
>                       <interp value="Evans" type="surname" inst="f17280717-1-person23"/>
>                          <interp value="Allen" type="given" inst="f17280717-1-person23"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person23"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person24">
>                          Henry
>                          Cowmbe
>                       <interp value="Cowmbe" type="surname" inst="f17280717-1-person24"/>
>                          <interp value="Henry" type="given" inst="f17280717-1-person24"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person24"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person25">
>                          Simon
>                          Parsons
>                       <interp value="Parsons" type="surname" inst="f17280717-1-person25"/>
>                          <interp value="Simon" type="given" inst="f17280717-1-person25"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person25"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person26">
>                          George
>                          Gilbert
>                       <interp value="Gilbert" type="surname" inst="f17280717-1-person26"/>
>                          <interp value="George" type="given" inst="f17280717-1-person26"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person26"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person27">
>                          Nicholas
>                          Gardner
>                       <interp value="Gardner" type="surname" inst="f17280717-1-person27"/>
>                          <interp value="Nicholas" type="given" inst="f17280717-1-person27"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person27"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person28">
>                          Thomas
>                          Ireland
>                       <interp value="Ireland" type="surname" inst="f17280717-1-person28"/>
>                          <interp value="Thomas" type="given" inst="f17280717-1-person28"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person28"/>
>                       </persName>
>                    .</p>
>
>                 </div1>
>
>
>                 <div1 id="t17280717-1" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-1"/>
>                    <interp value="1728" type="year" inst="t17280717-1"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-1"/>
>                    <interp value="17280717" type="date" inst="t17280717-1"/>
>                    <join targets="t17280717-1-defend29 t17280717-1-off2 t17280717-1-verdict5" targOrder="Y" id="t17280717-1-off2-c29" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-1-defend29">
>                       James
>                       Haddock
>                    <interp value="Haddock" type="surname" inst="t17280717-1-defend29"/>
>                          <interp value="James" type="given" inst="t17280717-1-defend29"/>
>                          <interp value="male" type="gender" inst="t17280717-1-defend29"/>
>                       </persName>
>                 , of <placeName id="t17280717-1-defloc1">St. Bennet's Paul's Wharf</placeName>
>                       <interp value="St. Bennet's Paul's Wharf" type="placeName" inst="t17280717-1-defloc1"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-1-defloc1"/>
>                       <join targets="t17280717-1-defend29 t17280717-1-defloc1" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-1-off2">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-1-off2"/>
>                          <interp value="theftFromPlace" type="offenceSubcategory" inst="t17280717-1-off2"/>
>                    feloniously stealing 2 Guineas, and 5 l. 16 s. in Silver, a Silver Cup, a Silver Cork Screw, 2 Silver Spoons, and a Nutmeg-grater, in the Dwelling-House of
>
>                             <persName type="victimName" id="t17280717-1-victim31">
>                                James
>                                Reeves
>                             <interp value="Reeves" type="surname" inst="t17280717-1-victim31"/>
>                             <interp value="James" type="given" inst="t17280717-1-victim31"/>
>                             <interp value="male" type="gender" inst="t17280717-1-victim31"/>
>                             <join targets="t17280717-1-off2 t17280717-1-victim31" targOrder="Y" result="offenceVictim"/>
>                          </persName>
>
>
>
>                       </rs>, on the <rs type="crimeDate" id="t17280717-1-cd3">9th of April, and in the 13th Year of his late Majesty King George the First</rs>
>                       <join targets="t17280717-1-off2 t17280717-1-cd3" targOrder="Y" result="offenceCrimeDate"/>, the Property of
>                    <persName id="t17280717-1-person32">
>                       James
>                       Reeves
>                    <interp value="Reeves" type="surname" inst="t17280717-1-person32"/>
>                          <interp value="James" type="given" inst="t17280717-1-person32"/>
>                          <interp value="male" type="gender" inst="t17280717-1-person32"/>
>                       </persName>
>                  aforesaid.</p>
>                    <p>
>
>                       <persName id="t17280717-1-person33">
>                       Elizabeth
>                       Reeves
>                    <interp value="Reeves" type="surname" inst="t17280717-1-person33"/>
>                          <interp value="Elizabeth" type="given" inst="t17280717-1-person33"/>
>                          <interp value="female" type="gender" inst="t17280717-1-person33"/>
>                       </persName>
>                  depos'd, That the Prisoner was a Lodger at her House on <placeName id="t17280717-1-crimeloc4">Addle-Hill</placeName>
>                       <interp value="Addle-Hill" type="placeName" inst="t17280717-1-crimeloc4"/>
>                       <interp value="crimeLocation" type="type" inst="t17280717-1-crimeloc4"/>
>                       <join targets="t17280717-1-off2 t17280717-1-crimeloc4" targOrder="Y" result="offencePlace"/>, near Doctors Commons, when this Robbery was committed, and that it being on the Sabbath Day, she desired the Prisoner, if he did not go abroad, to have an Eye to her Room, which she locked up, and which, he promised to have an Eye to; but when she came home, the Chamber Door and the Corner-Cupboard had been forced open, which appeared by the Mark of an Instrument, the Hinges tore off and the Money gone, the Drawers rifled, and the Plate taken out, though some of the Drawers, out of which the Plate was taken, she left lock'd when she went from Home; upon which she cried out, saying, she was robb'd, and the Prisoner's Wife being above Stairs, came down, and said, her Husband was gone out, that he had been guilty of Failings, and desired her to be easy and she would Work early and late to make Satisfaction, though she did not know he had taken the Things, but his not coming home again confirmed them the more in this Suspicion; and there were other Witnesses, who depos'd, That the Prosecutor left the Prisoner in Care of her Door, and that he promised to look after it.</p>
>                    <p>The Prisoner said in his Defence, That Mrs. Reeves had lost a pair of Silver Buckles out of her Drawers a Fortnight before this Robbery, and she said, she did believe it was done by a Char-woman that she employ'd, which Mrs. Reeves acknowledging, and the Prisoner telling a plausible Story of his being in bad Circumstances, and that Day the Robbery was committed, one Diston pittying his Case, lent him Money to go down to Bristol, to Trade there, if possible to retrieve himself, and he not daring to go out of Doors in the Week Days, went out of the House unfortunately on that Afternoon: There being no positive Evidence against him, the Jury <rs type="verdictDescription" id="t17280717-1-verdict5">
>                          <interp value="notGuilty" type="verdictCategory" inst="t17280717-1-verdict5"/>
>                    acquitted
>                 </rs> him.</p>
>                 </div1>
>
>
>                 <div1 id="t17280717-2" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-2"/>
>                    <interp value="1728" type="year" inst="t17280717-2"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-2"/>
>                    <interp value="17280717" type="date" inst="t17280717-2"/>
>                    <join targets="t17280717-2-defend35 t17280717-2-off6 t17280717-2-verdict7" targOrder="Y" id="t17280717-2-off6-c33" result="criminalCharge"/>
>
>                    <p>
>
>
>                       <persName type="defendantName" id="t17280717-2-defend35">
>                          David
>                          Ball
>                       <interp value="Ball" type="surname" inst="t17280717-2-defend35"/>
>                          <interp value="David" type="given" inst="t17280717-2-defend35"/>
>                          <interp value="male" type="gender" inst="t17280717-2-defend35"/>
>                       </persName>
>
>                  was indicted for <rs type="offenceDescription" id="t17280717-2-off6">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-2-off6"/>
>                          <interp value="pettyLarceny" type="offenceSubcategory" inst="t17280717-2-off6"/>
>                    a Petty Larceny, in stealing a Yard and a Quarter of Linnen Cloth, value 11 d.
>                 </rs> the Goods of
>
>                       <persName type="victimName" id="t17280717-2-victim37">
>                          John
>                          Williams
>                       <interp value="Williams" type="surname" inst="t17280717-2-victim37"/>
>                          <interp value="John" type="given" inst="t17280717-2-victim37"/>
>                          <interp value="male" type="gender" inst="t17280717-2-victim37"/>
>                       </persName>
>
>                  and
>
>                       <persName type="victimName" id="t17280717-2-victim39">
>                          William
>                          Williams
>                       <interp value="Williams" type="surname" inst="t17280717-2-victim39"/>
>                          <interp value="William" type="given" inst="t17280717-2-victim39"/>
>                          <interp value="male" type="gender" inst="t17280717-2-victim39"/>
>                       </persName>
>
>                 ; to which Indictment he <rs type="verdictDescription" id="t17280717-2-verdict7">
>                          <interp value="guilty" type="verdictCategory" inst="t17280717-2-verdict7"/>
>                          <interp value="pleadedGuilty" type="verdictSubcategory" inst="t17280717-2-verdict7"/>
>                    pleaded Guilty
>                 </rs>.</p>
>                    <p>
>                       <rs type="punishmentDescription" id="t17280717-2-punish8">
>                          <interp value="transport" type="punishmentCategory" inst="t17280717-2-punish8"/>
>                          <join targets="t17280717-2-defend35 t17280717-2-punish8" targOrder="Y" result="defendantPunishment"/>
>
>                          <note>[Transportation. See summary.]</note>
>
>                       </rs>
>                    </p>
>                 </div1>
>
>
>                 <div1 id="t17280717-3" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-3"/>
>                    <interp value="1728" type="year" inst="t17280717-3"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-3"/>
>                    <interp value="17280717" type="date" inst="t17280717-3"/>
>                    <join targets="t17280717-3-defend40 t17280717-3-off10 t17280717-3-verdict11" targOrder="Y" id="t17280717-3-off10-c36" result="criminalCharge"/>
>
>                    <p>
>                       <xptr doc="172807170002" type="pageFacsimile"/>
>
>                       <persName type="defendantName" id="t17280717-3-defend40">
>                       Nathaniel
>                       Mercy
>                    <interp value="Mercy" type="surname" inst="t17280717-3-defend40"/>
>                          <interp value="Nathaniel" type="given" inst="t17280717-3-defend40"/>
>                          <interp value="male" type="gender" inst="t17280717-3-defend40"/>
>                       </persName>
>                 , of <placeName id="t17280717-3-defloc9">St. James's Westminster</placeName>
>                       <interp value="St. James's Westminster" type="placeName" inst="t17280717-3-defloc9"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-3-defloc9"/>
>                       <join targets="t17280717-3-defend40 t17280717-3-defloc9" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-3-off10">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-3-off10"/>
>                          <interp value="grandLarceny" type="offenceSubcategory" inst="t17280717-3-off10"/>
>                    stealing a Coach Wheel, value 12 Shillings
>                 </rs>, the Goods of
>                    <persName type="victimName" id="t17280717-3-victim41">
>                       Thomas
>                       West
>                    <interp value="West" type="surname" inst="t17280717-3-victim41"/>
>                          <interp value="Thomas" type="given" inst="t17280717-3-victim41"/>
>                          <interp value="male" type="gender" inst="t17280717-3-victim41"/>
>                          <join targets="t17280717-3-off10 t17280717-3-victim41" targOrder="Y" result="offenceVictim"/>
>                       </persName>
>                  , on the 9th of this Instant, to which Indictment he <rs type="verdictDescription" id="t17280717-3-verdict11">
>                          <interp value="guilty" type="verdictCategory" inst="t17280717-3-verdict11"/>
>                          <interp value="pleadedGuilty" type="verdictSubcategory" inst="t17280717-3-verdict11"/>
>                    pleaded guilty
>                 </rs>.</p>
>                    <p>
>                       <rs type="punishmentDescription" id="t17280717-3-punish12">
>                          <interp value="transport" type="punishmentCategory" inst="t17280717-3-punish12"/>
>                          <join targets="t17280717-3-defend40 t17280717-3-punish12" targOrder="Y" result="defendantPunishment"/>
>
>                          <note>[Transportation. See summary.]</note>
>
>                       </rs>
>                    </p>
>                 </div1>
>
>
>                 <div1 id="t17280717-4" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-4"/>
>                    <interp value="1728" type="year" inst="t17280717-4"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-4"/>
>                    <interp value="17280717" type="date" inst="t17280717-4"/>
>                    <join targets="t17280717-4-defend42 t17280717-4-off14 t17280717-4-verdict17" targOrder="Y" id="t17280717-4-off14-c38" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-4-defend42">
>                       Margaret
>                       King
>                    <interp value="King" type="surname" inst="t17280717-4-defend42"/>
>                          <interp value="Margaret" type="given" inst="t17280717-4-defend42"/>
>                          <interp value="female" type="gender" inst="t17280717-4-defend42"/>
>                       </persName>
>                 , of <placeName id="t17280717-4-defloc13">St. Ann's Westminster</placeName>
>                       <interp value="St. Ann's Westminster" type="placeName" inst="t17280717-4-defloc13"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-4-defloc13"/>
>                       <join targets="t17280717-4-defend42 t17280717-4-defloc13" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-4-off14">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-4-off14"/>
>                          <interp value="grandLarceny" type="offenceSubcategory" inst="t17280717-4-off14"/>
>                    privately stealing a Gold Watch, value 16 l.
>                 </rs> on the <rs type="crimeDate" id="t17280717-4-cd15">first of May</rs>
>                       <join targets="t17280717-4-off14 t17280717-4-cd15" targOrder="Y" result="offenceCrimeDate"/> last, the Property of
>                    <persName type="victimName" id="t17280717-4-victim43">
>                       Timothy
>                       Conner
>                    <interp value="Conner" type="surname" inst="t17280717-4-victim43"/>
>                          <interp value="Timothy" type="given" inst="t17280717-4-victim43"/>
>                          <interp value="male" type="gender" inst="t17280717-4-victim43"/>
>                          <join targets="t17280717-4-off14 t17280717-4-victim43" targOrder="Y" result="offenceVictim"/>
>                       </persName>
>                  .</p>
>                    <p>The Prosecutor depos'd, That he met the Prisoner in the Street, and ask'd her to drink a Glass of Wine, to which she consented, and they went to the <placeName id="t17280717-4-crimeloc16">Swan Tavern in Newport Market</placeName>
>                       <interp value="Swan Tavern in Newport Market" type="placeName" inst="t17280717-4-crimeloc16"/>
>                       <interp value="crimeLocation" type="type" inst="t17280717-4-crimeloc16"/>
>                       <join targets="t17280717-4-off14 t17280717-4-crimeloc16" targOrder="Y" result="offencePlace"/>, and lovingly drank 4 Pints, the Prisoner asking him what it was a Clock, he pulled out his Gold Watch, and bid her look; that she took it in her Hand, but could not remember that she returned it again, neither could she say positively, that she had it, but as he mis'd it as soon as he parted with her he thought he had occasion of Suspicion; yet, said he, I had been drinking before, (tho' it was but Nine in the Morning)and can't tell directly how the Matter stood; which being all the Evidence he could give against her, she was <rs type="verdictDescription" id="t17280717-4-verdict17">
>                          <interp value="notGuilty" type="verdictCategory" inst="t17280717-4-verdict17"/>
>                    acquitted
>                 </rs>.</p>
>                 </div1>
>
>
>                 <div1 id="t17280717-5" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-5"/>
>                    <interp value="1728" type="year" inst="t17280717-5"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-5"/>
>                    <interp value="17280717" type="date" inst="t17280717-5"/>
>                    <join targets="t17280717-5-defend44 t17280717-5-off19 t17280717-5-verdict22" targOrder="Y" id="t17280717-5-off19-c40" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-5-defend44">
>                       Elizabeth
>                       Mould
>                    <interp value="Mould" type="surname" inst="t17280717-5-defend44"/>
>                          <interp value="Elizabeth" type="given" inst="t17280717-5-defend44"/>
>                          <interp value="female" type="gender" inst="t17280717-5-defend44"/>
>                       </persName>
>                 , of <placeName id="t17280717-5-defloc18">St. Martins in the Fields</placeName>
>                       <interp value="St. Martins in the Fields" type="placeName" inst="t17280717-5-defloc18"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-5-defloc18"/>
>                       <join targets="t17280717-5-defend44 t17280717-5-defloc18" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-5-off19">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-5-off19"/>
>                          <interp value="pocketpicking" type="offenceSubcategory" inst="t17280717-5-off19"/>
>                    privately stealing Fifty Pounds and eight Shillings, from the Person of
>                          <persName type="victimName" id="t17280717-5-victim45">
>                             John
>                             Coxall
>                          <interp value="Coxall" type="surname" inst="t17280717-5-victim45"/>
>                             <interp value="John" type="given" inst="t17280717-5-victim45"/>
>                             <interp value="male" type="gender" inst="t17280717-5-victim45"/>
>                             <join targets="t17280717-5-off19 t17280717-5-victim45" targOrder="Y" result="offenceVictim"/>
>                          </persName>
>
>
>                       </rs>, on the <rs type="crimeDate" id="t17280717-5-cd20">24th of June</rs>
>                       <join targets="t17280717-5-off19 t17280717-5-cd20" targOrder="Y" result="offenceCrimeDate"/> last</p>
>                    <p>The Prosecutor depos'd, That he being a <rs type="occupation" id="t17280717-5-viclabel21">Bricklayer</rs>
>                       <join targets="t17280717-5-victim45 t17280717-5-viclabel21" targOrder="Y" result="persNameOccupation"/>, was endeavouring to get some Business to do, at the House where the Prisoner lived, and it being a Chandler's Shop, to obtain the Good-Will of the People, he call'd for several Drams, and treated the Mistress of the House and one Mrs. Greaves, who was his Customer, with Cyder, Brandy, &amp;c. that whilst they were drinking the Prisoner came into the Room, and he likewise treated her, and she, in return, wip'd him over the Face, and seem'd very fond of him, saying, she would give him a fine Nosegay, and something to cheer his Heart, if he would go with her to Covent-Garden Market, and then (as his Expression was) she weagled him down into the Cellar, and there kept him lock'd up in a back Passage, that he was very much in Liquor, and scarce sensible of what he did, but he found what she had done to his Sorrow, for she had taken all his Money, which he missing, made a Noise, and the People of the House hearing him, let him out of his Place of Confinement, by conducting him up the Back Stairs; that he got Officers and search'd the Cellar, but could not find the Prisoner. The Prisoner desiring he might be ask'd if they did not drink together in the Cellar, the Prosecutor answer'd, No, she would fetch no Drink, saying, she did not care to be seen by the Publicans; that she was all for dry Money, and she, and None but she, had his Fifty Pounds, for save only him and her, there were neither Man, Woman, nor Child, nor Dog nor Cat in the Cellar.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person46">
>                       Elizabeth
>                       Harper
>                    <interp value="Harper" type="surname" inst="t17280717-5-person46"/>
>                          <interp value="Elizabeth" type="given" inst="t17280717-5-person46"/>
>                          <interp value="female" type="gender" inst="t17280717-5-person46"/>
>                       </persName>
>                  depos'd, That the Prosecutor came into her Shop, and drank Cyder, Usquebaugh and Brandy, and being disguis'd in Liquor he pull'd out eight or ten Guineas and said, he was no Scoundrel; that she and Mrs. Greaves, whom he brought in to treat, begg'd he would put up his Money and take Care of it, and about that Time the Prisoner came into the Room, and familiarly stroaking his cheeks, persuaded him to go into the Cellar, saying, he should pay his Footing, that they staid half an Hour below, and this Deponent looking down, saw the Prisoner's Mother there, that he came up with her again, and treated her with Usquebaugh, and at 11 o' Clock, which was several Hours after, they heard him in a back Passage, where the House was supplied with Water, and letting him into their House, he said he was robb'd, at which this Deponent's Husband thrust him out of doors.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person47">
>                       Isabella
>                       Greaves
>                    <interp value="Greaves" type="surname" inst="t17280717-5-person47"/>
>                          <interp value="Isabella" type="given" inst="t17280717-5-person47"/>
>                          <interp value="female" type="gender" inst="t17280717-5-person47"/>
>                       </persName>
>                  likewise confirm'd every Part of this Deposition, adding, That the Prisoner was not to be seen that Night after the Robbery.</p>
>                    <p>The Prisoner said in her Defence, That the Prosecutor went down into her Cellar, and behaved himself so rudely, that she was forced to threaten to send for an Officer, that she knew nothing of his Money, and had not gone away that Night, but as she was oblig'd by her Husband, and that she came next Morning and set her Greens out.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person48">
>                       Anthony
>                       Dyer
>                    <interp value="Dyer" type="surname" inst="t17280717-5-person48"/>
>                          <interp value="Anthony" type="given" inst="t17280717-5-person48"/>
>                          <interp value="male" type="gender" inst="t17280717-5-person48"/>
>                       </persName>
>                  depos'd, That the Prosecutor told him he had lost his Money being Drunk, that he had been in a Cellar and in a Vault, where he fell asleep, and gave a very odd account of the Adventure.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person49">
>                       George
>                       Smith
>                    <interp value="Smith" type="surname" inst="t17280717-5-person49"/>
>                          <interp value="George" type="given" inst="t17280717-5-person49"/>
>                          <interp value="male" type="gender" inst="t17280717-5-person49"/>
>                       </persName>
>                  depos'd, That the Prosecutor said he fell asleep upon a Vault, and could charge no Body.</p>
>                    <p>The Constable, the Watchman, and others severally depos'd, That the Morning after this happened, he said he could charge no particular person, but he would indict the House. The Prisoner having a very good Character from several reputable Witnesses, the Jury <rs type="verdictDescription" id="t17280717-5-verdict22">
>                          <interp value="notGuilty" type="verdictCategory" inst="t17280717-5-verdict22"/>
>                    acquitted
>                 </rs> her.</p>
>                 </div1>
>
>
>                 <div1 id="t17280717-6" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-6"/>
>                    <interp value="1728" type="year" inst="t17280717-6"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-6"/>
>                    <interp value="17280717" type="date" inst="t17280717-6"/>
>                    <join targets="t17280717-6-defend50 t17280717-6-off24 t17280717-6-verdict26" targOrder="Y" id="t17280717-6-off24-c46" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-6-defend50">
>                       Phillip
>                       Hilliard
>                    <interp value="Hilliard" type="surname" inst="t17280717-6-defend50"/>
>                          <interp value="Phillip" type="given" inst="t17280717-6-defend50"/>
>                          <interp value="male" type="gender" inst="t17280717-6-defend50"/>
>                       </persName>
>                 , of <placeName id="t17280717-6-defloc23">St. James's Westminster</placeName>
>                       <interp value="St. James's Westminster" type="placeName" inst="t17280717-6-defloc23"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-6-defloc23"/>
>                       <join targets="t17280717-6-defend50 t17280717-6-defloc23" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-6-off24">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-6-off24"/>
>                          <interp value="grandLarceny" type="offenceSubcategory" inst="t17280717-6-off24"/>
>                    feloniously stealing a Silver Spoon, value eleven Shillings, the Property of
>                          <persName type="victimName" id="t17280717-6-victim51">
>                             Abraham
>                             Mannio
>                          <interp value="Mannio" type="surname" inst="t17280717-6-victim51"/>
>                             <interp value="Abraham" type="given" inst="t17280717-6-victim51"/>
>                             <interp value="male" type="gender" inst="t17280717-6-victim51"/>
>                             <join targets="t17280717-6-off24 t17280717-6-victim51" targOrder="Y" result="offenceVictim"/>
>                          </persName>
>                        , on the <rs type="crimeDate" id="t17280717-6-cd25">27th of June</rs>
>                          <join targets="t17280717-6-off24 t17280717-6-cd25" targOrder="Y" result="offenceCrimeDate"/> last.
>                 </rs>
>                    </p>
>                    <p>Mr. Hayden depos'd, That the Prisoner brought the Spoon to him to pawn, and he suspecting it to be stolen, stopp'd him and carried him to the Round-House, where he confess'd he stole it at the Prosecutor's, he being invited there to Dinner with some Gentlemans Servants.</p>
>                    <p>
>
>                       <persName id="t17280717-6-person52">
>                       Robert
>                       Amey
>                    <interp value="Amey" type="surname" inst="t17280717-6-person52"/>
>                          <interp value="Robert" type="given" inst="t17280717-6-person52"/>
>                          <interp value="male" type="gender" inst="t17280717-6-person52"/>
>                       </persName>
>                  depos'd, That he attended the Gentlemens Servants at Dinner, and afterwards they miss'd a Spoon, that Mr. Haydon sent them word of the Spoon brought to him by the Prisoner, and he going to match it by the others, found it to be the same which they had lost, it being of the same Make and Mark: The Fact being thus plainly proved upon him, the Jury found him <rs type="verdictDescription" id="t17280717-6-verdict26">
>                          <interp value="guilty" type="verdictCategory" inst="t17280717-6-verdict26"/>
>                          <interp value="theftunder1s" type="verdictSubcategory" inst="t17280717-6-verdict26"/>
>                    guilty to the value of 10d.
>                 </rs>
>                    </p>
>                    <p>
>                       <rs type="punishmentDescription" id="t17280717-6-punish27">
>                          <interp value="transport" type="punishmentCategory" inst="t17280717-6-punish27"/>
>                          <join targets="t17280717-6-defend50 t17280717-6-punish27" targOrder="Y" result="defendantPunishment"/>
>
>                          <note>[Transportation. See summary.]</note>
>
>                       </rs>
>                    </p>
>                 </div1>
>
>
>                 <div1 id="t17280717-7" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-7"/>
>                    <interp value="1728" type="year" inst="t17280717-7"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-7"/>
>                    <interp value="17280717" type="date" inst="t17280717-7"/>
>                    <join targets="t17280717-7-defend53 t17280717-7-off29 t17280717-7-verdict31" targOrder="Y" id="t17280717-7-off29-c49" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-7-defend53">
>                       Robert
>                       Ashby
>                    <interp value="Ashby" type="surname" inst="t17280717-7-defend53"/>
>                          <interp value="Robert" type="given" inst="t17280717-7-defend53"/>
>                          <interp value="male" type="gender" inst="t17280717-7-defend53"/>
>                       </persName>
>                 , of <rs type="occupation" id="t17280717-7-deflabel28">St. Andrew's Holborn</rs>
>                       <join targets="t17280717-7-defend53 t17280717-7-deflabel28" targOrder="Y" result="persNameOccupation"/>, was indicted for <rs type="offenceDescription" id="t17280717-7-off29">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-7-off29"/>
>                          <interp value="grandLarceny" type="offenceSubcategory" inst="t17280717-7-off29"/>
>                    stealing a Gold Watch, value 16 l. a Chain, value 4 l. a Cornelian Seal set in Gold, value 10s. the Property of
>                          <persName type="victimName" id="t17280717-7-victim54">
>                             Nicholas
>                             Roper
>                          <interp value="Roper" type="surname" inst="t17280717-7-victim54"/>
>                             <interp value="Nicholas" type="given" inst="t17280717-7-victim54"/>
>                             <interp value="male" type="gender" inst="t17280717-7-victim54"/>
>                          </persName>
>                         from the Person of
>                          <persName type="victimName" id="t17280717-7-victim55">
>                             Phoebe
>                             Thickpenny
>                          <interp value="Thickpenny" type="surname" inst="t17280717-7-victim55"/>
>                             <interp value="Phoebe" type="given" inst="t17280717-7-victim55"/>
>                             <interp value="female" type="gender" inst="t17280717-7-victim55"/>
>                          </persName>
>
>
>                       </rs>, who depos'd, That her Mistress being at Little Chelsea, sent her to their House in Lad-lane, for the Watch and other Things she had occasion for, that she call'd at the Prisoner's in Castle Yard, Chick-Lane, as she was going towards Chelsea with the Watch and the Bundle, where she drank part of a Pint of Beer, some Tea 2 Quarterns of Brandy, and a Bottle of Cyder; that the Prisoner would go part of the Way home with her, and in <placeName id="t17280717-7-crimeloc30">Leather-Lane</placeName>
>                       <interp value="Leather-Lane" type="placeName" inst="t17280717-7-crimeloc30"/>
>                       <interp value="crimeLocation" type="type" inst="t17280717-7-crimeloc30"/>
>                       <join targets="t17280717-7-off29 t17280717-7-crimeloc30" targOrder="Y" result="offencePlace"/>, he said, faith they would not part Dry lips, and accordingly they went into a publick House and drank a Pint of Twopenny, and two Quarterns of Brandy, that she had the Watch then, and at the Door the Prisoner kiss'd her, and gave her a shilling for a Coach, she having out-staid her Time; that when he kiss'd her, he put one Hand around her Waist, but what he did with the other she could not tell, that she then cross'd the Way to another House, and immediately miss'd the Watch, and she was sure she had pinn'd it so to her Side, that she could not drop it.</p>
>                    <p>
>
>                       <persName id="t17280717-7-person56">
>                       Margaret
>                       Nelson
>                    <interp value="Nelson" type="surname" inst="t17280717-7-person56"/>
>                          <interp value="Margaret" type="given" inst="t17280717-7-person56"/>
>                          <interp value="female" type="gender" inst="t17280717-7-person56"/>
>                       </persName>
>                  depos'd, That the Prisoner and Phoebe Thickpenny, came to her House in Leather-Lane, and he call'd for a private Room, to which the Girl would not consent, that the Girl wanted to go to, &amp;c. and she went with her, when the Girl said, she had her Mistress's Gold Watch, and seeming to look on it, told her it was 7 o' Clock, but this Deponent did not see the Watch, yet she said, she did verily believe she heard it beat, that they soon parted, and in a Quarter of an Hour the maid returned, and said she had lost the Watch.</p>
>                    <p>The Prisoner said in his Defence, That he knew nothing of it, any further than she said it was her Mistress's, that they parted very good Friends, and she having been an old Sweetheart of his, laid her Head upon his Shoulder, and said, She could live and die there, which would have been no little Aggravation to his Crime, had he wrong'd so good Natur'd a Creature: But the Evidence against him being weak, and several appearing to his Character, the Jury <rs type="verdictDescription" id="t17280717-7-verdict31">
>                          <interp value="notGuilty" type="verdictCategory" inst="t17280717-7-verdict31"/>
>                    acquitted
>                 </rs> him. </p>
>                 </div1>
>
>
>                 <div1 id="t17280717-8" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-8"/>
>                    <interp value="1728" type="year" inst="t17280717-8"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-8"/>
>                    <interp value="17280717" type="date" inst="t17280717-8"/>
>                    <join targets="t17280717-8-defend57 t17280717-8-off33 t17280717-8-verdict35" targOrder="Y" id="t17280717-8-off33-c53" result="criminalCharge"/>
>                    <join targets="t17280717-8-defend58 t17280717-8-off33 t17280717-8-verdict35" targOrder="Y" id="t17280717-8-off33-c54" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-8-defend57">
>                       Joseph
>                       Plummer
>                    <interp value="Plummer" type="surname" inst="t17280717-8-defend57"/>
>                          <interp value="Joseph" type="given" inst="t17280717-8-defend57"/>
>                          <interp value="male" type="gender" inst="t17280717-8-defend57"/>
>                       </persName>
>                  , and
>                    <persName type="defendantName" id="t17280717-8-defend58">
>                       Henry
>                       Coleman
>                    <interp value="Coleman" type="surname" inst="t17280717-8-defend58"/>
>                          <interp value="Henry" type="given" inst="t17280717-8-defend58"/>
>                          <interp value="male" type="gender" inst="t17280717-8-defend58"/>
>                       </persName>
>                  , of <placeName id="t17280717-8-defloc32">Edmonton</placeName>
>                       <interp value="Edmont...

In [ ]:
elem

  

>     res3: scala.xml.Elem =
>     <TEI.2>
>        <text>
>           <body>
>              <div0 id="17280717" type="sessionsPaper">
>                 <interp value="BAILEY" type="collection" inst="17280717"/>
>                 <interp value="1728" type="year" inst="17280717"/>
>                 <interp value="sessionsPapers/17280717" type="uri" inst="17280717"/>
>                 <interp value="17280717" type="date" inst="17280717"/>
>                 <xptr doc="17280717" type="transcription"/>
>
>                 <div1 id="f17280717-1" type="frontMatter">
>                    <interp value="BAILEY" type="collection" inst="f17280717-1"/>
>                    <interp value="1728" type="year" inst="f17280717-1"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="f17280717-1"/>
>                    <interp value="17280717" type="date" inst="f17280717-1"/>
>
>                    <xptr doc="172807170001" type="pageFacsimile"/>THE PROCEEDINGS AT THE Sessions of the Peace, and Oyer and Terminer for the City of LONDON: AND
>              <p>On the King's Commission of Goal-Delivery of Newgate, held at Justice-Hall in the Old Baily, for the CITY of LONDON and COUNTY of MIDDLESEX.</p>
>                    <p>On Wednesday, Thursday, and Friday, being the 17th, 18th, and 19th of July, 1728, in the Second Year of His MAJESTY's Reign.</p>
>                    <p>(Price Six Pence)</p>
>                    <p>BEFORE the Right Honourable Sir
>                       <persName type="judiciaryName" id="f17280717-1-person1">
>                          EDWARD
>                          BECHER
>                       <interp value="BECHER" type="surname" inst="f17280717-1-person1"/>
>                          <interp value="EDWARD" type="given" inst="f17280717-1-person1"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person1"/>
>                       </persName>
>                    , Lord Mayor of the City of London; the Right Honourable the Lord Chief
>                       <persName type="judiciaryName" id="f17280717-1-person2">
>                          Baron
>                          Pengelly
>                       <interp value="Pengelly" type="surname" inst="f17280717-1-person2"/>
>                          <interp value="Baron" type="given" inst="f17280717-1-person2"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person2"/>
>                       </persName>
>                    ; Mr. Justice Reynolds; Mr.
>                       <persName type="judiciaryName" id="f17280717-1-person3">
>                          Baron
>                          Thompson
>                       <interp value="Thompson" type="surname" inst="f17280717-1-person3"/>
>                          <interp value="Baron" type="given" inst="f17280717-1-person3"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person3"/>
>                       </persName>
>                    , Recorder of the City of London; and
>                       <persName type="judiciaryName" id="f17280717-1-person4">
>                          John
>                          Raby
>                       <interp value="Raby" type="surname" inst="f17280717-1-person4"/>
>                          <interp value="John" type="given" inst="f17280717-1-person4"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person4"/>
>                       </persName>
>                    , Esq; Serjeant at Law; and other His Majesty's Justices of Goal-Delivery, and Oyer and Terminer aforesaid; Together with several of His Majesty's Justices of the Peace for the said City of London.</p>
>
>                    <p>London Jury.</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person5">
>                          John
>                          Land
>                       <interp value="Land" type="surname" inst="f17280717-1-person5"/>
>                          <interp value="John" type="given" inst="f17280717-1-person5"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person5"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person6">
>                          Nathaniel
>                          Mason
>                       <interp value="Mason" type="surname" inst="f17280717-1-person6"/>
>                          <interp value="Nathaniel" type="given" inst="f17280717-1-person6"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person6"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person7">
>                          Benjamin
>                          Allibone
>                       <interp value="Allibone" type="surname" inst="f17280717-1-person7"/>
>                          <interp value="Benjamin" type="given" inst="f17280717-1-person7"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person7"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person8">
>                          Joseph
>                          Westwood
>                       <interp value="Westwood" type="surname" inst="f17280717-1-person8"/>
>                          <interp value="Joseph" type="given" inst="f17280717-1-person8"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person8"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person9">
>                          Gabriel
>                          Wittacre
>                       <interp value="Wittacre" type="surname" inst="f17280717-1-person9"/>
>                          <interp value="Gabriel" type="given" inst="f17280717-1-person9"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person9"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person10">
>                          Samuel
>                          Tilley
>                       <interp value="Tilley" type="surname" inst="f17280717-1-person10"/>
>                          <interp value="Samuel" type="given" inst="f17280717-1-person10"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person10"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person11">
>                          Robert
>                          Lathwell
>                       <interp value="Lathwell" type="surname" inst="f17280717-1-person11"/>
>                          <interp value="Robert" type="given" inst="f17280717-1-person11"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person11"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person12">
>                          Edward
>                          Newman
>                       <interp value="Newman" type="surname" inst="f17280717-1-person12"/>
>                          <interp value="Edward" type="given" inst="f17280717-1-person12"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person12"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person13">
>                          Nathaniel
>                          Pickering
>                       <interp value="Pickering" type="surname" inst="f17280717-1-person13"/>
>                          <interp value="Nathaniel" type="given" inst="f17280717-1-person13"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person13"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person14">
>                          Simon
>                          Tunks
>                       <interp value="Tunks" type="surname" inst="f17280717-1-person14"/>
>                          <interp value="Simon" type="given" inst="f17280717-1-person14"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person14"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person15">
>                          Thomas
>                          Maud
>                       <interp value="Maud" type="surname" inst="f17280717-1-person15"/>
>                          <interp value="Thomas" type="given" inst="f17280717-1-person15"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person15"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person16">
>                          John
>                          Bond
>                       <interp value="Bond" type="surname" inst="f17280717-1-person16"/>
>                          <interp value="John" type="given" inst="f17280717-1-person16"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person16"/>
>                       </persName>
>                    .</p>
>
>
>                    <p>Middlesex Jury.</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person17">
>                          Elisha
>                          Impey
>                       <interp value="Impey" type="surname" inst="f17280717-1-person17"/>
>                          <interp value="Elisha" type="given" inst="f17280717-1-person17"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person17"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person18">
>                          Christopher
>                          Harris
>                       <interp value="Harris" type="surname" inst="f17280717-1-person18"/>
>                          <interp value="Christopher" type="given" inst="f17280717-1-person18"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person18"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person19">
>                          William
>                          Perkins
>                       <interp value="Perkins" type="surname" inst="f17280717-1-person19"/>
>                          <interp value="William" type="given" inst="f17280717-1-person19"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person19"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person20">
>                          Gilbert
>                          Watson
>                       <interp value="Watson" type="surname" inst="f17280717-1-person20"/>
>                          <interp value="Gilbert" type="given" inst="f17280717-1-person20"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person20"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person21">
>                          John
>                          Wells
>                       <interp value="Wells" type="surname" inst="f17280717-1-person21"/>
>                          <interp value="John" type="given" inst="f17280717-1-person21"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person21"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person22">
>                          William
>                          Carpenter
>                       <interp value="Carpenter" type="surname" inst="f17280717-1-person22"/>
>                          <interp value="William" type="given" inst="f17280717-1-person22"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person22"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person23">
>                          Allen
>                          Evans
>                       <interp value="Evans" type="surname" inst="f17280717-1-person23"/>
>                          <interp value="Allen" type="given" inst="f17280717-1-person23"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person23"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person24">
>                          Henry
>                          Cowmbe
>                       <interp value="Cowmbe" type="surname" inst="f17280717-1-person24"/>
>                          <interp value="Henry" type="given" inst="f17280717-1-person24"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person24"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person25">
>                          Simon
>                          Parsons
>                       <interp value="Parsons" type="surname" inst="f17280717-1-person25"/>
>                          <interp value="Simon" type="given" inst="f17280717-1-person25"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person25"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person26">
>                          George
>                          Gilbert
>                       <interp value="Gilbert" type="surname" inst="f17280717-1-person26"/>
>                          <interp value="George" type="given" inst="f17280717-1-person26"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person26"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person27">
>                          Nicholas
>                          Gardner
>                       <interp value="Gardner" type="surname" inst="f17280717-1-person27"/>
>                          <interp value="Nicholas" type="given" inst="f17280717-1-person27"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person27"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person28">
>                          Thomas
>                          Ireland
>                       <interp value="Ireland" type="surname" inst="f17280717-1-person28"/>
>                          <interp value="Thomas" type="given" inst="f17280717-1-person28"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person28"/>
>                       </persName>
>                    .</p>
>
>                 </div1>
>
>
>                 <div1 id="t17280717-1" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-1"/>
>                    <interp value="1728" type="year" inst="t17280717-1"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-1"/>
>                    <interp value="17280717" type="date" inst="t17280717-1"/>
>                    <join targets="t17280717-1-defend29 t17280717-1-off2 t17280717-1-verdict5" targOrder="Y" id="t17280717-1-off2-c29" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-1-defend29">
>                       James
>                       Haddock
>                    <interp value="Haddock" type="surname" inst="t17280717-1-defend29"/>
>                          <interp value="James" type="given" inst="t17280717-1-defend29"/>
>                          <interp value="male" type="gender" inst="t17280717-1-defend29"/>
>                       </persName>
>                 , of <placeName id="t17280717-1-defloc1">St. Bennet's Paul's Wharf</placeName>
>                       <interp value="St. Bennet's Paul's Wharf" type="placeName" inst="t17280717-1-defloc1"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-1-defloc1"/>
>                       <join targets="t17280717-1-defend29 t17280717-1-defloc1" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-1-off2">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-1-off2"/>
>                          <interp value="theftFromPlace" type="offenceSubcategory" inst="t17280717-1-off2"/>
>                    feloniously stealing 2 Guineas, and 5 l. 16 s. in Silver, a Silver Cup, a Silver Cork Screw, 2 Silver Spoons, and a Nutmeg-grater, in the Dwelling-House of
>
>                             <persName type="victimName" id="t17280717-1-victim31">
>                                James
>                                Reeves
>                             <interp value="Reeves" type="surname" inst="t17280717-1-victim31"/>
>                             <interp value="James" type="given" inst="t17280717-1-victim31"/>
>                             <interp value="male" type="gender" inst="t17280717-1-victim31"/>
>                             <join targets="t17280717-1-off2 t17280717-1-victim31" targOrder="Y" result="offenceVictim"/>
>                          </persName>
>
>
>
>                       </rs>, on the <rs type="crimeDate" id="t17280717-1-cd3">9th of April, and in the 13th Year of his late Majesty King George the First</rs>
>                       <join targets="t17280717-1-off2 t17280717-1-cd3" targOrder="Y" result="offenceCrimeDate"/>, the Property of
>                    <persName id="t17280717-1-person32">
>                       James
>                       Reeves
>                    <interp value="Reeves" type="surname" inst="t17280717-1-person32"/>
>                          <interp value="James" type="given" inst="t17280717-1-person32"/>
>                          <interp value="male" type="gender" inst="t17280717-1-person32"/>
>                       </persName>
>                  aforesaid.</p>
>                    <p>
>
>                       <persName id="t17280717-1-person33">
>                       Elizabeth
>                       Reeves
>                    <interp value="Reeves" type="surname" inst="t17280717-1-person33"/>
>                          <interp value="Elizabeth" type="given" inst="t17280717-1-person33"/>
>                          <interp value="female" type="gender" inst="t17280717-1-person33"/>
>                       </persName>
>                  depos'd, That the Prisoner was a Lodger at her House on <placeName id="t17280717-1-crimeloc4">Addle-Hill</placeName>
>                       <interp value="Addle-Hill" type="placeName" inst="t17280717-1-crimeloc4"/>
>                       <interp value="crimeLocation" type="type" inst="t17280717-1-crimeloc4"/>
>                       <join targets="t17280717-1-off2 t17280717-1-crimeloc4" targOrder="Y" result="offencePlace"/>, near Doctors Commons, when this Robbery was committed, and that it being on the Sabbath Day, she desired the Prisoner, if he did not go abroad, to have an Eye to her Room, which she locked up, and which, he promised to have an Eye to; but when she came home, the Chamber Door and the Corner-Cupboard had been forced open, which appeared by the Mark of an Instrument, the Hinges tore off and the Money gone, the Drawers rifled, and the Plate taken out, though some of the Drawers, out of which the Plate was taken, she left lock'd when she went from Home; upon which she cried out, saying, she was robb'd, and the Prisoner's Wife being above Stairs, came down, and said, her Husband was gone out, that he had been guilty of Failings, and desired her to be easy and she would Work early and late to make Satisfaction, though she did not know he had taken the Things, but his not coming home again confirmed them the more in this Suspicion; and there were other Witnesses, who depos'd, That the Prosecutor left the Prisoner in Care of her Door, and that he promised to look after it.</p>
>                    <p>The Prisoner said in his Defence, That Mrs. Reeves had lost a pair of Silver Buckles out of her Drawers a Fortnight before this Robbery, and she said, she did believe it was done by a Char-woman that she employ'd, which Mrs. Reeves acknowledging, and the Prisoner telling a plausible Story of his being in bad Circumstances, and that Day the Robbery was committed, one Diston pittying his Case, lent him Money to go down to Bristol, to Trade there, if possible to retrieve himself, and he not daring to go out of Doors in the Week Days, went out of the House unfortunately on that Afternoon: There being no positive Evidence against him, the Jury <rs type="verdictDescription" id="t17280717-1-verdict5">
>                          <interp value="notGuilty" type="verdictCategory" inst="t17280717-1-verdict5"/>
>                    acquitted
>                 </rs> him.</p>
>                 </div1>
>
>
>                 <div1 id="t17280717-2" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-2"/>
>                    <interp value="1728" type="year" inst="t17280717-2"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-2"/>
>                    <interp value="17280717" type="date" inst="t17280717-2"/>
>                    <join targets="t17280717-2-defend35 t17280717-2-off6 t17280717-2-verdict7" targOrder="Y" id="t17280717-2-off6-c33" result="criminalCharge"/>
>
>                    <p>
>
>
>                       <persName type="defendantName" id="t17280717-2-defend35">
>                          David
>                          Ball
>                       <interp value="Ball" type="surname" inst="t17280717-2-defend35"/>
>                          <interp value="David" type="given" inst="t17280717-2-defend35"/>
>                          <interp value="male" type="gender" inst="t17280717-2-defend35"/>
>                       </persName>
>
>                  was indicted for <rs type="offenceDescription" id="t17280717-2-off6">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-2-off6"/>
>                          <interp value="pettyLarceny" type="offenceSubcategory" inst="t17280717-2-off6"/>
>                    a Petty Larceny, in stealing a Yard and a Quarter of Linnen Cloth, value 11 d.
>                 </rs> the Goods of
>
>                       <persName type="victimName" id="t17280717-2-victim37">
>                          John
>                          Williams
>                       <interp value="Williams" type="surname" inst="t17280717-2-victim37"/>
>                          <interp value="John" type="given" inst="t17280717-2-victim37"/>
>                          <interp value="male" type="gender" inst="t17280717-2-victim37"/>
>                       </persName>
>
>                  and
>
>                       <persName type="victimName" id="t17280717-2-victim39">
>                          William
>                          Williams
>                       <interp value="Williams" type="surname" inst="t17280717-2-victim39"/>
>                          <interp value="William" type="given" inst="t17280717-2-victim39"/>
>                          <interp value="male" type="gender" inst="t17280717-2-victim39"/>
>                       </persName>
>
>                 ; to which Indictment he <rs type="verdictDescription" id="t17280717-2-verdict7">
>                          <interp value="guilty" type="verdictCategory" inst="t17280717-2-verdict7"/>
>                          <interp value="pleadedGuilty" type="verdictSubcategory" inst="t17280717-2-verdict7"/>
>                    pleaded Guilty
>                 </rs>.</p>
>                    <p>
>                       <rs type="punishmentDescription" id="t17280717-2-punish8">
>                          <interp value="transport" type="punishmentCategory" inst="t17280717-2-punish8"/>
>                          <join targets="t17280717-2-defend35 t17280717-2-punish8" targOrder="Y" result="defendantPunishment"/>
>
>                          <note>[Transportation. See summary.]</note>
>
>                       </rs>
>                    </p>
>                 </div1>
>
>
>                 <div1 id="t17280717-3" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-3"/>
>                    <interp value="1728" type="year" inst="t17280717-3"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-3"/>
>                    <interp value="17280717" type="date" inst="t17280717-3"/>
>                    <join targets="t17280717-3-defend40 t17280717-3-off10 t17280717-3-verdict11" targOrder="Y" id="t17280717-3-off10-c36" result="criminalCharge"/>
>
>                    <p>
>                       <xptr doc="172807170002" type="pageFacsimile"/>
>
>                       <persName type="defendantName" id="t17280717-3-defend40">
>                       Nathaniel
>                       Mercy
>                    <interp value="Mercy" type="surname" inst="t17280717-3-defend40"/>
>                          <interp value="Nathaniel" type="given" inst="t17280717-3-defend40"/>
>                          <interp value="male" type="gender" inst="t17280717-3-defend40"/>
>                       </persName>
>                 , of <placeName id="t17280717-3-defloc9">St. James's Westminster</placeName>
>                       <interp value="St. James's Westminster" type="placeName" inst="t17280717-3-defloc9"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-3-defloc9"/>
>                       <join targets="t17280717-3-defend40 t17280717-3-defloc9" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-3-off10">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-3-off10"/>
>                          <interp value="grandLarceny" type="offenceSubcategory" inst="t17280717-3-off10"/>
>                    stealing a Coach Wheel, value 12 Shillings
>                 </rs>, the Goods of
>                    <persName type="victimName" id="t17280717-3-victim41">
>                       Thomas
>                       West
>                    <interp value="West" type="surname" inst="t17280717-3-victim41"/>
>                          <interp value="Thomas" type="given" inst="t17280717-3-victim41"/>
>                          <interp value="male" type="gender" inst="t17280717-3-victim41"/>
>                          <join targets="t17280717-3-off10 t17280717-3-victim41" targOrder="Y" result="offenceVictim"/>
>                       </persName>
>                  , on the 9th of this Instant, to which Indictment he <rs type="verdictDescription" id="t17280717-3-verdict11">
>                          <interp value="guilty" type="verdictCategory" inst="t17280717-3-verdict11"/>
>                          <interp value="pleadedGuilty" type="verdictSubcategory" inst="t17280717-3-verdict11"/>
>                    pleaded guilty
>                 </rs>.</p>
>                    <p>
>                       <rs type="punishmentDescription" id="t17280717-3-punish12">
>                          <interp value="transport" type="punishmentCategory" inst="t17280717-3-punish12"/>
>                          <join targets="t17280717-3-defend40 t17280717-3-punish12" targOrder="Y" result="defendantPunishment"/>
>
>                          <note>[Transportation. See summary.]</note>
>
>                       </rs>
>                    </p>
>                 </div1>
>
>
>                 <div1 id="t17280717-4" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-4"/>
>                    <interp value="1728" type="year" inst="t17280717-4"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-4"/>
>                    <interp value="17280717" type="date" inst="t17280717-4"/>
>                    <join targets="t17280717-4-defend42 t17280717-4-off14 t17280717-4-verdict17" targOrder="Y" id="t17280717-4-off14-c38" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-4-defend42">
>                       Margaret
>                       King
>                    <interp value="King" type="surname" inst="t17280717-4-defend42"/>
>                          <interp value="Margaret" type="given" inst="t17280717-4-defend42"/>
>                          <interp value="female" type="gender" inst="t17280717-4-defend42"/>
>                       </persName>
>                 , of <placeName id="t17280717-4-defloc13">St. Ann's Westminster</placeName>
>                       <interp value="St. Ann's Westminster" type="placeName" inst="t17280717-4-defloc13"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-4-defloc13"/>
>                       <join targets="t17280717-4-defend42 t17280717-4-defloc13" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-4-off14">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-4-off14"/>
>                          <interp value="grandLarceny" type="offenceSubcategory" inst="t17280717-4-off14"/>
>                    privately stealing a Gold Watch, value 16 l.
>                 </rs> on the <rs type="crimeDate" id="t17280717-4-cd15">first of May</rs>
>                       <join targets="t17280717-4-off14 t17280717-4-cd15" targOrder="Y" result="offenceCrimeDate"/> last, the Property of
>                    <persName type="victimName" id="t17280717-4-victim43">
>                       Timothy
>                       Conner
>                    <interp value="Conner" type="surname" inst="t17280717-4-victim43"/>
>                          <interp value="Timothy" type="given" inst="t17280717-4-victim43"/>
>                          <interp value="male" type="gender" inst="t17280717-4-victim43"/>
>                          <join targets="t17280717-4-off14 t17280717-4-victim43" targOrder="Y" result="offenceVictim"/>
>                       </persName>
>                  .</p>
>                    <p>The Prosecutor depos'd, That he met the Prisoner in the Street, and ask'd her to drink a Glass of Wine, to which she consented, and they went to the <placeName id="t17280717-4-crimeloc16">Swan Tavern in Newport Market</placeName>
>                       <interp value="Swan Tavern in Newport Market" type="placeName" inst="t17280717-4-crimeloc16"/>
>                       <interp value="crimeLocation" type="type" inst="t17280717-4-crimeloc16"/>
>                       <join targets="t17280717-4-off14 t17280717-4-crimeloc16" targOrder="Y" result="offencePlace"/>, and lovingly drank 4 Pints, the Prisoner asking him what it was a Clock, he pulled out his Gold Watch, and bid her look; that she took it in her Hand, but could not remember that she returned it again, neither could she say positively, that she had it, but as he mis'd it as soon as he parted with her he thought he had occasion of Suspicion; yet, said he, I had been drinking before, (tho' it was but Nine in the Morning)and can't tell directly how the Matter stood; which being all the Evidence he could give against her, she was <rs type="verdictDescription" id="t17280717-4-verdict17">
>                          <interp value="notGuilty" type="verdictCategory" inst="t17280717-4-verdict17"/>
>                    acquitted
>                 </rs>.</p>
>                 </div1>
>
>
>                 <div1 id="t17280717-5" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-5"/>
>                    <interp value="1728" type="year" inst="t17280717-5"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-5"/>
>                    <interp value="17280717" type="date" inst="t17280717-5"/>
>                    <join targets="t17280717-5-defend44 t17280717-5-off19 t17280717-5-verdict22" targOrder="Y" id="t17280717-5-off19-c40" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-5-defend44">
>                       Elizabeth
>                       Mould
>                    <interp value="Mould" type="surname" inst="t17280717-5-defend44"/>
>                          <interp value="Elizabeth" type="given" inst="t17280717-5-defend44"/>
>                          <interp value="female" type="gender" inst="t17280717-5-defend44"/>
>                       </persName>
>                 , of <placeName id="t17280717-5-defloc18">St. Martins in the Fields</placeName>
>                       <interp value="St. Martins in the Fields" type="placeName" inst="t17280717-5-defloc18"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-5-defloc18"/>
>                       <join targets="t17280717-5-defend44 t17280717-5-defloc18" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-5-off19">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-5-off19"/>
>                          <interp value="pocketpicking" type="offenceSubcategory" inst="t17280717-5-off19"/>
>                    privately stealing Fifty Pounds and eight Shillings, from the Person of
>                          <persName type="victimName" id="t17280717-5-victim45">
>                             John
>                             Coxall
>                          <interp value="Coxall" type="surname" inst="t17280717-5-victim45"/>
>                             <interp value="John" type="given" inst="t17280717-5-victim45"/>
>                             <interp value="male" type="gender" inst="t17280717-5-victim45"/>
>                             <join targets="t17280717-5-off19 t17280717-5-victim45" targOrder="Y" result="offenceVictim"/>
>                          </persName>
>
>
>                       </rs>, on the <rs type="crimeDate" id="t17280717-5-cd20">24th of June</rs>
>                       <join targets="t17280717-5-off19 t17280717-5-cd20" targOrder="Y" result="offenceCrimeDate"/> last</p>
>                    <p>The Prosecutor depos'd, That he being a <rs type="occupation" id="t17280717-5-viclabel21">Bricklayer</rs>
>                       <join targets="t17280717-5-victim45 t17280717-5-viclabel21" targOrder="Y" result="persNameOccupation"/>, was endeavouring to get some Business to do, at the House where the Prisoner lived, and it being a Chandler's Shop, to obtain the Good-Will of the People, he call'd for several Drams, and treated the Mistress of the House and one Mrs. Greaves, who was his Customer, with Cyder, Brandy, &amp;c. that whilst they were drinking the Prisoner came into the Room, and he likewise treated her, and she, in return, wip'd him over the Face, and seem'd very fond of him, saying, she would give him a fine Nosegay, and something to cheer his Heart, if he would go with her to Covent-Garden Market, and then (as his Expression was) she weagled him down into the Cellar, and there kept him lock'd up in a back Passage, that he was very much in Liquor, and scarce sensible of what he did, but he found what she had done to his Sorrow, for she had taken all his Money, which he missing, made a Noise, and the People of the House hearing him, let him out of his Place of Confinement, by conducting him up the Back Stairs; that he got Officers and search'd the Cellar, but could not find the Prisoner. The Prisoner desiring he might be ask'd if they did not drink together in the Cellar, the Prosecutor answer'd, No, she would fetch no Drink, saying, she did not care to be seen by the Publicans; that she was all for dry Money, and she, and None but she, had his Fifty Pounds, for save only him and her, there were neither Man, Woman, nor Child, nor Dog nor Cat in the Cellar.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person46">
>                       Elizabeth
>                       Harper
>                    <interp value="Harper" type="surname" inst="t17280717-5-person46"/>
>                          <interp value="Elizabeth" type="given" inst="t17280717-5-person46"/>
>                          <interp value="female" type="gender" inst="t17280717-5-person46"/>
>                       </persName>
>                  depos'd, That the Prosecutor came into her Shop, and drank Cyder, Usquebaugh and Brandy, and being disguis'd in Liquor he pull'd out eight or ten Guineas and said, he was no Scoundrel; that she and Mrs. Greaves, whom he brought in to treat, begg'd he would put up his Money and take Care of it, and about that Time the Prisoner came into the Room, and familiarly stroaking his cheeks, persuaded him to go into the Cellar, saying, he should pay his Footing, that they staid half an Hour below, and this Deponent looking down, saw the Prisoner's Mother there, that he came up with her again, and treated her with Usquebaugh, and at 11 o' Clock, which was several Hours after, they heard him in a back Passage, where the House was supplied with Water, and letting him into their House, he said he was robb'd, at which this Deponent's Husband thrust him out of doors.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person47">
>                       Isabella
>                       Greaves
>                    <interp value="Greaves" type="surname" inst="t17280717-5-person47"/>
>                          <interp value="Isabella" type="given" inst="t17280717-5-person47"/>
>                          <interp value="female" type="gender" inst="t17280717-5-person47"/>
>                       </persName>
>                  likewise confirm'd every Part of this Deposition, adding, That the Prisoner was not to be seen that Night after the Robbery.</p>
>                    <p>The Prisoner said in her Defence, That the Prosecutor went down into her Cellar, and behaved himself so rudely, that she was forced to threaten to send for an Officer, that she knew nothing of his Money, and had not gone away that Night, but as she was oblig'd by her Husband, and that she came next Morning and set her Greens out.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person48">
>                       Anthony
>                       Dyer
>                    <interp value="Dyer" type="surname" inst="t17280717-5-person48"/>
>                          <interp value="Anthony" type="given" inst="t17280717-5-person48"/>
>                          <interp value="male" type="gender" inst="t17280717-5-person48"/>
>                       </persName>
>                  depos'd, That the Prosecutor told him he had lost his Money being Drunk, that he had been in a Cellar and in a Vault, where he fell asleep, and gave a very odd account of the Adventure.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person49">
>                       George
>                       Smith
>                    <interp value="Smith" type="surname" inst="t17280717-5-person49"/>
>                          <interp value="George" type="given" inst="t17280717-5-person49"/>
>                          <interp value="male" type="gender" inst="t17280717-5-person49"/>
>                       </persName>
>                  depos'd, That the Prosecutor said he fell asleep upon a Vault, and could charge no Body.</p>
>                    <p>The Constable, the Watchman, and others severally depos'd, That the Morning after this happened, he said he could charge no particular person, but he would indict the House. The Prisoner having a very good Character from several reputable Witnesses, the Jury <rs type="verdictDescription" id="t17280717-5-verdict22">
>                          <interp value="notGuilty" type="verdictCategory" inst="t17280717-5-verdict22"/>
>                    acquitted
>                 </rs> her.</p>
>                 </div1>
>
>
>                 <div1 id="t17280717-6" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-6"/>
>                    <interp value="1728" type="year" inst="t17280717-6"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-6"/>
>                    <interp value="17280717" type="date" inst="t17280717-6"/>
>                    <join targets="t17280717-6-defend50 t17280717-6-off24 t17280717-6-verdict26" targOrder="Y" id="t17280717-6-off24-c46" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-6-defend50">
>                       Phillip
>                       Hilliard
>                    <interp value="Hilliard" type="surname" inst="t17280717-6-defend50"/>
>                          <interp value="Phillip" type="given" inst="t17280717-6-defend50"/>
>                          <interp value="male" type="gender" inst="t17280717-6-defend50"/>
>                       </persName>
>                 , of <placeName id="t17280717-6-defloc23">St. James's Westminster</placeName>
>                       <interp value="St. James's Westminster" type="placeName" inst="t17280717-6-defloc23"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-6-defloc23"/>
>                       <join targets="t17280717-6-defend50 t17280717-6-defloc23" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-6-off24">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-6-off24"/>
>                          <interp value="grandLarceny" type="offenceSubcategory" inst="t17280717-6-off24"/>
>                    feloniously stealing a Silver Spoon, value eleven Shillings, the Property of
>                          <persName type="victimName" id="t17280717-6-victim51">
>                             Abraham
>                             Mannio
>                          <interp value="Mannio" type="surname" inst="t17280717-6-victim51"/>
>                             <interp value="Abraham" type="given" inst="t17280717-6-victim51"/>
>                             <interp value="male" type="gender" inst="t17280717-6-victim51"/>
>                             <join targets="t17280717-6-off24 t17280717-6-victim51" targOrder="Y" result="offenceVictim"/>
>                          </persName>
>                        , on the <rs type="crimeDate" id="t17280717-6-cd25">27th of June</rs>
>                          <join targets="t17280717-6-off24 t17280717-6-cd25" targOrder="Y" result="offenceCrimeDate"/> last.
>                 </rs>
>                    </p>
>                    <p>Mr. Hayden depos'd, That the Prisoner brought the Spoon to him to pawn, and he suspecting it to be stolen, stopp'd him and carried him to the Round-House, where he confess'd he stole it at the Prosecutor's, he being invited there to Dinner with some Gentlemans Servants.</p>
>                    <p>
>
>                       <persName id="t17280717-6-person52">
>                       Robert
>                       Amey
>                    <interp value="Amey" type="surname" inst="t17280717-6-person52"/>
>                          <interp value="Robert" type="given" inst="t17280717-6-person52"/>
>                          <interp value="male" type="gender" inst="t17280717-6-person52"/>
>                       </persName>
>                  depos'd, That he attended the Gentlemens Servants at Dinner, and afterwards they miss'd a Spoon, that Mr. Haydon sent them word of the Spoon brought to him by the Prisoner, and he going to match it by the others, found it to be the same which they had lost, it being of the same Make and Mark: The Fact being thus plainly proved upon him, the Jury found him <rs type="verdictDescription" id="t17280717-6-verdict26">
>                          <interp value="guilty" type="verdictCategory" inst="t17280717-6-verdict26"/>
>                          <interp value="theftunder1s" type="verdictSubcategory" inst="t17280717-6-verdict26"/>
>                    guilty to the value of 10d.
>                 </rs>
>                    </p>
>                    <p>
>                       <rs type="punishmentDescription" id="t17280717-6-punish27">
>                          <interp value="transport" type="punishmentCategory" inst="t17280717-6-punish27"/>
>                          <join targets="t17280717-6-defend50 t17280717-6-punish27" targOrder="Y" result="defendantPunishment"/>
>
>                          <note>[Transportation. See summary.]</note>
>
>                       </rs>
>                    </p>
>                 </div1>
>
>
>                 <div1 id="t17280717-7" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-7"/>
>                    <interp value="1728" type="year" inst="t17280717-7"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-7"/>
>                    <interp value="17280717" type="date" inst="t17280717-7"/>
>                    <join targets="t17280717-7-defend53 t17280717-7-off29 t17280717-7-verdict31" targOrder="Y" id="t17280717-7-off29-c49" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-7-defend53">
>                       Robert
>                       Ashby
>                    <interp value="Ashby" type="surname" inst="t17280717-7-defend53"/>
>                          <interp value="Robert" type="given" inst="t17280717-7-defend53"/>
>                          <interp value="male" type="gender" inst="t17280717-7-defend53"/>
>                       </persName>
>                 , of <rs type="occupation" id="t17280717-7-deflabel28">St. Andrew's Holborn</rs>
>                       <join targets="t17280717-7-defend53 t17280717-7-deflabel28" targOrder="Y" result="persNameOccupation"/>, was indicted for <rs type="offenceDescription" id="t17280717-7-off29">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-7-off29"/>
>                          <interp value="grandLarceny" type="offenceSubcategory" inst="t17280717-7-off29"/>
>                    stealing a Gold Watch, value 16 l. a Chain, value 4 l. a Cornelian Seal set in Gold, value 10s. the Property of
>                          <persName type="victimName" id="t17280717-7-victim54">
>                             Nicholas
>                             Roper
>                          <interp value="Roper" type="surname" inst="t17280717-7-victim54"/>
>                             <interp value="Nicholas" type="given" inst="t17280717-7-victim54"/>
>                             <interp value="male" type="gender" inst="t17280717-7-victim54"/>
>                          </persName>
>                         from the Person of
>                          <persName type="victimName" id="t17280717-7-victim55">
>                             Phoebe
>                             Thickpenny
>                          <interp value="Thickpenny" type="surname" inst="t17280717-7-victim55"/>
>                             <interp value="Phoebe" type="given" inst="t17280717-7-victim55"/>
>                             <interp value="female" type="gender" inst="t17280717-7-victim55"/>
>                          </persName>
>
>
>                       </rs>, who depos'd, That her Mistress being at Little Chelsea, sent her to their House in Lad-lane, for the Watch and other Things she had occasion for, that she call'd at the Prisoner's in Castle Yard, Chick-Lane, as she was going towards Chelsea with the Watch and the Bundle, where she drank part of a Pint of Beer, some Tea 2 Quarterns of Brandy, and a Bottle of Cyder; that the Prisoner would go part of the Way home with her, and in <placeName id="t17280717-7-crimeloc30">Leather-Lane</placeName>
>                       <interp value="Leather-Lane" type="placeName" inst="t17280717-7-crimeloc30"/>
>                       <interp value="crimeLocation" type="type" inst="t17280717-7-crimeloc30"/>
>                       <join targets="t17280717-7-off29 t17280717-7-crimeloc30" targOrder="Y" result="offencePlace"/>, he said, faith they would not part Dry lips, and accordingly they went into a publick House and drank a Pint of Twopenny, and two Quarterns of Brandy, that she had the Watch then, and at the Door the Prisoner kiss'd her, and gave her a shilling for a Coach, she having out-staid her Time; that when he kiss'd her, he put one Hand around her Waist, but what he did with the other she could not tell, that she then cross'd the Way to another House, and immediately miss'd the Watch, and she was sure she had pinn'd it so to her Side, that she could not drop it.</p>
>                    <p>
>
>                       <persName id="t17280717-7-person56">
>                       Margaret
>                       Nelson
>                    <interp value="Nelson" type="surname" inst="t17280717-7-person56"/>
>                          <interp value="Margaret" type="given" inst="t17280717-7-person56"/>
>                          <interp value="female" type="gender" inst="t17280717-7-person56"/>
>                       </persName>
>                  depos'd, That the Prisoner and Phoebe Thickpenny, came to her House in Leather-Lane, and he call'd for a private Room, to which the Girl would not consent, that the Girl wanted to go to, &amp;c. and she went with her, when the Girl said, she had her Mistress's Gold Watch, and seeming to look on it, told her it was 7 o' Clock, but this Deponent did not see the Watch, yet she said, she did verily believe she heard it beat, that they soon parted, and in a Quarter of an Hour the maid returned, and said she had lost the Watch.</p>
>                    <p>The Prisoner said in his Defence, That he knew nothing of it, any further than she said it was her Mistress's, that they parted very good Friends, and she having been an old Sweetheart of his, laid her Head upon his Shoulder, and said, She could live and die there, which would have been no little Aggravation to his Crime, had he wrong'd so good Natur'd a Creature: But the Evidence against him being weak, and several appearing to his Character, the Jury <rs type="verdictDescription" id="t17280717-7-verdict31">
>                          <interp value="notGuilty" type="verdictCategory" inst="t17280717-7-verdict31"/>
>                    acquitted
>                 </rs> him. </p>
>                 </div1>
>
>
>                 <div1 id="t17280717-8" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-8"/>
>                    <interp value="1728" type="year" inst="t17280717-8"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-8"/>
>                    <interp value="17280717" type="date" inst="t17280717-8"/>
>                    <join targets="t17280717-8-defend57 t17280717-8-off33 t17280717-8-verdict35" targOrder="Y" id="t17280717-8-off33-c53" result="criminalCharge"/>
>                    <join targets="t17280717-8-defend58 t17280717-8-off33 t17280717-8-verdict35" targOrder="Y" id="t17280717-8-off33-c54" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-8-defend57">
>                       Joseph
>                       Plummer
>                    <interp value="Plummer" type="surname" inst="t17280717-8-defend57"/>
>                          <interp value="Joseph" type="given" inst="t17280717-8-defend57"/>
>                          <interp value="male" type="gender" inst="t17280717-8-defend57"/>
>                       </persName>
>                  , and
>                    <persName type="defendantName" id="t17280717-8-defend58">
>                       Henry
>                       Coleman
>                    <interp value="Coleman" type="surname" inst="t17280717-8-defend58"/>
>                          <interp value="Henry" type="given" inst="t17280717-8-defend58"/>
>                          <interp value="male" type="gender" inst="t17280717-8-defend58"/>
>                       </persName>
>                  , of <placeName id="t17280717-8-defloc32">Edmonton</placeName>
>                       <interp value="Edmont...

  

Quick Preparation
-----------------

#### Some examples to learn xml and scala in a hurry

In [ ]:
val p = new scala.xml.PrettyPrinter(80, 2)

p.format(elem)

  

>     p: scala.xml.PrettyPrinter = scala.xml.PrettyPrinter@4fb2a9c8
>     res4: String =
>     <TEI.2>
>       <text>
>         <body>
>           <div0 id="17280717" type="sessionsPaper">
>             <interp value="BAILEY" type="collection" inst="17280717"/>
>             <interp value="1728" type="year" inst="17280717"/>
>             <interp value="sessionsPapers/17280717" type="uri" inst="17280717"/>
>             <interp value="17280717" type="date" inst="17280717"/>
>             <xptr doc="17280717" type="transcription"/>
>             <div1 id="f17280717-1" type="frontMatter">
>               <interp value="BAILEY" type="collection" inst="f17280717-1"/>
>               <interp value="1728" type="year" inst="f17280717-1"/>
>               <interp value="sessionsPapers/17280717" type="uri" inst="f17280717-1"/>
>               <interp value="17280717" type="date" inst="f17280717-1"/>
>               <xptr doc="172807170001" type="pageFacsimile"/>
>               THE PROCEEDINGS AT THE Sessions of the Peace, and Oyer and Terminer for the City of LONDON: AND
>               <p>
>                 On the King's Commission of Goal-Delivery of Newgate, held at Justice-Hall in the Old Baily, for the CITY of LONDON and COUNTY of MIDDLESEX.
>               </p>
>               <p>
>                 On Wednesday, Thursday, and Friday, being the 17th, 18th, and 19th of July, 1728, in the Second Year of His MAJESTY's Reign.
>               </p>
>               <p>(Price Six Pence)</p>
>               <p>
>                 BEFORE the Right Honourable Sir
>                 <persName type="judiciaryName" id="f17280717-1-person1">
>                   EDWARD
>                          BECHER
>                   <interp value="BECHER" type="surname" inst="f17280717-1-person1"/>
>                   <interp value="EDWARD" type="given" inst="f17280717-1-person1"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person1"/>
>                 </persName>
>                 , Lord Mayor of the City of London; the Right Honourable the Lord Chief
>                 <persName type="judiciaryName" id="f17280717-1-person2">
>                   Baron
>                          Pengelly
>                   <interp value="Pengelly" type="surname" inst="f17280717-1-person2"/>
>                   <interp value="Baron" type="given" inst="f17280717-1-person2"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person2"/>
>                 </persName>
>                 ; Mr. Justice Reynolds; Mr.
>                 <persName type="judiciaryName" id="f17280717-1-person3">
>                   Baron
>                          Thompson
>                   <interp value="Thompson" type="surname" inst="f17280717-1-person3"/>
>                   <interp value="Baron" type="given" inst="f17280717-1-person3"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person3"/>
>                 </persName>
>                 , Recorder of the City of London; and
>                 <persName type="judiciaryName" id="f17280717-1-person4">
>                   John
>                          Raby
>                   <interp value="Raby" type="surname" inst="f17280717-1-person4"/>
>                   <interp value="John" type="given" inst="f17280717-1-person4"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person4"/>
>                 </persName>
>                 , Esq; Serjeant at Law; and other His Majesty's Justices of Goal-Delivery, and Oyer and Terminer aforesaid; Together with several of His Majesty's Justices of the Peace for the said City of London.
>               </p>
>               <p>London Jury.</p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person5">
>                   John
>                          Land
>                   <interp value="Land" type="surname" inst="f17280717-1-person5"/>
>                   <interp value="John" type="given" inst="f17280717-1-person5"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person5"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person6">
>                   Nathaniel
>                          Mason
>                   <interp value="Mason" type="surname" inst="f17280717-1-person6"/>
>                   <interp value="Nathaniel" type="given" inst="f17280717-1-person6"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person6"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person7">
>                   Benjamin
>                          Allibone
>                   <interp value="Allibone" type="surname" inst="f17280717-1-person7"/>
>                   <interp value="Benjamin" type="given" inst="f17280717-1-person7"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person7"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person8">
>                   Joseph
>                          Westwood
>                   <interp value="Westwood" type="surname" inst="f17280717-1-person8"/>
>                   <interp value="Joseph" type="given" inst="f17280717-1-person8"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person8"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person9">
>                   Gabriel
>                          Wittacre
>                   <interp value="Wittacre" type="surname" inst="f17280717-1-person9"/>
>                   <interp value="Gabriel" type="given" inst="f17280717-1-person9"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person9"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person10">
>                   Samuel
>                          Tilley
>                   <interp value="Tilley" type="surname" inst="f17280717-1-person10"/>
>                   <interp value="Samuel" type="given" inst="f17280717-1-person10"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person10"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person11">
>                   Robert
>                          Lathwell
>                   <interp value="Lathwell" type="surname" inst="f17280717-1-person11"/>
>                   <interp value="Robert" type="given" inst="f17280717-1-person11"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person11"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person12">
>                   Edward
>                          Newman
>                   <interp value="Newman" type="surname" inst="f17280717-1-person12"/>
>                   <interp value="Edward" type="given" inst="f17280717-1-person12"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person12"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person13">
>                   Nathaniel
>                          Pickering
>                   <interp value="Pickering" type="surname" inst="f17280717-1-person13"/>
>                   <interp value="Nathaniel" type="given" inst="f17280717-1-person13"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person13"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person14">
>                   Simon
>                          Tunks
>                   <interp value="Tunks" type="surname" inst="f17280717-1-person14"/>
>                   <interp value="Simon" type="given" inst="f17280717-1-person14"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person14"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person15">
>                   Thomas
>                          Maud
>                   <interp value="Maud" type="surname" inst="f17280717-1-person15"/>
>                   <interp value="Thomas" type="given" inst="f17280717-1-person15"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person15"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person16">
>                   John
>                          Bond
>                   <interp value="Bond" type="surname" inst="f17280717-1-person16"/>
>                   <interp value="John" type="given" inst="f17280717-1-person16"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person16"/>
>                 </persName>
>                 .
>               </p>
>               <p>Middlesex Jury.</p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person17">
>                   Elisha
>                          Impey
>                   <interp value="Impey" type="surname" inst="f17280717-1-person17"/>
>                   <interp value="Elisha" type="given" inst="f17280717-1-person17"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person17"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person18">
>                   Christopher
>                          Harris
>                   <interp value="Harris" type="surname" inst="f17280717-1-person18"/>
>                   <interp value="Christopher" type="given" inst="f17280717-1-person18"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person18"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person19">
>                   William
>                          Perkins
>                   <interp value="Perkins" type="surname" inst="f17280717-1-person19"/>
>                   <interp value="William" type="given" inst="f17280717-1-person19"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person19"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person20">
>                   Gilbert
>                          Watson
>                   <interp value="Watson" type="surname" inst="f17280717-1-person20"/>
>                   <interp value="Gilbert" type="given" inst="f17280717-1-person20"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person20"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person21">
>                   John
>                          Wells
>                   <interp value="Wells" type="surname" inst="f17280717-1-person21"/>
>                   <interp value="John" type="given" inst="f17280717-1-person21"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person21"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person22">
>                   William
>                          Carpenter
>                   <interp value="Carpenter" type="surname" inst="f17280717-1-person22"/>
>                   <interp value="William" type="given" inst="f17280717-1-person22"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person22"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person23">
>                   Allen
>                          Evans
>                   <interp value="Evans" type="surname" inst="f17280717-1-person23"/>
>                   <interp value="Allen" type="given" inst="f17280717-1-person23"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person23"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person24">
>                   Henry
>                          Cowmbe
>                   <interp value="Cowmbe" type="surname" inst="f17280717-1-person24"/>
>                   <interp value="Henry" type="given" inst="f17280717-1-person24"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person24"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person25">
>                   Simon
>                          Parsons
>                   <interp value="Parsons" type="surname" inst="f17280717-1-person25"/>
>                   <interp value="Simon" type="given" inst="f17280717-1-person25"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person25"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person26">
>                   George
>                          Gilbert
>                   <interp value="Gilbert" type="surname" inst="f17280717-1-person26"/>
>                   <interp value="George" type="given" inst="f17280717-1-person26"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person26"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person27">
>                   Nicholas
>                          Gardner
>                   <interp value="Gardner" type="surname" inst="f17280717-1-person27"/>
>                   <interp value="Nicholas" type="given" inst="f17280717-1-person27"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person27"/>
>                 </persName>
>                 ,
>               </p>
>               <p>
>                 <persName type="jurorName" id="f17280717-1-person28">
>                   Thomas
>                          Ireland
>                   <interp value="Ireland" type="surname" inst="f17280717-1-person28"/>
>                   <interp value="Thomas" type="given" inst="f17280717-1-person28"/>
>                   <interp value="male" type="gender" inst="f17280717-1-person28"/>
>                 </persName>
>                 .
>               </p>
>             </div1>
>             <div1 id="t17280717-1" type="trialAccount">
>               <interp value="BAILEY" type="collection" inst="t17280717-1"/>
>               <interp value="1728" type="year" inst="t17280717-1"/>
>               <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-1"/>
>               <interp value="17280717" type="date" inst="t17280717-1"/>
>               <join
>               targets="t17280717-1-defend29 t17280717-1-off2 t17280717-1-verdict5" targOrder="Y" id="t17280717-1-off2-c29" result="criminalCharge">
>     </join>
>               <p>
>                 <persName type="defendantName" id="t17280717-1-defend29">
>                   James
>                       Haddock
>                   <interp value="Haddock" type="surname" inst="t17280717-1-defend29"/>
>                   <interp value="James" type="given" inst="t17280717-1-defend29"/>
>                   <interp value="male" type="gender" inst="t17280717-1-defend29"/>
>                 </persName>
>                 , of
>                 <placeName id="t17280717-1-defloc1">St. Bennet's Paul's Wharf</placeName>
>                 <interp
>                 value="St. Bennet's Paul's Wharf" type="placeName" inst="t17280717-1-defloc1">
>     </interp>
>                 <interp value="defendantHome" type="type" inst="t17280717-1-defloc1"/>
>                 <join
>                 targets="t17280717-1-defend29 t17280717-1-defloc1" targOrder="Y" result="persNamePlace">
>     </join>
>                 , was indicted for
>                 <rs type="offenceDescription" id="t17280717-1-off2">
>                   <interp value="theft" type="offenceCategory" inst="t17280717-1-off2"/>
>                   <interp
>                   value="theftFromPlace" type="offenceSubcategory" inst="t17280717-1-off2">
>     </interp>
>                   feloniously stealing 2 Guineas, and 5 l. 16 s. in Silver, a Silver Cup, a Silver Cork Screw, 2 Silver Spoons, and a Nutmeg-grater, in the Dwelling-House of
>                   <persName type="victimName" id="t17280717-1-victim31">
>                     James
>                                Reeves
>                     <interp value="Reeves" type="surname" inst="t17280717-1-victim31"/>
>                     <interp value="James" type="given" inst="t17280717-1-victim31"/>
>                     <interp value="male" type="gender" inst="t17280717-1-victim31"/>
>                     <join
>                     targets="t17280717-1-off2 t17280717-1-victim31" targOrder="Y" result="offenceVictim">
>     </join>
>                   </persName>
>                 </rs>
>                 , on the
>                 <rs type="crimeDate" id="t17280717-1-cd3">
>                   9th of April, and in the 13th Year of his late Majesty King George the First
>                 </rs>
>                 <join
>                 targets="t17280717-1-off2 t17280717-1-cd3" targOrder="Y" result="offenceCrimeDate">
>     </join>
>                 , the Property of
>                 <persName id="t17280717-1-person32">
>                   James
>                       Reeves
>                   <interp value="Reeves" type="surname" inst="t17280717-1-person32"/>
>                   <interp value="James" type="given" inst="t17280717-1-person32"/>
>                   <interp value="male" type="gender" inst="t17280717-1-person32"/>
>                 </persName>
>                 aforesaid.
>               </p>
>               <p>
>                 <persName id="t17280717-1-person33">
>                   Elizabeth
>                       Reeves
>                   <interp value="Reeves" type="surname" inst="t17280717-1-person33"/>
>                   <interp value="Elizabeth" type="given" inst="t17280717-1-person33"/>
>                   <interp value="female" type="gender" inst="t17280717-1-person33"/>
>                 </persName>
>                 depos'd, That the Prisoner was a Lodger at her House on
>                 <placeName id="t17280717-1-crimeloc4">Addle-Hill</placeName>
>                 <interp value="Addle-Hill" type="placeName" inst="t17280717-1-crimeloc4"/>
>                 <interp value="crimeLocation" type="type" inst="t17280717-1-crimeloc4"/>
>                 <join
>                 targets="t17280717-1-off2 t17280717-1-crimeloc4" targOrder="Y" result="offencePlace">
>     </join>
>                 , near Doctors Commons, when this Robbery was committed, and that it being on the Sabbath Day, she desired the Prisoner, if he did not go abroad, to have an Eye to her Room, which she locked up, and which, he promised to have an Eye to; but when she came home, the Chamber Door and the Corner-Cupboard had been forced open, which appeared by the Mark of an Instrument, the Hinges tore off and the Money gone, the Drawers rifled, and the Plate taken out, though some of the Drawers, out of which the Plate was taken, she left lock'd when she went from Home; upon which she cried out, saying, she was robb'd, and the Prisoner's Wife being above Stairs, came down, and said, her Husband was gone out, that he had been guilty of Failings, and desired her to be easy and she would Work early and late to make Satisfaction, though she did not know he had taken the Things, but his not coming home again confirmed them the more in this Suspicion; and there were other Witnesses, who depos'd, That the Prosecutor left the Prisoner in Care of her Door, and that he promised to look after it.
>               </p>
>               <p>
>                 The Prisoner said in his Defence, That Mrs. Reeves had lost a pair of Silver Buckles out of her Drawers a Fortnight before this Robbery, and she said, she did believe it was done by a Char-woman that she employ'd, which Mrs. Reeves acknowledging, and the Prisoner telling a plausible Story of his being in bad Circumstances, and that Day the Robbery was committed, one Diston pittying his Case, lent him Money to go down to Bristol, to Trade there, if possible to retrieve himself, and he not daring to go out of Doors in the Week Days, went out of the House unfortunately on that Afternoon: There being no positive Evidence against him, the Jury
>                 <rs type="verdictDescription" id="t17280717-1-verdict5">
>                   <interp value="notGuilty" type="verdictCategory" inst="t17280717-1-verdict5"/>
>                   acquitted
>                 </rs>
>                 him.
>               </p>
>             </div1>
>             <div1 id="t17280717-2" type="trialAccount">
>               <interp value="BAILEY" type="collection" inst="t17280717-2"/>
>               <interp value="1728" type="year" inst="t17280717-2"/>
>               <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-2"/>
>               <interp value="17280717" type="date" inst="t17280717-2"/>
>               <join
>               targets="t17280717-2-defend35 t17280717-2-off6 t17280717-2-verdict7" targOrder="Y" id="t17280717-2-off6-c33" result="criminalCharge">
>     </join>
>               <p>
>                 <persName type="defendantName" id="t17280717-2-defend35">
>                   David
>                          Ball
>                   <interp value="Ball" type="surname" inst="t17280717-2-defend35"/>
>                   <interp value="David" type="given" inst="t17280717-2-defend35"/>
>                   <interp value="male" type="gender" inst="t17280717-2-defend35"/>
>                 </persName>
>                 was indicted for
>                 <rs type="offenceDescription" id="t17280717-2-off6">
>                   <interp value="theft" type="offenceCategory" inst="t17280717-2-off6"/>
>                   <interp value="pettyLarceny" type="offenceSubcategory" inst="t17280717-2-off6">
>                   </interp>
>                   a Petty Larceny, in stealing a Yard and a Quarter of Linnen Cloth, value 11 d.
>                 </rs>
>                 the Goods of
>                 <persName type="victimName" id="t17280717-2-victim37">
>                   John
>                          Williams
>                   <interp value="Williams" type="surname" inst="t17280717-2-victim37"/>
>                   <interp value="John" type="given" inst="t17280717-2-victim37"/>
>                   <interp value="male" type="gender" inst="t17280717-2-victim37"/>
>                 </persName>
>                 and
>                 <persName type="victimName" id="t17280717-2-victim39">
>                   William
>                          Williams
>                   <interp value="Williams" type="surname" inst="t17280717-2-victim39"/>
>                   <interp value="William" type="given" inst="t17280717-2-victim39"/>
>                   <interp value="male" type="gender" inst="t17280717-2-victim39"/>
>                 </persName>
>                 ; to which Indictment he
>                 <rs type="verdictDescription" id="t17280717-2-verdict7">
>                   <interp value="guilty" type="verdictCategory" inst="t17280717-2-verdict7"/>
>                   <interp
>                   value="pleadedGuilty" type="verdictSubcategory" inst="t17280717-2-verdict7">
>     </interp>
>                   pleaded Guilty
>                 </rs>
>                 .
>               </p>
>               <p>
>                 <rs type="punishmentDescription" id="t17280717-2-punish8">
>                   <interp value="transport" type="punishmentCategory" inst="t17280717-2-punish8">
>                   </interp>
>                   <join
>                   targets="t17280717-2-defend35 t17280717-2-punish8" targOrder="Y" result="defendantPunishment">
>     </join>
>                   <note>[Transportation. See summary.]</note>
>                 </rs>
>               </p>
>             </div1>
>             <div1 id="t17280717-3" type="trialAccount">
>               <interp value="BAILEY" type="collection" inst="t17280717-3"/>
>               <interp value="1728" type="year" inst="t17280717-3"/>
>               <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-3"/>
>               <interp value="17280717" type="date" inst="t17280717-3"/>
>               <join
>               targets="t17280717-3-defend40 t17280717-3-off10 t17280717-3-verdict11" targOrder="Y" id="t17280717-3-off10-c36" result="criminalCharge">
>     </join>
>               <p>
>                 <xptr doc="172807170002" type="pageFacsimile"/>
>                 <persName type="defendantName" id="t17280717-3-defend40">
>                   Nathaniel
>                       Mercy
>                   <interp value="Mercy" type="surname" inst="t17280717-3-defend40"/>
>                   <interp value="Nathaniel" type="given" inst="t17280717-3-defend40"/>
>                   <interp value="male" type="gender" inst="t17280717-3-defend40"/>
>                 </persName>
>                 , of
>                 <placeName id="t17280717-3-defloc9">St. James's Westminster</placeName>
>                 <interp
>                 value="St. James's Westminster" type="placeName" inst="t17280717-3-defloc9">
>     </interp>
>                 <interp value="defendantHome" type="type" inst="t17280717-3-defloc9"/>
>                 <join
>                 targets="t17280717-3-defend40 t17280717-3-defloc9" targOrder="Y" result="persNamePlace">
>     </join>
>                 , was indicted for
>                 <rs type="offenceDescription" id="t17280717-3-off10">
>                   <interp value="theft" type="offenceCategory" inst="t17280717-3-off10"/>
>                   <interp
>                   value="grandLarceny" type="offenceSubcategory" inst="t17280717-3-off10">
>     </interp>
>                   stealing a Coach Wheel, value 12 Shillings
>                 </rs>
>                 , the Goods of
>                 <persName type="victimName" id="t17280717-3-victim41">
>                   Thomas
>                       West
>                   <interp value="West" type="surname" inst="t17280717-3-victim41"/>
>                   <interp value="Thomas" type="given" inst="t17280717-3-victim41"/>
>                   <interp value="male" type="gender" inst="t17280717-3-victim41"/>
>                   <join
>                   targets="t17280717-3-off10 t17280717-3-victim41" targOrder="Y" result="offenceVictim">
>     </join>
>                 </persName>
>                 , on the 9th of this Instant, to which Indictment he
>                 <rs type="verdictDescription" id="t17280717-3-verdict11">
>                   <interp value="guilty" type="verdictCategory" inst="t17280717-3-verdict11"/>
>                   <interp
>                   value="pleadedGuilty" type="verdictSubcategory" inst="t17280717-3-verdict11">
>     </interp>
>                   pleaded guilty
>                 </rs>
>                 .
>               </p>
>               <p>
>                 <rs type="punishmentDescription" id="t17280717-3-punish12">
>                   <interp
>                   value="transport" type="punishmentCategory" inst="t17280717-3-punish12">
>     </interp>
>                   <join
>                   targets="t17280717-3-defend40 t17280717-3-punish12" targOrder="Y" result="defendantPunishment">
>     </join>
>                   <note>[Transportation. See summary.]</note>
>                 </rs>
>               </p>
>             </div1>
>             <div1 id="t17280717-4" type="trialAccount">
>               <interp value="BAILEY" type="collection" inst="t17280717-4"/>
>               <interp value="1728" type="year" inst="t17280717-4"/>
>               <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-4"/>
>               <interp value="17280717" type="date" inst="t17280717-4"/>
>               <join
>               targets="t17280717-4-defend42 t17280717-4-off14 t17280717-4-verdict17" targOrder="Y" id="t17280717-4-off14-c38" result="criminalCharge">
>     </join>
>               <p>
>                 <persName type="defendantName" id="t17280717-4-defend42">
>                   Margaret
>                       King
>                   <interp value="King" type="surname" inst="t17280717-4-defend42"/>
>                   <interp value="Margaret" type="given" inst="t17280717-4-defend42"/>
>                   <interp value="female" type="gender" inst="t17280717-4-defend42"/>
>                 </persName>
>                 , of
>                 <placeName id="t17280717-4-defloc13">St. Ann's Westminster</placeName>
>                 <interp
>                 value="St. Ann's Westminster" type="placeName" inst="t17280717-4-defloc13">
>     </interp>
>                 <interp value="defendantHome" type="type" inst="t17280717-4-defloc13"/>
>                 <join
>                 targets="t17280717-4-defend42 t17280717-4-defloc13" targOrder="Y" result="persNamePlace">
>     </join>
>                 , was indicted for
>                 <rs type="offenceDescription" id="t17280717-4-off14">
>                   <interp value="theft" type="offenceCategory" inst="t17280717-4-off14"/>
>                   <interp
>                   value="grandLarceny" type="offenceSubcategory" inst="t17280717-4-off14">
>     </interp>
>                   privately stealing a Gold Watch, value 16 l.
>                 </rs>
>                 on the
>                 <rs type="crimeDate" id="t17280717-4-cd15">first of May</rs>
>                 <join
>                 targets="t17280717-4-off14 t17280717-4-cd15" targOrder="Y" result="offenceCrimeDate">
>     </join>
>                 last, the Property of
>                 <persName type="victimName" id="t17280717-4-victim43">
>                   Timothy
>                       Conner
>                   <interp value="Conner" type="surname" inst="t17280717-4-victim43"/>
>                   <interp value="Timothy" type="given" inst="t17280717-4-victim43"/>
>                   <interp value="male" type="gender" inst="t17280717-4-victim43"/>
>                   <join
>                   targets="t17280717-4-off14 t17280717-4-victim43" targOrder="Y" result="offenceVictim">
>     </join>
>                 </persName>
>                 .
>               </p>
>               <p>
>                 The Prosecutor depos'd, That he met the Prisoner in the Street, and ask'd her to drink a Glass of Wine, to which she consented, and they went to the
>                 <placeName id="t17280717-4-crimeloc16">
>                   Swan Tavern in Newport Market
>                 </placeName>
>                 <interp
>                 value="Swan Tavern in Newport Market" type="placeName" inst="t17280717-4-crimeloc16">
>     </interp>
>                 <interp value="crimeLocation" type="type" inst="t17280717-4-crimeloc16"/>
>                 <join
>                 targets="t17280717-4-off14 t17280717-4-crimeloc16" targOrder="Y" result="offencePlace">
>     </join>
>                 , and lovingly drank 4 Pints, the Prisoner asking him what it was a Clock, he pulled out his Gold Watch, and bid her look; that she took it in her Hand, but could not remember that she returned it again, neither could she say positively, that she had it, but as he mis'd it as soon as he parted with her he thought he had occasion of Suspicion; yet, said he, I had been drinking before, (tho' it was but Nine in the Morning)and can't tell directly how the Matter stood; which being all the Evidence he could give against her, she was
>                 <rs type="verdictDescription" id="t17280717-4-verdict17">
>                   <interp value="notGuilty" type="verdictCategory" inst="t17280717-4-verdict17"/>
>                   acquitted
>                 </rs>
>                 .
>               </p>
>             </div1>
>             <div1 id="t17280717-5" type="trialAccount">
>               <interp value="BAILEY" type="collection" inst="t17280717-5"/>
>               <interp value="1728" type="year" inst="t17280717-5"/>
>               <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-5"/>
>               <interp value="17280717" type="date" inst="t17280717-5"/>
>               <join
>               targets="t17280717-5-defend44 t17280717-5-off19 t17280717-5-verdict22" targOrder="Y" id="t17280717-5-off19-c40" result="criminalCharge">
>     </join>
>               <p>
>                 <persName type="defendantName" id="t17280717-5-defend44">
>                   Elizabeth
>                       Mould
>                   <interp value="Mould" type="surname" inst="t17280717-5-defend44"/>
>                   <interp value="Elizabeth" type="given" inst="t17280717-5-defend44"/>
>                   <interp value="female" type="gender" inst="t17280717-5-defend44"/>
>                 </persName>
>                 , of
>                 <placeName id="t17280717-5-defloc18">St. Martins in the Fields</placeName>
>                 <interp
>                 value="St. Martins in the Fields" type="placeName" inst="t17280717-5-defloc18">
>     </interp>
>                 <interp value="defendantHome" type="type" inst="t17280717-5-defloc18"/>
>                 <join
>                 targets="t17280717-5-defend44 t17280717-5-defloc18" targOrder="Y" result="persNamePlace">
>     </join>
>                 , was indicted for
>                 <rs type="offenceDescription" id="t17280717-5-off19">
>                   <interp value="theft" type="offenceCategory" inst="t17280717-5-off19"/>
>                   <interp
>                   value="pocketpicking" type="offenceSubcategory" inst="t17280717-5-off19">
>     </interp>
>                   privately stealing Fifty Pounds and eight Shillings, from the Person of
>                   <persName type="victimName" id="t17280717-5-victim45">
>                     John
>                             Coxall
>                     <interp value="Coxall" type="surname" inst="t17280717-5-victim45"/>
>                     <interp value="John" type="given" inst="t17280717-5-victim45"/>
>                     <interp value="male" type="gender" inst="t17280717-5-victim45"/>
>                     <join
>                     targets="t17280717-5-off19 t17280717-5-victim45" targOrder="Y" result="offenceVictim">
>     </join>
>                   </persName>
>                 </rs>
>                 , on the
>                 <rs type="crimeDate" id="t17280717-5-cd20">24th of June</rs>
>                 <join
>                 targets="t17280717-5-off19 t17280717-5-cd20" targOrder="Y" result="offenceCrimeDate">
>     </join>
>                 last
>               </p>
>               <p>
>                 The Prosecutor depos'd, That he being a
>                 <rs type="occupation" id="t17280717-5-viclabel21">Bricklayer</rs>
>                 <join
>                 targets="t17280717-5-victim45 t17280717-5-viclabel21" targOrder="Y" result="persNameOccupation">
>     </join>
>                 , was endeavouring to get some Business to do, at the House where the Prisoner lived, and it being a Chandler's Shop, to obtain the Good-Will of the People, he call'd for several Drams, and treated the Mistress of the House and one Mrs. Greaves, who was his Customer, with Cyder, Brandy, &amp;c. that whilst they were drinking the Prisoner came into the Room, and he likewise treated her, and she, in return, wip'd him over the Face, and seem'd very fond of him, saying, she would give him a fine Nosegay, and something to cheer his Heart, if he would go with her to Covent-Garden Market, and then (as his Expression was) she weagled him down into the Cellar, and there kept him lock'd up in a back Passage, that he was very much in Liquor, and scarce sensible of what he did, but he found what she had done to his Sorrow, for she had taken all his Money, which he missing, made a Noise, and the People of the House hearing him, let him out of his Place of Confinement, by conducting him up the Back Stairs; that he got Officers and search'd the Cellar, but could not find the Prisoner. The Prisoner desiring he might be ask'd if they did not drink together in the Cellar, the Prosecutor answer'd, No, she would fetch no Drink, saying, she did not care to be seen by the Publicans; that she was all for dry Money, and she, and None but she, had his Fifty Pounds, for save only him and her, there were neither Man, Woman, nor Child, nor Dog nor Cat in the Cellar.
>               </p>
>               <p>
>                 <persName id="t17280717-5-person46">
>                   Elizabeth
>                       Harper
>                   <interp value="Harper" type="surname" inst="t17280717-5-person46"/>
>                   <interp value="Elizabeth" type="given" inst="t17280717-5-person46"/>
>                   <interp value="female" type="gender" inst="t17280717-5-person46"/>
>                 </persName>
>                 depos'd, That the Prosecutor came into her Shop, and drank Cyder, Usquebaugh and Brandy, and being disguis'd in Liquor he pull'd out eight or ten Guineas and said, he was no Scoundrel; that she and Mrs. Greaves, whom he brought in to treat, begg'd he would put up his Money and take Care of it, and about that Time the Prisoner came into the Room, and familiarly stroaking his cheeks, persuaded him to go into the Cellar, saying, he should pay his Footing, that they staid half an Hour below, and this Deponent looking down, saw the Prisoner's Mother there, that he came up with her again, and treated her with Usquebaugh, and at 11 o' Clock, which was several Hours after, they heard him in a back Passage, where the House was supplied with Water, and letting him into their House, he said he was robb'd, at which this Deponent's Husband thrust him out of doors.
>               </p>
>               <p>
>                 <persName id="t17280717-5-person47">
>                   Isabella
>                       Greaves
>                   <interp value="Greaves" type="surname" inst="t17280717-5-person47"/>
>                   <interp value="Isabella" type="given" inst="t17280717-5-person47"/>
>                   <interp value="female" type="gender" inst="t17280717-5-person47"/>
>                 </persName>
>                 likewise confirm'd every Part of this Deposition, adding, That the Prisoner was not to be seen that Night after the Robbery.
>               </p>
>               <p>
>                 The Prisoner said in her Defence, That the Prosecutor went down into her Cellar, and behaved himself so rudely, that she was forced to threaten to send for an Officer, that she knew nothing of his Money, and had not gone away that Night, but as she was oblig'd by her Husband, and that she came next Morning and set her Greens out.
>               </p>
>               <p>
>                 <persName id="t17280717-5-person48">
>                   Anthony
>                       Dyer
>                   <interp value="Dyer" type="surname" inst="t17280717-5-person48"/>
>                   <interp value="Anthony" type="given" inst="t17280717-5-person48"/>
>                   <interp value="male" type="gender" inst="t17280717-5-person48"/>
>                 </persName>
>                 depos'd, That the Prosecutor told him he had lost his Money being Drunk, that he had been in a Cellar and in a Vault, where he fell asleep, and gave a very odd account of the Adventure.
>               </p>
>               <p>
>                 <persName id="t17280717-5-person49">
>                   George
>                       Smith
>                   <interp value="Smith" type="surname" inst="t17280717-5-person49"/>
>                   <interp value="George" type="given" inst="t17280717-5-person49"/>
>                   <interp value="male" type="gender" inst="t17280717-5-person49"/>
>                 </persName>
>                 depos'd, That the Prosecutor said he fell asleep upon a Vault, and could charge no Body.
>               </p>
>               <p>
>                 The Constable, the Watchman, and others severally depos'd, That the Morning after this happened, he said he could charge no particular person, but he would indict the House. The Prisoner having a very good Character from several reputable Witnesses, the Jury
>                 <rs type="verdictDescription" id="t17280717-5-verdict22">
>                   <interp value="notGuilty" type="verdictCategory" inst="t17280717-5-verdict22"/>
>                   acquitted
>                 </rs>
>                 her.
>               </p>
>             </div1>
>             <div1 id="t17280717-6" type="trialAccount">
>               <interp value="BAILEY" type="collection" inst="t17280717-6"/>
>               <interp value="1728" type="year" inst="t17280717-6"/>
>               <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-6"/>
>               <interp value="17280717" type="date" inst="t17280717-6"/>
>               <join
>               targets="t17280717-6-defend50 t17280717-6-off24 t17280717-6-verdict26" targOrder="Y" id="t17280717-6-off24-c46" result="criminalCharge">
>     </join>
>               <p>
>                 <persName type="defendantName" id="t17280717-6-defend50">
>                   Phillip
>                       Hilliard
>                   <interp value="Hilliard" type="surname" inst="t17280717-6-defend50"/>
>                   <interp value="Phillip" type="given" inst="t17280717-6-defend50"/>
>                   <interp value="male" type="gender" inst="t17280717-6-defend50"/>
>                 </persName>
>                 , of
>                 <placeName id="t17280717-6-defloc23">St. James's Westminster</placeName>
>                 <interp
>                 value="St. James's Westminster" type="placeName" inst="t17280717-6-defloc23">
>     </interp>
>                 <interp value="defendantHome" type="type" inst="t17280717-6-defloc23"/>
>                 <join
>                 targets="t17280717-6-defend50 t17280717-6-defloc23" targOrder="Y" result="persNamePlace">
>     </join>
>                 , was indicted for
>                 <rs type="offenceDescription" id="t17280717-6-off24">
>                   <interp value="theft" type="offenceCategory" inst="t17280717-6-off24"/>
>                   <interp
>                   value="grandLarceny" type="offenceSubcategory" inst="t17280717-6-off24">
>     </interp>
>                   feloniously stealing a Silver Spoon, value eleven Shillings, the Property of
>                   <persName type="victimName" id="t17280717-6-victim51">
>                     Abraham
>                             Mannio
>                     <interp value="Mannio" type="surname" inst="t17280717-6-victim51"/>
>                     <interp value="Abraham" type="given" inst="t17280717-6-victim51"/>
>                     <interp value="male" type="gender" inst="t17280717-6-victim51"/>
>                     <join
>                     targets="t17280717-6-off24 t17280717-6-victim51" targOrder="Y" result="offenceVictim">
>     </join>
>                   </persName>
>                   , on the
>                   <rs type="crimeDate" id="t17280717-6-cd25">27th of June</rs>
>                   <join
>                   targets="t17280717-6-off24 t17280717-6-cd25" targOrder="Y" result="offenceCrimeDate">
>     </join>
>                   last.
>                 </rs>
>               </p>
>               <p>
>                 Mr. Hayden depos'd, That the Prisoner brought the Spoon to him to pawn, and he suspecting it to be stolen, stopp'd him and carried him to the Round-House, where he confess'd he stole it at the Prosecutor's, he being invited there to Dinner with some Gentlemans Servants.
>               </p>
>               <p>
>                 <persName id="t17280717-6-person52">
>                   Robert
>                       Amey
>                   <interp value="Amey" type="surname" inst="t17280717-6-person52"/>
>                   <interp value="Robert" type="given" inst="t17280717-6-person52"/>
>                   <interp value="male" type="gender" inst="t17280717-6-person52"/>
>                 </persName>
>                 depos'd, That he attended the Gentlemens Servants at Dinner, and afterwards they miss'd a Spoon, that Mr. Haydon sent them word of the Spoon brought to him by the Prisoner, and he going to match it by the others, found it to be the same which they had lost, it being of the same Make and Mark: The Fact being thus plainly proved upon him, the Jury found him
>                 <rs type="verdictDescription" id="t17280717-6-verdict26">
>                   <interp value="guilty" type="verdictCategory" inst="t17280717-6-verdict26"/>
>                   <interp
>                   value="theftunder1s" type="verdictSubcategory" inst="t17280717-6-verdict26">
>     </interp>
>                   guilty to the value of 10d.
>                 </rs>
>               </p>
>               <p>
>                 <rs type="punishmentDescription" id="t17280717-6-punish27">
>                   <interp
>                   value="transport" type="punishmentCategory" inst="t17280717-6-punish27">
>     </interp>
>                   <join
>                   targets="t17280717-6-defend50 t17280717-6-punish27" targOrder="Y" result="defendantPunishment">
>     </join>
>                   <note>[Transportation. See summary.]</note>
>                 </rs>
>               </p>
>             </div1>
>             <div1 id="t17280717-7" type="trialAccount">
>               <interp value="BAILEY" type="collection" inst="t17280717-7"/>
>               <interp value="1728" type="year" inst="t17280717-7"/>
>               <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-7"/>
>               <interp value="17280717" type="date" inst="t17280717-7"/>
>               <join
>               targets="t17280717-7-defend53 t17280717-7-off29 t17280717-7-verdict31" targOrder="Y" id="t17280717-7-off29-c49" result="criminalCharge">
>     </join>
>               <p>
>                 <persName type="defendantName" id="t17280717-7-defend53">
>                   Robert
>                       Ashby
>                   <interp value="Ashby" type="surname" inst="t17280717-7-defend53"/>
>                   <interp value="Robert" type="given" inst="t17280717-7-defend53"/>
>                   <interp value="male" type="gender" inst="t17280717-7-defend53"/>
>                 </persName>
>                 , of
>                 <rs type="occupation" id="t17280717-7-deflabel28">St. Andrew's Holborn</rs>
>                 <join
>                 targets="t17280717-7-defend53 t17280717-7-deflabel28" targOrder="Y" result="persNameOccupation">
>     </join>
>                 , was indicted for
>                 <rs type="offenceDescription" id="t17280717-7-off29">
>                   <interp value="theft" type="offenceCategory" inst="t17280717-7-off29"/>
>                   <interp
>                   value="grandLarceny" type="offenceSubcategory" inst="t17280717-7-off29">
>     </interp>
>                   stealing a Gold Watch, value 16 l. a Chain, value 4 l. a Cornelian Seal set in Gold, value 10s. the Property of
>                   <persName type="victimName" id="t17280717-7-victim54">
>                     Nicholas
>                             Roper
>                     <interp value="Roper" type="surname" inst="t17280717-7-victim54"/>
>                     <interp value="Nicholas" type="given" inst="t17280717-7-victim54"/>
>                     <interp value="male" type="gender" inst="t17280717-7-victim54"/>
>                   </persName>
>                   from the Person of
>                   <persName type="victimName" id="t17280717-7-victim55">
>                     Phoebe
>                             Thickpenny
>                     <interp value="Thickpenny" type="surname" inst="t17280717-7-victim55"/>
>                     <interp value="Phoebe" type="given" inst="t17280717-7-victim55"/>
>                     <interp value="female" type="gender" inst="t17280717-7-victim55"/>
>                   </persName>
>                 </rs>
>                 , who depos'd, That her Mistress being at Little Chelsea, sent her to their House in Lad-lane, for the Watch and other Things she had occasion for, that she call'd at the Prisoner's in Castle Yard, Chick-Lane, as she was going towards Chelsea with the Watch and the Bundle, where she drank part of a Pint of Beer, some Tea 2 Quarterns of Brandy, and a Bottle of Cyder; that the Prisoner would go part of the Way home with her, and in
>                 <placeName id="t17280717-7-crimeloc30">Leather-Lane</placeName>
>                 <interp value="Leather-Lane" type="placeName" inst="t17280717-7-crimeloc30"/>
>                 <interp value="crimeLocation" type="type" inst="t17280717-7-crimeloc30"/>
>                 <join
>                 targets="t17280717-7-off29 t17280717-7-crimeloc30" targOrder="Y" result="offencePlace">
>     </join>
>                 , he said, faith they would not part Dry lips, and accordingly they went into a publick House and drank a Pint of Twopenny, and two Quarterns of Brandy, that she had the Watch then, and at the Door the Prisoner kiss'd her, and gave her a shilling for a Coach, she having out-staid her Time; that when he kiss'd her, he put one Hand around her Waist, but what he did with the other she could not tell, that she then cross'd the Way to another House, and immediately miss'd the Watch, and she was sure she had pinn'd it so to her Side, that she could not drop it.
>               </p>
>               <p>
>                 <persName id="t17280717-7-person56">
>                   Margaret
>                       Nelson
>                   <interp value="Nelson" type="surname" inst="t17280717-7-person56"/>
>                   <interp value="Margaret" type="given" inst="t17280717-7-person56"/>
>                   <interp value="female" type="gender" inst="t17280717-7-person56"/>
>                 </persName>
>                 depos'd, That the Prisoner and Phoebe Thickpenny, came to her House in Leather-Lane, and he call'd for a private Room, to which the Girl would not consent, that the Girl wanted to go to, &amp;c. and she went with her, when the Girl said, she had her Mistress's Gold Watch, and seeming to look on it, told her it was 7 o' Clock, but this Deponent did not see the Watch, yet she said, she did verily believe she heard it beat, that they soon parted, and in a Quarter of an Hour the maid returned, and said she had lost the Watch.
>               </p>
>               <p>
>                 The Prisoner said in his Defence, That he knew nothing of it, any further than she said it was her Mistress's, that they parted very good Friends, and she having been an old Sweetheart of his, laid her Head upon his Shoulder, and said, She could live and die there, which would have been no little Aggravation to his Crime, had he wrong'd so good Natur'd a Creature: But the Evidence against him being weak, and several appearing to his Character, the Jury
>                 <rs type="verdictDescription" id="t17280717-7-verdict31">
>                   <interp value="notGuilty" type="verdictCategory" inst="t17280717-7-verdict31"/>
>                   acquitted
>                 </rs>
>                 him.
>               </p>
>             </div1>
>             <div1 id="t17280717-8" type="trialAccount">
>               <interp value="BAILEY" type="collection" inst="t17280717-8"/>
>               <interp value="1728" type="year" inst="t17280717-8"/>
>               <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-8"/>
>               <interp value="17280717" type="date" inst="t17280717-8"/>
>               <join
>               targets="t17280717-8-defend57 t17280717-8-off33 t17280717-8-verdict35" targOrder="Y" id="t17280717-8-off33-c53" result="criminalCharge">
>     </join>
>               <join
>               targets="t17280717-8-defend58 t17280717-8-off33 t17280717-8-verdict35" targOrder="Y" id="t17280717-8-off33-c54" result="criminalCharge">
>     </join>
>               <p>
>                 <persName type="defendantName" id="t17280717-8-defend57">
>                   Joseph
>                       Plummer
>                   <interp value="Plummer" type="surname" inst="t17280717-8-defend57"/>
>                   <interp value="Joseph" type="given" inst="t17280717-8-defend57"/>
>                   <interp value="male" type="gender" inst="t17280717-8-defend57"/>
>                 </persName>
>                 , and
>                 <persName type="defendantName" id="t17280717-8-defend58">
>                   Henry
>                       Coleman
>                   <interp value="Coleman" type="surname" inst="t17280717-8-defend58"/>
>                   <interp value="Henry" type="given" inst="t17280717-8-defend58"/>
>                   <interp value="male" type="gender" inst="t17280717-8-defend58"/>
>                 </persName>
>                 , of
>                 <placeName id="t17280717-8-defloc32">Edmonton</placeName>
>                 <interp value="Edmonton" type="placeName" inst="t17280717-8-defloc32"/>
>                 <interp value="defendantHome" type="type" inst="t17280717-8-defloc32"/>
>                 <join
>                 targets="t17280717-8-defend57 t17280717-8-defloc32" targOrder="Y" result="persNamePlace">
>     </join>
>                 <join
>                 targets="t17280717-8-defend58 t17280717-8-defloc32" targOrder="Y" result="persNamePlace">
>     </join>
>                 , were indicted for
>                 <rs type="offenceDescription" id="t17280717-8-off33">
>                   <interp value="theft" type="offenceCategory" inst="t17280717-8-off33"/>
>                   <interp
>                   value="pettyLarceny" type="offenceSubcategory" inst="t17280717-8-off33">
>     </interp>
>                   stealing 2 Shirts, value 8d. a Silk Handkerchief, value 1d. and a Pair of Stockings, value 1d.
>       ...

  

### Better examples:

http://alvinalexander.com/scala/how-to-extract-data-from-xml-nodes-in-scala

http://alvinalexander.com/scala/scala-xml-xpath-example

#### More advanced topics:

https://alvinalexander.com/scala/serializing-deserializing-xml-scala-classes

#### XML to JSON, if you want to go this route:

https://stackoverflow.com/questions/9516973/xml-to-json-with-scala

Our Parsing Problem
-------------------

Let's dive deep on this data right away. See links above to learn xml
more systematically to be able to parse other subsets of the data for
your own project.

For now, we will jump in to parse the input data of counts used in
[Jasper Mackenzie, Raazesh Sainudiin, James Smithies and Heather
Wolffram, A nonparametric view of the civilizing process in London's Old
Bailey, Research Report UCDMS2015/1, 32 pages,
2015](http://lamastex.org/preprints/20150828_civilizingProcOBO.pdf).

In [ ]:
(elem \\ "div0").map(Node => (Node \ "@type").text) // types of div0 node, the singleton root node for the file

  

>     res5: scala.collection.immutable.Seq[String] = List(sessionsPaper)

In [ ]:
(elem \\ "div1").map(Node => (Node \ "@type").text) // types of div1 node

  

>     res6: scala.collection.immutable.Seq[String] = List(frontMatter, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, trialAccount, supplementaryMaterial)

In [ ]:
(elem \\ "div1")

  

>     res7: scala.xml.NodeSeq =
>     NodeSeq(<div1 id="f17280717-1" type="frontMatter">
>                    <interp value="BAILEY" type="collection" inst="f17280717-1"/>
>                    <interp value="1728" type="year" inst="f17280717-1"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="f17280717-1"/>
>                    <interp value="17280717" type="date" inst="f17280717-1"/>
>
>                    <xptr doc="172807170001" type="pageFacsimile"/>THE PROCEEDINGS AT THE Sessions of the Peace, and Oyer and Terminer for the City of LONDON: AND
>              <p>On the King's Commission of Goal-Delivery of Newgate, held at Justice-Hall in the Old Baily, for the CITY of LONDON and COUNTY of MIDDLESEX.</p>
>                    <p>On Wednesday, Thursday, and Friday, being the 17th, 18th, and 19th of July, 1728, in the Second Year of His MAJESTY's Reign.</p>
>                    <p>(Price Six Pence)</p>
>                    <p>BEFORE the Right Honourable Sir
>                       <persName type="judiciaryName" id="f17280717-1-person1">
>                          EDWARD
>                          BECHER
>                       <interp value="BECHER" type="surname" inst="f17280717-1-person1"/>
>                          <interp value="EDWARD" type="given" inst="f17280717-1-person1"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person1"/>
>                       </persName>
>                    , Lord Mayor of the City of London; the Right Honourable the Lord Chief
>                       <persName type="judiciaryName" id="f17280717-1-person2">
>                          Baron
>                          Pengelly
>                       <interp value="Pengelly" type="surname" inst="f17280717-1-person2"/>
>                          <interp value="Baron" type="given" inst="f17280717-1-person2"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person2"/>
>                       </persName>
>                    ; Mr. Justice Reynolds; Mr.
>                       <persName type="judiciaryName" id="f17280717-1-person3">
>                          Baron
>                          Thompson
>                       <interp value="Thompson" type="surname" inst="f17280717-1-person3"/>
>                          <interp value="Baron" type="given" inst="f17280717-1-person3"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person3"/>
>                       </persName>
>                    , Recorder of the City of London; and
>                       <persName type="judiciaryName" id="f17280717-1-person4">
>                          John
>                          Raby
>                       <interp value="Raby" type="surname" inst="f17280717-1-person4"/>
>                          <interp value="John" type="given" inst="f17280717-1-person4"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person4"/>
>                       </persName>
>                    , Esq; Serjeant at Law; and other His Majesty's Justices of Goal-Delivery, and Oyer and Terminer aforesaid; Together with several of His Majesty's Justices of the Peace for the said City of London.</p>
>
>                    <p>London Jury.</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person5">
>                          John
>                          Land
>                       <interp value="Land" type="surname" inst="f17280717-1-person5"/>
>                          <interp value="John" type="given" inst="f17280717-1-person5"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person5"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person6">
>                          Nathaniel
>                          Mason
>                       <interp value="Mason" type="surname" inst="f17280717-1-person6"/>
>                          <interp value="Nathaniel" type="given" inst="f17280717-1-person6"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person6"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person7">
>                          Benjamin
>                          Allibone
>                       <interp value="Allibone" type="surname" inst="f17280717-1-person7"/>
>                          <interp value="Benjamin" type="given" inst="f17280717-1-person7"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person7"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person8">
>                          Joseph
>                          Westwood
>                       <interp value="Westwood" type="surname" inst="f17280717-1-person8"/>
>                          <interp value="Joseph" type="given" inst="f17280717-1-person8"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person8"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person9">
>                          Gabriel
>                          Wittacre
>                       <interp value="Wittacre" type="surname" inst="f17280717-1-person9"/>
>                          <interp value="Gabriel" type="given" inst="f17280717-1-person9"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person9"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person10">
>                          Samuel
>                          Tilley
>                       <interp value="Tilley" type="surname" inst="f17280717-1-person10"/>
>                          <interp value="Samuel" type="given" inst="f17280717-1-person10"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person10"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person11">
>                          Robert
>                          Lathwell
>                       <interp value="Lathwell" type="surname" inst="f17280717-1-person11"/>
>                          <interp value="Robert" type="given" inst="f17280717-1-person11"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person11"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person12">
>                          Edward
>                          Newman
>                       <interp value="Newman" type="surname" inst="f17280717-1-person12"/>
>                          <interp value="Edward" type="given" inst="f17280717-1-person12"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person12"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person13">
>                          Nathaniel
>                          Pickering
>                       <interp value="Pickering" type="surname" inst="f17280717-1-person13"/>
>                          <interp value="Nathaniel" type="given" inst="f17280717-1-person13"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person13"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person14">
>                          Simon
>                          Tunks
>                       <interp value="Tunks" type="surname" inst="f17280717-1-person14"/>
>                          <interp value="Simon" type="given" inst="f17280717-1-person14"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person14"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person15">
>                          Thomas
>                          Maud
>                       <interp value="Maud" type="surname" inst="f17280717-1-person15"/>
>                          <interp value="Thomas" type="given" inst="f17280717-1-person15"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person15"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person16">
>                          John
>                          Bond
>                       <interp value="Bond" type="surname" inst="f17280717-1-person16"/>
>                          <interp value="John" type="given" inst="f17280717-1-person16"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person16"/>
>                       </persName>
>                    .</p>
>
>
>                    <p>Middlesex Jury.</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person17">
>                          Elisha
>                          Impey
>                       <interp value="Impey" type="surname" inst="f17280717-1-person17"/>
>                          <interp value="Elisha" type="given" inst="f17280717-1-person17"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person17"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person18">
>                          Christopher
>                          Harris
>                       <interp value="Harris" type="surname" inst="f17280717-1-person18"/>
>                          <interp value="Christopher" type="given" inst="f17280717-1-person18"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person18"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person19">
>                          William
>                          Perkins
>                       <interp value="Perkins" type="surname" inst="f17280717-1-person19"/>
>                          <interp value="William" type="given" inst="f17280717-1-person19"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person19"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person20">
>                          Gilbert
>                          Watson
>                       <interp value="Watson" type="surname" inst="f17280717-1-person20"/>
>                          <interp value="Gilbert" type="given" inst="f17280717-1-person20"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person20"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person21">
>                          John
>                          Wells
>                       <interp value="Wells" type="surname" inst="f17280717-1-person21"/>
>                          <interp value="John" type="given" inst="f17280717-1-person21"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person21"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person22">
>                          William
>                          Carpenter
>                       <interp value="Carpenter" type="surname" inst="f17280717-1-person22"/>
>                          <interp value="William" type="given" inst="f17280717-1-person22"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person22"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person23">
>                          Allen
>                          Evans
>                       <interp value="Evans" type="surname" inst="f17280717-1-person23"/>
>                          <interp value="Allen" type="given" inst="f17280717-1-person23"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person23"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person24">
>                          Henry
>                          Cowmbe
>                       <interp value="Cowmbe" type="surname" inst="f17280717-1-person24"/>
>                          <interp value="Henry" type="given" inst="f17280717-1-person24"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person24"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person25">
>                          Simon
>                          Parsons
>                       <interp value="Parsons" type="surname" inst="f17280717-1-person25"/>
>                          <interp value="Simon" type="given" inst="f17280717-1-person25"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person25"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person26">
>                          George
>                          Gilbert
>                       <interp value="Gilbert" type="surname" inst="f17280717-1-person26"/>
>                          <interp value="George" type="given" inst="f17280717-1-person26"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person26"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person27">
>                          Nicholas
>                          Gardner
>                       <interp value="Gardner" type="surname" inst="f17280717-1-person27"/>
>                          <interp value="Nicholas" type="given" inst="f17280717-1-person27"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person27"/>
>                       </persName>
>                    ,</p>
>                    <p>
>
>                       <persName type="jurorName" id="f17280717-1-person28">
>                          Thomas
>                          Ireland
>                       <interp value="Ireland" type="surname" inst="f17280717-1-person28"/>
>                          <interp value="Thomas" type="given" inst="f17280717-1-person28"/>
>                          <interp value="male" type="gender" inst="f17280717-1-person28"/>
>                       </persName>
>                    .</p>
>
>                 </div1>, <div1 id="t17280717-1" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-1"/>
>                    <interp value="1728" type="year" inst="t17280717-1"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-1"/>
>                    <interp value="17280717" type="date" inst="t17280717-1"/>
>                    <join targets="t17280717-1-defend29 t17280717-1-off2 t17280717-1-verdict5" targOrder="Y" id="t17280717-1-off2-c29" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-1-defend29">
>                       James
>                       Haddock
>                    <interp value="Haddock" type="surname" inst="t17280717-1-defend29"/>
>                          <interp value="James" type="given" inst="t17280717-1-defend29"/>
>                          <interp value="male" type="gender" inst="t17280717-1-defend29"/>
>                       </persName>
>                 , of <placeName id="t17280717-1-defloc1">St. Bennet's Paul's Wharf</placeName>
>                       <interp value="St. Bennet's Paul's Wharf" type="placeName" inst="t17280717-1-defloc1"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-1-defloc1"/>
>                       <join targets="t17280717-1-defend29 t17280717-1-defloc1" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-1-off2">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-1-off2"/>
>                          <interp value="theftFromPlace" type="offenceSubcategory" inst="t17280717-1-off2"/>
>                    feloniously stealing 2 Guineas, and 5 l. 16 s. in Silver, a Silver Cup, a Silver Cork Screw, 2 Silver Spoons, and a Nutmeg-grater, in the Dwelling-House of
>
>                             <persName type="victimName" id="t17280717-1-victim31">
>                                James
>                                Reeves
>                             <interp value="Reeves" type="surname" inst="t17280717-1-victim31"/>
>                             <interp value="James" type="given" inst="t17280717-1-victim31"/>
>                             <interp value="male" type="gender" inst="t17280717-1-victim31"/>
>                             <join targets="t17280717-1-off2 t17280717-1-victim31" targOrder="Y" result="offenceVictim"/>
>                          </persName>
>
>
>
>                       </rs>, on the <rs type="crimeDate" id="t17280717-1-cd3">9th of April, and in the 13th Year of his late Majesty King George the First</rs>
>                       <join targets="t17280717-1-off2 t17280717-1-cd3" targOrder="Y" result="offenceCrimeDate"/>, the Property of
>                    <persName id="t17280717-1-person32">
>                       James
>                       Reeves
>                    <interp value="Reeves" type="surname" inst="t17280717-1-person32"/>
>                          <interp value="James" type="given" inst="t17280717-1-person32"/>
>                          <interp value="male" type="gender" inst="t17280717-1-person32"/>
>                       </persName>
>                  aforesaid.</p>
>                    <p>
>
>                       <persName id="t17280717-1-person33">
>                       Elizabeth
>                       Reeves
>                    <interp value="Reeves" type="surname" inst="t17280717-1-person33"/>
>                          <interp value="Elizabeth" type="given" inst="t17280717-1-person33"/>
>                          <interp value="female" type="gender" inst="t17280717-1-person33"/>
>                       </persName>
>                  depos'd, That the Prisoner was a Lodger at her House on <placeName id="t17280717-1-crimeloc4">Addle-Hill</placeName>
>                       <interp value="Addle-Hill" type="placeName" inst="t17280717-1-crimeloc4"/>
>                       <interp value="crimeLocation" type="type" inst="t17280717-1-crimeloc4"/>
>                       <join targets="t17280717-1-off2 t17280717-1-crimeloc4" targOrder="Y" result="offencePlace"/>, near Doctors Commons, when this Robbery was committed, and that it being on the Sabbath Day, she desired the Prisoner, if he did not go abroad, to have an Eye to her Room, which she locked up, and which, he promised to have an Eye to; but when she came home, the Chamber Door and the Corner-Cupboard had been forced open, which appeared by the Mark of an Instrument, the Hinges tore off and the Money gone, the Drawers rifled, and the Plate taken out, though some of the Drawers, out of which the Plate was taken, she left lock'd when she went from Home; upon which she cried out, saying, she was robb'd, and the Prisoner's Wife being above Stairs, came down, and said, her Husband was gone out, that he had been guilty of Failings, and desired her to be easy and she would Work early and late to make Satisfaction, though she did not know he had taken the Things, but his not coming home again confirmed them the more in this Suspicion; and there were other Witnesses, who depos'd, That the Prosecutor left the Prisoner in Care of her Door, and that he promised to look after it.</p>
>                    <p>The Prisoner said in his Defence, That Mrs. Reeves had lost a pair of Silver Buckles out of her Drawers a Fortnight before this Robbery, and she said, she did believe it was done by a Char-woman that she employ'd, which Mrs. Reeves acknowledging, and the Prisoner telling a plausible Story of his being in bad Circumstances, and that Day the Robbery was committed, one Diston pittying his Case, lent him Money to go down to Bristol, to Trade there, if possible to retrieve himself, and he not daring to go out of Doors in the Week Days, went out of the House unfortunately on that Afternoon: There being no positive Evidence against him, the Jury <rs type="verdictDescription" id="t17280717-1-verdict5">
>                          <interp value="notGuilty" type="verdictCategory" inst="t17280717-1-verdict5"/>
>                    acquitted
>                 </rs> him.</p>
>                 </div1>, <div1 id="t17280717-2" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-2"/>
>                    <interp value="1728" type="year" inst="t17280717-2"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-2"/>
>                    <interp value="17280717" type="date" inst="t17280717-2"/>
>                    <join targets="t17280717-2-defend35 t17280717-2-off6 t17280717-2-verdict7" targOrder="Y" id="t17280717-2-off6-c33" result="criminalCharge"/>
>
>                    <p>
>
>
>                       <persName type="defendantName" id="t17280717-2-defend35">
>                          David
>                          Ball
>                       <interp value="Ball" type="surname" inst="t17280717-2-defend35"/>
>                          <interp value="David" type="given" inst="t17280717-2-defend35"/>
>                          <interp value="male" type="gender" inst="t17280717-2-defend35"/>
>                       </persName>
>
>                  was indicted for <rs type="offenceDescription" id="t17280717-2-off6">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-2-off6"/>
>                          <interp value="pettyLarceny" type="offenceSubcategory" inst="t17280717-2-off6"/>
>                    a Petty Larceny, in stealing a Yard and a Quarter of Linnen Cloth, value 11 d.
>                 </rs> the Goods of
>
>                       <persName type="victimName" id="t17280717-2-victim37">
>                          John
>                          Williams
>                       <interp value="Williams" type="surname" inst="t17280717-2-victim37"/>
>                          <interp value="John" type="given" inst="t17280717-2-victim37"/>
>                          <interp value="male" type="gender" inst="t17280717-2-victim37"/>
>                       </persName>
>
>                  and
>
>                       <persName type="victimName" id="t17280717-2-victim39">
>                          William
>                          Williams
>                       <interp value="Williams" type="surname" inst="t17280717-2-victim39"/>
>                          <interp value="William" type="given" inst="t17280717-2-victim39"/>
>                          <interp value="male" type="gender" inst="t17280717-2-victim39"/>
>                       </persName>
>
>                 ; to which Indictment he <rs type="verdictDescription" id="t17280717-2-verdict7">
>                          <interp value="guilty" type="verdictCategory" inst="t17280717-2-verdict7"/>
>                          <interp value="pleadedGuilty" type="verdictSubcategory" inst="t17280717-2-verdict7"/>
>                    pleaded Guilty
>                 </rs>.</p>
>                    <p>
>                       <rs type="punishmentDescription" id="t17280717-2-punish8">
>                          <interp value="transport" type="punishmentCategory" inst="t17280717-2-punish8"/>
>                          <join targets="t17280717-2-defend35 t17280717-2-punish8" targOrder="Y" result="defendantPunishment"/>
>
>                          <note>[Transportation. See summary.]</note>
>
>                       </rs>
>                    </p>
>                 </div1>, <div1 id="t17280717-3" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-3"/>
>                    <interp value="1728" type="year" inst="t17280717-3"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-3"/>
>                    <interp value="17280717" type="date" inst="t17280717-3"/>
>                    <join targets="t17280717-3-defend40 t17280717-3-off10 t17280717-3-verdict11" targOrder="Y" id="t17280717-3-off10-c36" result="criminalCharge"/>
>
>                    <p>
>                       <xptr doc="172807170002" type="pageFacsimile"/>
>
>                       <persName type="defendantName" id="t17280717-3-defend40">
>                       Nathaniel
>                       Mercy
>                    <interp value="Mercy" type="surname" inst="t17280717-3-defend40"/>
>                          <interp value="Nathaniel" type="given" inst="t17280717-3-defend40"/>
>                          <interp value="male" type="gender" inst="t17280717-3-defend40"/>
>                       </persName>
>                 , of <placeName id="t17280717-3-defloc9">St. James's Westminster</placeName>
>                       <interp value="St. James's Westminster" type="placeName" inst="t17280717-3-defloc9"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-3-defloc9"/>
>                       <join targets="t17280717-3-defend40 t17280717-3-defloc9" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-3-off10">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-3-off10"/>
>                          <interp value="grandLarceny" type="offenceSubcategory" inst="t17280717-3-off10"/>
>                    stealing a Coach Wheel, value 12 Shillings
>                 </rs>, the Goods of
>                    <persName type="victimName" id="t17280717-3-victim41">
>                       Thomas
>                       West
>                    <interp value="West" type="surname" inst="t17280717-3-victim41"/>
>                          <interp value="Thomas" type="given" inst="t17280717-3-victim41"/>
>                          <interp value="male" type="gender" inst="t17280717-3-victim41"/>
>                          <join targets="t17280717-3-off10 t17280717-3-victim41" targOrder="Y" result="offenceVictim"/>
>                       </persName>
>                  , on the 9th of this Instant, to which Indictment he <rs type="verdictDescription" id="t17280717-3-verdict11">
>                          <interp value="guilty" type="verdictCategory" inst="t17280717-3-verdict11"/>
>                          <interp value="pleadedGuilty" type="verdictSubcategory" inst="t17280717-3-verdict11"/>
>                    pleaded guilty
>                 </rs>.</p>
>                    <p>
>                       <rs type="punishmentDescription" id="t17280717-3-punish12">
>                          <interp value="transport" type="punishmentCategory" inst="t17280717-3-punish12"/>
>                          <join targets="t17280717-3-defend40 t17280717-3-punish12" targOrder="Y" result="defendantPunishment"/>
>
>                          <note>[Transportation. See summary.]</note>
>
>                       </rs>
>                    </p>
>                 </div1>, <div1 id="t17280717-4" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-4"/>
>                    <interp value="1728" type="year" inst="t17280717-4"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-4"/>
>                    <interp value="17280717" type="date" inst="t17280717-4"/>
>                    <join targets="t17280717-4-defend42 t17280717-4-off14 t17280717-4-verdict17" targOrder="Y" id="t17280717-4-off14-c38" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-4-defend42">
>                       Margaret
>                       King
>                    <interp value="King" type="surname" inst="t17280717-4-defend42"/>
>                          <interp value="Margaret" type="given" inst="t17280717-4-defend42"/>
>                          <interp value="female" type="gender" inst="t17280717-4-defend42"/>
>                       </persName>
>                 , of <placeName id="t17280717-4-defloc13">St. Ann's Westminster</placeName>
>                       <interp value="St. Ann's Westminster" type="placeName" inst="t17280717-4-defloc13"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-4-defloc13"/>
>                       <join targets="t17280717-4-defend42 t17280717-4-defloc13" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-4-off14">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-4-off14"/>
>                          <interp value="grandLarceny" type="offenceSubcategory" inst="t17280717-4-off14"/>
>                    privately stealing a Gold Watch, value 16 l.
>                 </rs> on the <rs type="crimeDate" id="t17280717-4-cd15">first of May</rs>
>                       <join targets="t17280717-4-off14 t17280717-4-cd15" targOrder="Y" result="offenceCrimeDate"/> last, the Property of
>                    <persName type="victimName" id="t17280717-4-victim43">
>                       Timothy
>                       Conner
>                    <interp value="Conner" type="surname" inst="t17280717-4-victim43"/>
>                          <interp value="Timothy" type="given" inst="t17280717-4-victim43"/>
>                          <interp value="male" type="gender" inst="t17280717-4-victim43"/>
>                          <join targets="t17280717-4-off14 t17280717-4-victim43" targOrder="Y" result="offenceVictim"/>
>                       </persName>
>                  .</p>
>                    <p>The Prosecutor depos'd, That he met the Prisoner in the Street, and ask'd her to drink a Glass of Wine, to which she consented, and they went to the <placeName id="t17280717-4-crimeloc16">Swan Tavern in Newport Market</placeName>
>                       <interp value="Swan Tavern in Newport Market" type="placeName" inst="t17280717-4-crimeloc16"/>
>                       <interp value="crimeLocation" type="type" inst="t17280717-4-crimeloc16"/>
>                       <join targets="t17280717-4-off14 t17280717-4-crimeloc16" targOrder="Y" result="offencePlace"/>, and lovingly drank 4 Pints, the Prisoner asking him what it was a Clock, he pulled out his Gold Watch, and bid her look; that she took it in her Hand, but could not remember that she returned it again, neither could she say positively, that she had it, but as he mis'd it as soon as he parted with her he thought he had occasion of Suspicion; yet, said he, I had been drinking before, (tho' it was but Nine in the Morning)and can't tell directly how the Matter stood; which being all the Evidence he could give against her, she was <rs type="verdictDescription" id="t17280717-4-verdict17">
>                          <interp value="notGuilty" type="verdictCategory" inst="t17280717-4-verdict17"/>
>                    acquitted
>                 </rs>.</p>
>                 </div1>, <div1 id="t17280717-5" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-5"/>
>                    <interp value="1728" type="year" inst="t17280717-5"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-5"/>
>                    <interp value="17280717" type="date" inst="t17280717-5"/>
>                    <join targets="t17280717-5-defend44 t17280717-5-off19 t17280717-5-verdict22" targOrder="Y" id="t17280717-5-off19-c40" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-5-defend44">
>                       Elizabeth
>                       Mould
>                    <interp value="Mould" type="surname" inst="t17280717-5-defend44"/>
>                          <interp value="Elizabeth" type="given" inst="t17280717-5-defend44"/>
>                          <interp value="female" type="gender" inst="t17280717-5-defend44"/>
>                       </persName>
>                 , of <placeName id="t17280717-5-defloc18">St. Martins in the Fields</placeName>
>                       <interp value="St. Martins in the Fields" type="placeName" inst="t17280717-5-defloc18"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-5-defloc18"/>
>                       <join targets="t17280717-5-defend44 t17280717-5-defloc18" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-5-off19">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-5-off19"/>
>                          <interp value="pocketpicking" type="offenceSubcategory" inst="t17280717-5-off19"/>
>                    privately stealing Fifty Pounds and eight Shillings, from the Person of
>                          <persName type="victimName" id="t17280717-5-victim45">
>                             John
>                             Coxall
>                          <interp value="Coxall" type="surname" inst="t17280717-5-victim45"/>
>                             <interp value="John" type="given" inst="t17280717-5-victim45"/>
>                             <interp value="male" type="gender" inst="t17280717-5-victim45"/>
>                             <join targets="t17280717-5-off19 t17280717-5-victim45" targOrder="Y" result="offenceVictim"/>
>                          </persName>
>
>
>                       </rs>, on the <rs type="crimeDate" id="t17280717-5-cd20">24th of June</rs>
>                       <join targets="t17280717-5-off19 t17280717-5-cd20" targOrder="Y" result="offenceCrimeDate"/> last</p>
>                    <p>The Prosecutor depos'd, That he being a <rs type="occupation" id="t17280717-5-viclabel21">Bricklayer</rs>
>                       <join targets="t17280717-5-victim45 t17280717-5-viclabel21" targOrder="Y" result="persNameOccupation"/>, was endeavouring to get some Business to do, at the House where the Prisoner lived, and it being a Chandler's Shop, to obtain the Good-Will of the People, he call'd for several Drams, and treated the Mistress of the House and one Mrs. Greaves, who was his Customer, with Cyder, Brandy, &amp;c. that whilst they were drinking the Prisoner came into the Room, and he likewise treated her, and she, in return, wip'd him over the Face, and seem'd very fond of him, saying, she would give him a fine Nosegay, and something to cheer his Heart, if he would go with her to Covent-Garden Market, and then (as his Expression was) she weagled him down into the Cellar, and there kept him lock'd up in a back Passage, that he was very much in Liquor, and scarce sensible of what he did, but he found what she had done to his Sorrow, for she had taken all his Money, which he missing, made a Noise, and the People of the House hearing him, let him out of his Place of Confinement, by conducting him up the Back Stairs; that he got Officers and search'd the Cellar, but could not find the Prisoner. The Prisoner desiring he might be ask'd if they did not drink together in the Cellar, the Prosecutor answer'd, No, she would fetch no Drink, saying, she did not care to be seen by the Publicans; that she was all for dry Money, and she, and None but she, had his Fifty Pounds, for save only him and her, there were neither Man, Woman, nor Child, nor Dog nor Cat in the Cellar.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person46">
>                       Elizabeth
>                       Harper
>                    <interp value="Harper" type="surname" inst="t17280717-5-person46"/>
>                          <interp value="Elizabeth" type="given" inst="t17280717-5-person46"/>
>                          <interp value="female" type="gender" inst="t17280717-5-person46"/>
>                       </persName>
>                  depos'd, That the Prosecutor came into her Shop, and drank Cyder, Usquebaugh and Brandy, and being disguis'd in Liquor he pull'd out eight or ten Guineas and said, he was no Scoundrel; that she and Mrs. Greaves, whom he brought in to treat, begg'd he would put up his Money and take Care of it, and about that Time the Prisoner came into the Room, and familiarly stroaking his cheeks, persuaded him to go into the Cellar, saying, he should pay his Footing, that they staid half an Hour below, and this Deponent looking down, saw the Prisoner's Mother there, that he came up with her again, and treated her with Usquebaugh, and at 11 o' Clock, which was several Hours after, they heard him in a back Passage, where the House was supplied with Water, and letting him into their House, he said he was robb'd, at which this Deponent's Husband thrust him out of doors.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person47">
>                       Isabella
>                       Greaves
>                    <interp value="Greaves" type="surname" inst="t17280717-5-person47"/>
>                          <interp value="Isabella" type="given" inst="t17280717-5-person47"/>
>                          <interp value="female" type="gender" inst="t17280717-5-person47"/>
>                       </persName>
>                  likewise confirm'd every Part of this Deposition, adding, That the Prisoner was not to be seen that Night after the Robbery.</p>
>                    <p>The Prisoner said in her Defence, That the Prosecutor went down into her Cellar, and behaved himself so rudely, that she was forced to threaten to send for an Officer, that she knew nothing of his Money, and had not gone away that Night, but as she was oblig'd by her Husband, and that she came next Morning and set her Greens out.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person48">
>                       Anthony
>                       Dyer
>                    <interp value="Dyer" type="surname" inst="t17280717-5-person48"/>
>                          <interp value="Anthony" type="given" inst="t17280717-5-person48"/>
>                          <interp value="male" type="gender" inst="t17280717-5-person48"/>
>                       </persName>
>                  depos'd, That the Prosecutor told him he had lost his Money being Drunk, that he had been in a Cellar and in a Vault, where he fell asleep, and gave a very odd account of the Adventure.</p>
>                    <p>
>
>                       <persName id="t17280717-5-person49">
>                       George
>                       Smith
>                    <interp value="Smith" type="surname" inst="t17280717-5-person49"/>
>                          <interp value="George" type="given" inst="t17280717-5-person49"/>
>                          <interp value="male" type="gender" inst="t17280717-5-person49"/>
>                       </persName>
>                  depos'd, That the Prosecutor said he fell asleep upon a Vault, and could charge no Body.</p>
>                    <p>The Constable, the Watchman, and others severally depos'd, That the Morning after this happened, he said he could charge no particular person, but he would indict the House. The Prisoner having a very good Character from several reputable Witnesses, the Jury <rs type="verdictDescription" id="t17280717-5-verdict22">
>                          <interp value="notGuilty" type="verdictCategory" inst="t17280717-5-verdict22"/>
>                    acquitted
>                 </rs> her.</p>
>                 </div1>, <div1 id="t17280717-6" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-6"/>
>                    <interp value="1728" type="year" inst="t17280717-6"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-6"/>
>                    <interp value="17280717" type="date" inst="t17280717-6"/>
>                    <join targets="t17280717-6-defend50 t17280717-6-off24 t17280717-6-verdict26" targOrder="Y" id="t17280717-6-off24-c46" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-6-defend50">
>                       Phillip
>                       Hilliard
>                    <interp value="Hilliard" type="surname" inst="t17280717-6-defend50"/>
>                          <interp value="Phillip" type="given" inst="t17280717-6-defend50"/>
>                          <interp value="male" type="gender" inst="t17280717-6-defend50"/>
>                       </persName>
>                 , of <placeName id="t17280717-6-defloc23">St. James's Westminster</placeName>
>                       <interp value="St. James's Westminster" type="placeName" inst="t17280717-6-defloc23"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-6-defloc23"/>
>                       <join targets="t17280717-6-defend50 t17280717-6-defloc23" targOrder="Y" result="persNamePlace"/>, was indicted for <rs type="offenceDescription" id="t17280717-6-off24">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-6-off24"/>
>                          <interp value="grandLarceny" type="offenceSubcategory" inst="t17280717-6-off24"/>
>                    feloniously stealing a Silver Spoon, value eleven Shillings, the Property of
>                          <persName type="victimName" id="t17280717-6-victim51">
>                             Abraham
>                             Mannio
>                          <interp value="Mannio" type="surname" inst="t17280717-6-victim51"/>
>                             <interp value="Abraham" type="given" inst="t17280717-6-victim51"/>
>                             <interp value="male" type="gender" inst="t17280717-6-victim51"/>
>                             <join targets="t17280717-6-off24 t17280717-6-victim51" targOrder="Y" result="offenceVictim"/>
>                          </persName>
>                        , on the <rs type="crimeDate" id="t17280717-6-cd25">27th of June</rs>
>                          <join targets="t17280717-6-off24 t17280717-6-cd25" targOrder="Y" result="offenceCrimeDate"/> last.
>                 </rs>
>                    </p>
>                    <p>Mr. Hayden depos'd, That the Prisoner brought the Spoon to him to pawn, and he suspecting it to be stolen, stopp'd him and carried him to the Round-House, where he confess'd he stole it at the Prosecutor's, he being invited there to Dinner with some Gentlemans Servants.</p>
>                    <p>
>
>                       <persName id="t17280717-6-person52">
>                       Robert
>                       Amey
>                    <interp value="Amey" type="surname" inst="t17280717-6-person52"/>
>                          <interp value="Robert" type="given" inst="t17280717-6-person52"/>
>                          <interp value="male" type="gender" inst="t17280717-6-person52"/>
>                       </persName>
>                  depos'd, That he attended the Gentlemens Servants at Dinner, and afterwards they miss'd a Spoon, that Mr. Haydon sent them word of the Spoon brought to him by the Prisoner, and he going to match it by the others, found it to be the same which they had lost, it being of the same Make and Mark: The Fact being thus plainly proved upon him, the Jury found him <rs type="verdictDescription" id="t17280717-6-verdict26">
>                          <interp value="guilty" type="verdictCategory" inst="t17280717-6-verdict26"/>
>                          <interp value="theftunder1s" type="verdictSubcategory" inst="t17280717-6-verdict26"/>
>                    guilty to the value of 10d.
>                 </rs>
>                    </p>
>                    <p>
>                       <rs type="punishmentDescription" id="t17280717-6-punish27">
>                          <interp value="transport" type="punishmentCategory" inst="t17280717-6-punish27"/>
>                          <join targets="t17280717-6-defend50 t17280717-6-punish27" targOrder="Y" result="defendantPunishment"/>
>
>                          <note>[Transportation. See summary.]</note>
>
>                       </rs>
>                    </p>
>                 </div1>, <div1 id="t17280717-7" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-7"/>
>                    <interp value="1728" type="year" inst="t17280717-7"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-7"/>
>                    <interp value="17280717" type="date" inst="t17280717-7"/>
>                    <join targets="t17280717-7-defend53 t17280717-7-off29 t17280717-7-verdict31" targOrder="Y" id="t17280717-7-off29-c49" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-7-defend53">
>                       Robert
>                       Ashby
>                    <interp value="Ashby" type="surname" inst="t17280717-7-defend53"/>
>                          <interp value="Robert" type="given" inst="t17280717-7-defend53"/>
>                          <interp value="male" type="gender" inst="t17280717-7-defend53"/>
>                       </persName>
>                 , of <rs type="occupation" id="t17280717-7-deflabel28">St. Andrew's Holborn</rs>
>                       <join targets="t17280717-7-defend53 t17280717-7-deflabel28" targOrder="Y" result="persNameOccupation"/>, was indicted for <rs type="offenceDescription" id="t17280717-7-off29">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-7-off29"/>
>                          <interp value="grandLarceny" type="offenceSubcategory" inst="t17280717-7-off29"/>
>                    stealing a Gold Watch, value 16 l. a Chain, value 4 l. a Cornelian Seal set in Gold, value 10s. the Property of
>                          <persName type="victimName" id="t17280717-7-victim54">
>                             Nicholas
>                             Roper
>                          <interp value="Roper" type="surname" inst="t17280717-7-victim54"/>
>                             <interp value="Nicholas" type="given" inst="t17280717-7-victim54"/>
>                             <interp value="male" type="gender" inst="t17280717-7-victim54"/>
>                          </persName>
>                         from the Person of
>                          <persName type="victimName" id="t17280717-7-victim55">
>                             Phoebe
>                             Thickpenny
>                          <interp value="Thickpenny" type="surname" inst="t17280717-7-victim55"/>
>                             <interp value="Phoebe" type="given" inst="t17280717-7-victim55"/>
>                             <interp value="female" type="gender" inst="t17280717-7-victim55"/>
>                          </persName>
>
>
>                       </rs>, who depos'd, That her Mistress being at Little Chelsea, sent her to their House in Lad-lane, for the Watch and other Things she had occasion for, that she call'd at the Prisoner's in Castle Yard, Chick-Lane, as she was going towards Chelsea with the Watch and the Bundle, where she drank part of a Pint of Beer, some Tea 2 Quarterns of Brandy, and a Bottle of Cyder; that the Prisoner would go part of the Way home with her, and in <placeName id="t17280717-7-crimeloc30">Leather-Lane</placeName>
>                       <interp value="Leather-Lane" type="placeName" inst="t17280717-7-crimeloc30"/>
>                       <interp value="crimeLocation" type="type" inst="t17280717-7-crimeloc30"/>
>                       <join targets="t17280717-7-off29 t17280717-7-crimeloc30" targOrder="Y" result="offencePlace"/>, he said, faith they would not part Dry lips, and accordingly they went into a publick House and drank a Pint of Twopenny, and two Quarterns of Brandy, that she had the Watch then, and at the Door the Prisoner kiss'd her, and gave her a shilling for a Coach, she having out-staid her Time; that when he kiss'd her, he put one Hand around her Waist, but what he did with the other she could not tell, that she then cross'd the Way to another House, and immediately miss'd the Watch, and she was sure she had pinn'd it so to her Side, that she could not drop it.</p>
>                    <p>
>
>                       <persName id="t17280717-7-person56">
>                       Margaret
>                       Nelson
>                    <interp value="Nelson" type="surname" inst="t17280717-7-person56"/>
>                          <interp value="Margaret" type="given" inst="t17280717-7-person56"/>
>                          <interp value="female" type="gender" inst="t17280717-7-person56"/>
>                       </persName>
>                  depos'd, That the Prisoner and Phoebe Thickpenny, came to her House in Leather-Lane, and he call'd for a private Room, to which the Girl would not consent, that the Girl wanted to go to, &amp;c. and she went with her, when the Girl said, she had her Mistress's Gold Watch, and seeming to look on it, told her it was 7 o' Clock, but this Deponent did not see the Watch, yet she said, she did verily believe she heard it beat, that they soon parted, and in a Quarter of an Hour the maid returned, and said she had lost the Watch.</p>
>                    <p>The Prisoner said in his Defence, That he knew nothing of it, any further than she said it was her Mistress's, that they parted very good Friends, and she having been an old Sweetheart of his, laid her Head upon his Shoulder, and said, She could live and die there, which would have been no little Aggravation to his Crime, had he wrong'd so good Natur'd a Creature: But the Evidence against him being weak, and several appearing to his Character, the Jury <rs type="verdictDescription" id="t17280717-7-verdict31">
>                          <interp value="notGuilty" type="verdictCategory" inst="t17280717-7-verdict31"/>
>                    acquitted
>                 </rs> him. </p>
>                 </div1>, <div1 id="t17280717-8" type="trialAccount">
>                    <interp value="BAILEY" type="collection" inst="t17280717-8"/>
>                    <interp value="1728" type="year" inst="t17280717-8"/>
>                    <interp value="sessionsPapers/17280717" type="uri" inst="t17280717-8"/>
>                    <interp value="17280717" type="date" inst="t17280717-8"/>
>                    <join targets="t17280717-8-defend57 t17280717-8-off33 t17280717-8-verdict35" targOrder="Y" id="t17280717-8-off33-c53" result="criminalCharge"/>
>                    <join targets="t17280717-8-defend58 t17280717-8-off33 t17280717-8-verdict35" targOrder="Y" id="t17280717-8-off33-c54" result="criminalCharge"/>
>
>                    <p>
>
>                       <persName type="defendantName" id="t17280717-8-defend57">
>                       Joseph
>                       Plummer
>                    <interp value="Plummer" type="surname" inst="t17280717-8-defend57"/>
>                          <interp value="Joseph" type="given" inst="t17280717-8-defend57"/>
>                          <interp value="male" type="gender" inst="t17280717-8-defend57"/>
>                       </persName>
>                  , and
>                    <persName type="defendantName" id="t17280717-8-defend58">
>                       Henry
>                       Coleman
>                    <interp value="Coleman" type="surname" inst="t17280717-8-defend58"/>
>                          <interp value="Henry" type="given" inst="t17280717-8-defend58"/>
>                          <interp value="male" type="gender" inst="t17280717-8-defend58"/>
>                       </persName>
>                  , of <placeName id="t17280717-8-defloc32">Edmonton</placeName>
>                       <interp value="Edmonton" type="placeName" inst="t17280717-8-defloc32"/>
>                       <interp value="defendantHome" type="type" inst="t17280717-8-defloc32"/>
>                       <join targets="t17280717-8-defend57 t17280717-8-defloc32" targOrder="Y" result="persNamePlace"/>
>                       <join targets="t17280717-8-defend58 t17280717-8-defloc32" targOrder="Y" result="persNamePlace"/>, were indicted for <rs type="offenceDescription" id="t17280717-8-off33">
>                          <interp value="theft" type="offenceCategory" inst="t17280717-...

In [ ]:
(elem \\ "div1").filter(Node => ((Node \ "@type").text == "trialAccount"))
                 .map(Node => (Node \ "@type", Node \ "@id" ))

  

>     res8: scala.collection.immutable.Seq[(scala.xml.NodeSeq, scala.xml.NodeSeq)] = List((trialAccount,t17280717-1), (trialAccount,t17280717-2), (trialAccount,t17280717-3), (trialAccount,t17280717-4), (trialAccount,t17280717-5), (trialAccount,t17280717-6), (trialAccount,t17280717-7), (trialAccount,t17280717-8), (trialAccount,t17280717-9), (trialAccount,t17280717-10), (trialAccount,t17280717-11), (trialAccount,t17280717-12), (trialAccount,t17280717-13), (trialAccount,t17280717-14), (trialAccount,t17280717-15), (trialAccount,t17280717-16), (trialAccount,t17280717-17), (trialAccount,t17280717-18), (trialAccount,t17280717-19), (trialAccount,t17280717-20), (trialAccount,t17280717-21), (trialAccount,t17280717-22), (trialAccount,t17280717-23), (trialAccount,t17280717-24), (trialAccount,t17280717-25), (trialAccount,t17280717-26), (trialAccount,t17280717-27), (trialAccount,t17280717-28), (trialAccount,t17280717-29), (trialAccount,t17280717-30), (trialAccount,t17280717-31), (trialAccount,t17280717-32), (trialAccount,t17280717-33), (trialAccount,t17280717-34), (trialAccount,t17280717-35), (trialAccount,t17280717-36), (trialAccount,t17280717-37), (trialAccount,t17280717-38), (trialAccount,t17280717-39), (trialAccount,t17280717-40), (trialAccount,t17280717-41), (trialAccount,t17280717-42), (trialAccount,t17280717-43), (trialAccount,t17280717-44), (trialAccount,t17280717-45), (trialAccount,t17280717-46), (trialAccount,t17280717-47), (trialAccount,t17280717-48), (trialAccount,t17280717-49), (trialAccount,t17280717-50), (trialAccount,t17280717-51), (trialAccount,t17280717-52), (trialAccount,t17280717-53), (trialAccount,t17280717-54), (trialAccount,t17280717-55), (trialAccount,t17280717-56), (trialAccount,t17280717-57))

In [ ]:
val trials = (elem \\ "div1").filter(Node => ((Node \ "@type").text == "trialAccount"))
                 .map(Node => (Node \ "@type", Node \ "@id", (Node \\ "rs" \\ "interp").map( n => ((n \\ "@type").text, (n \\ "@value").text ))))

  

>     trials: scala.collection.immutable.Seq[(scala.xml.NodeSeq, scala.xml.NodeSeq, scala.collection.immutable.Seq[(String, String)])] = List((trialAccount,t17280717-1,List((offenceCategory,theft), (offenceSubcategory,theftFromPlace), (surname,Reeves), (given,James), (gender,male), (verdictCategory,notGuilty))), (trialAccount,t17280717-2,List((offenceCategory,theft), (offenceSubcategory,pettyLarceny), (verdictCategory,guilty), (verdictSubcategory,pleadedGuilty), (punishmentCategory,transport))), (trialAccount,t17280717-3,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,guilty), (verdictSubcategory,pleadedGuilty), (punishmentCategory,transport))), (trialAccount,t17280717-4,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,notGuilty))), (trialAccount,t17280717-5,List((offenceCategory,theft), (offenceSubcategory,pocketpicking), (surname,Coxall), (given,John), (gender,male), (verdictCategory,notGuilty))), (trialAccount,t17280717-6,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (surname,Mannio), (given,Abraham), (gender,male), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-7,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (surname,Roper), (given,Nicholas), (gender,male), (surname,Thickpenny), (given,Phoebe), (gender,female), (verdictCategory,notGuilty))), (trialAccount,t17280717-8,List((offenceCategory,theft), (offenceSubcategory,pettyLarceny), (verdictCategory,notGuilty))), (trialAccount,t17280717-9,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,notGuilty))), (trialAccount,t17280717-10,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,corporal), (punishmentSubcategory,whipping))), (trialAccount,t17280717-11,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,notGuilty))), (trialAccount,t17280717-12,List((offenceCategory,theft), (offenceSubcategory,pocketpicking), (surname,Bete), (given,Thomas), (gender,male), (verdictCategory,notGuilty))), (trialAccount,t17280717-13,List((offenceCategory,theft), (offenceSubcategory,pocketpicking), (surname,Polliter), (given,John), (gender,male), (offenceCategory,theft), (offenceSubcategory,theftFromPlace), (verdictCategory,guilty), (verdictSubcategory,theftunder5s), (verdictCategory,notGuilty), (punishmentCategory,transport))), (trialAccount,t17280717-14,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (surname,Freeman), (given,Moses), (gender,male), (verdictCategory,guilty), (verdictSubcategory,theftunder5s), (punishmentCategory,transport))), (trialAccount,t17280717-15,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-16,List((offenceCategory,theft), (offenceSubcategory,theftFromPlace), (surname,Yew), (given,Thomas), (gender,male), (verdictCategory,guilty), (punishmentCategory,death))), (trialAccount,t17280717-17,List((offenceCategory,theft), (offenceSubcategory,animalTheft), (offenceCategory,theft), (offenceSubcategory,animalTheft), (verdictCategory,guilty), (punishmentCategory,death))), (trialAccount,t17280717-18,List((offenceCategory,theft), (offenceSubcategory,animalTheft), (verdictCategory,guilty), (punishmentCategory,death))), (trialAccount,t17280717-19,List((offenceCategory,theft), (offenceSubcategory,shoplifting), (surname,Harrison), (given,Mary), (gender,female), (verdictCategory,guilty), (verdictSubcategory,theftunder5s), (punishmentCategory,transport))), (trialAccount,t17280717-20,List((offenceCategory,theft), (offenceSubcategory,theftFromPlace), (surname,Ward), (given,Samuel), (gender,male), (surname,Harrison), (given,Joseph), (gender,male), (verdictCategory,guilty), (verdictSubcategory,theftunder5s), (punishmentCategory,transport))), (trialAccount,t17280717-21,List((offenceCategory,theft), (offenceSubcategory,shoplifting), (surname,Townshend), (given,George), (gender,male), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-22,List((offenceCategory,theft), (offenceSubcategory,pocketpicking), (surname,Butler), (given,James), (gender,male), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-23,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-24,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,guilty))), (trialAccount,t17280717-25,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-26,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-27,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-28,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (surname,Swift), (given,Grace), (gender,female), (gender,female), (verdictCategory,guilty), (verdictSubcategory,theftunder40s), (punishmentCategory,transport))), (trialAccount,t17280717-29,List((offenceCategory,sexual), (offenceSubcategory,bigamy), (surname,Danbrine), (given,Abraham), (gender,male), (surname,Par), (given,Henry), (gender,male), (verdictCategory,guilty), (punishmentCategory,miscPunish), (punishmentSubcategory,branding))), (trialAccount,t17280717-30,List((offenceCategory,theft), (offenceSubcategory,pettyLarceny), (surname,Smith), (given,Richard), (gender,male), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-31,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,notGuilty))), (trialAccount,t17280717-32,List((offenceCategory,theft), (offenceSubcategory,pocketpicking), (surname,Butterfield), (given,Anthony), (gender,male), (verdictCategory,notGuilty))), (trialAccount,t17280717-33,List((offenceCategory,violentTheft), (offenceSubcategory,highwayRobbery), (surname,Russel), (given,Nathaniel), (gender,male), (verdictCategory,notGuilty))), (trialAccount,t17280717-34,List((offenceCategory,theft), (offenceSubcategory,pocketpicking), (surname,Dalby), (given,Mary), (gender,female), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-35,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,notGuilty))), (trialAccount,t17280717-36,List((offenceCategory,theft), (offenceSubcategory,other), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,corporal), (punishmentSubcategory,whipping))), (trialAccount,t17280717-37,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,notGuilty))), (trialAccount,t17280717-38,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-39,List((offenceCategory,theft), (offenceSubcategory,other), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-40,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,notGuilty))), (trialAccount,t17280717-41,List((offenceCategory,royalOffences), (offenceSubcategory,coiningOffences), (verdictCategory,notGuilty))), (trialAccount,t17280717-42,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,corporal), (punishmentSubcategory,whipping))), (trialAccount,t17280717-43,List((offenceCategory,violentTheft), (offenceSubcategory,robbery), (surname,Herbert), (given,Henry Arthur), (gender,male), (verdictCategory,notGuilty))), (trialAccount,t17280717-44,List((offenceCategory,sexual), (offenceSubcategory,bigamy), (given,Way), (gender,indeterminate), (surname,Towers), (given,John), (gender,male), (verdictCategory,notGuilty))), (trialAccount,t17280717-45,List((offenceCategory,theft), (offenceSubcategory,burglary), (surname,Tomkinson), (given,John), (gender,male), (surname,Tomkinson), (given,John), (gender,male), (verdictCategory,notGuilty))), (trialAccount,t17280717-46,List((offenceCategory,theft), (offenceSubcategory,other), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (verdictCategory,notGuilty), (punishmentCategory,transport))), (trialAccount,t17280717-47,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-48,List((offenceCategory,theft), (offenceSubcategory,other), (surname,Dearing), (given,Samuel), (gender,male), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-49,List((offenceCategory,theft), (offenceSubcategory,burglary), (surname,Tilley), (given,William), (gender,male), (verdictCategory,notGuilty), (verdictCategory,guilty), (verdictSubcategory,theftunder1s), (punishmentCategory,transport))), (trialAccount,t17280717-50,List((offenceCategory,theft), (offenceSubcategory,grandLarceny), (surname,Mills), (given,Jane), (gender,female), (verdictCategory,notGuilty))), (trialAccount,t17280717-51,List((offenceCategory,theft), (offenceSubcategory,pocketpicking), (surname,Evans), (given,Joseph), (gender,male), (verdictCategory,notGuilty))), (trialAccount,t17280717-52,List((offenceCategory,theft), (offenceSubcategory,other), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-53,List((offenceCategory,theft), (offenceSubcategory,other), (verdictCategory,notGuilty), (verdictSubcategory,noProsecutor))), (trialAccount,t17280717-54,List((offenceCategory,theft), (offenceSubcategory,theftFromPlace), (placeName,Belsize House), (type,crimeLocation), (verdictCategory,guilty), (punishmentCategory,imprison))), (trialAccount,t17280717-55,List((offenceCategory,sexual), (offenceSubcategory,bigamy), (surname,Pack), (given,William), (gender,male), (surname,Selby), (given,Francis), (gender,male), (verdictCategory,notGuilty))), (trialAccount,t17280717-56,List((offenceCategory,deception), (offenceSubcategory,perjury), (surname,Matthews), (given,Richard), (gender,male), (verdictCategory,guilty), (verdictSubcategory,pleadedGuilty), (punishmentCategory,corporal), (punishmentSubcategory,pillory))), (trialAccount,t17280717-57,List((offenceCategory,deception), (offenceSubcategory,perjury), (surname,Matthews), (given,Richard), (gender,male), (verdictCategory,guilty), (verdictSubcategory,pleadedGuilty))))

In [ ]:
val wantedFields = Seq("verdictCategory","punishmentCategory","offenceCategory").toSet

  

>     wantedFields: scala.collection.immutable.Set[String] = Set(verdictCategory, punishmentCategory, offenceCategory)

In [ ]:
val trials = (elem \\ "div1").filter(Node => ((Node \ "@type").text == "trialAccount"))
                 .map(Node => ((Node \ "@type").text, (Node \ "@id").text, (Node \\ "rs" \\ "interp")
                                                               .filter(n => wantedFields.contains( (n \\ "@type").text))
                                                               .map( n => ((n \\ "@type").text, (n \\ "@value").text ))))

  

>     trials: scala.collection.immutable.Seq[(String, String, scala.collection.immutable.Seq[(String, String)])] = List((trialAccount,t17280717-1,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-2,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-3,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-4,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-5,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-6,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-7,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-8,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-9,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-10,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,corporal))), (trialAccount,t17280717-11,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-12,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-13,List((offenceCategory,theft), (offenceCategory,theft), (verdictCategory,guilty), (verdictCategory,notGuilty), (punishmentCategory,transport))), (trialAccount,t17280717-14,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-15,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-16,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,death))), (trialAccount,t17280717-17,List((offenceCategory,theft), (offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,death))), (trialAccount,t17280717-18,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,death))), (trialAccount,t17280717-19,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-20,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-21,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-22,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-23,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-24,List((offenceCategory,theft), (verdictCategory,guilty))), (trialAccount,t17280717-25,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-26,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-27,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-28,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-29,List((offenceCategory,sexual), (verdictCategory,guilty), (punishmentCategory,miscPunish))), (trialAccount,t17280717-30,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-31,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-32,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-33,List((offenceCategory,violentTheft), (verdictCategory,notGuilty))), (trialAccount,t17280717-34,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-35,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-36,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,corporal))), (trialAccount,t17280717-37,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-38,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-39,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-40,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-41,List((offenceCategory,royalOffences), (verdictCategory,notGuilty))), (trialAccount,t17280717-42,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,corporal))), (trialAccount,t17280717-43,List((offenceCategory,violentTheft), (verdictCategory,notGuilty))), (trialAccount,t17280717-44,List((offenceCategory,sexual), (verdictCategory,notGuilty))), (trialAccount,t17280717-45,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-46,List((offenceCategory,theft), (verdictCategory,guilty), (verdictCategory,notGuilty), (punishmentCategory,transport))), (trialAccount,t17280717-47,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-48,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-49,List((offenceCategory,theft), (verdictCategory,notGuilty), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-50,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-51,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-52,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,transport))), (trialAccount,t17280717-53,List((offenceCategory,theft), (verdictCategory,notGuilty))), (trialAccount,t17280717-54,List((offenceCategory,theft), (verdictCategory,guilty), (punishmentCategory,imprison))), (trialAccount,t17280717-55,List((offenceCategory,sexual), (verdictCategory,notGuilty))), (trialAccount,t17280717-56,List((offenceCategory,deception), (verdictCategory,guilty), (punishmentCategory,corporal))), (trialAccount,t17280717-57,List((offenceCategory,deception), (verdictCategory,guilty))))

  

Since there can be more than one defendant in a trial, we need to reduce
by key as follows.

In [ ]:
def reduceByKey(collection: Traversable[Tuple2[String, Int]]) = {    
    collection
      .groupBy(_._1)
      .map { case (group: String, traversable) => traversable.reduce{(a,b) => (a._1, a._2 + b._2)} }
  }

  

>     reduceByKey: (collection: Traversable[(String, Int)])scala.collection.immutable.Map[String,Int]

  

Let's process the coarsest data on the trial as json strings.

In [ ]:
val trials = (elem \\ "div1").filter(Node => ((Node \ "@type").text == "trialAccount"))
                 .map(Node => {val trialId = (Node \ "@id").text;
                               val trialInterps = (Node \\ "rs" \\ "interp")
                                                                 .filter(n => wantedFields.contains( (n \\ "@type").text))
                                                                 //.map( n => ((n \\ "@type").text, (n \\ "@value").text ));
                                                                 .map( n => ((n \\ "@value").text , 1 ));
                               val trialCounts = reduceByKey(trialInterps).toMap;
                               //(trialId, trialInterps, trialCounts)
                               scala.util.parsing.json.JSONObject(trialCounts updated ("id", trialId))
                              })

  

>     trials: scala.collection.immutable.Seq[scala.util.parsing.json.JSONObject] = List({"theft" : 1, "notGuilty" : 1, "id" : "t17280717-1"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-2"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-3"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-4"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-5"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-6"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-7"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-8"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-9"}, {"corporal" : 1, "theft" : 1, "guilty" : 1, "id" : "t17280717-10"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-11"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-12"}, {"guilty" : 1, "theft" : 2, "notGuilty" : 1, "transport" : 1, "id" : "t17280717-13"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-14"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-15"}, {"theft" : 1, "guilty" : 1, "death" : 1, "id" : "t17280717-16"}, {"theft" : 2, "guilty" : 1, "death" : 1, "id" : "t17280717-17"}, {"theft" : 1, "guilty" : 1, "death" : 1, "id" : "t17280717-18"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-19"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-20"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-21"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-22"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-23"}, {"theft" : 1, "guilty" : 1, "id" : "t17280717-24"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-25"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-26"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-27"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-28"}, {"miscPunish" : 1, "sexual" : 1, "guilty" : 1, "id" : "t17280717-29"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-30"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-31"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-32"}, {"violentTheft" : 1, "notGuilty" : 1, "id" : "t17280717-33"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-34"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-35"}, {"corporal" : 1, "theft" : 1, "guilty" : 1, "id" : "t17280717-36"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-37"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-38"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-39"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-40"}, {"royalOffences" : 1, "notGuilty" : 1, "id" : "t17280717-41"}, {"corporal" : 1, "theft" : 1, "guilty" : 1, "id" : "t17280717-42"}, {"violentTheft" : 1, "notGuilty" : 1, "id" : "t17280717-43"}, {"notGuilty" : 1, "sexual" : 1, "id" : "t17280717-44"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-45"}, {"guilty" : 1, "theft" : 1, "notGuilty" : 1, "transport" : 1, "id" : "t17280717-46"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-47"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-48"}, {"guilty" : 1, "theft" : 1, "notGuilty" : 1, "transport" : 1, "id" : "t17280717-49"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-50"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-51"}, {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-52"}, {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-53"}, {"theft" : 1, "imprison" : 1, "guilty" : 1, "id" : "t17280717-54"}, {"notGuilty" : 1, "sexual" : 1, "id" : "t17280717-55"}, {"corporal" : 1, "deception" : 1, "guilty" : 1, "id" : "t17280717-56"}, {"deception" : 1, "guilty" : 1, "id" : "t17280717-57"})

In [ ]:
trials.foreach(println)

  

>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-1"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-2"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-3"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-4"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-5"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-6"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-7"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-8"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-9"}
>     {"corporal" : 1, "theft" : 1, "guilty" : 1, "id" : "t17280717-10"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-11"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-12"}
>     {"guilty" : 1, "theft" : 2, "notGuilty" : 1, "transport" : 1, "id" : "t17280717-13"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-14"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-15"}
>     {"theft" : 1, "guilty" : 1, "death" : 1, "id" : "t17280717-16"}
>     {"theft" : 2, "guilty" : 1, "death" : 1, "id" : "t17280717-17"}
>     {"theft" : 1, "guilty" : 1, "death" : 1, "id" : "t17280717-18"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-19"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-20"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-21"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-22"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-23"}
>     {"theft" : 1, "guilty" : 1, "id" : "t17280717-24"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-25"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-26"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-27"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-28"}
>     {"miscPunish" : 1, "sexual" : 1, "guilty" : 1, "id" : "t17280717-29"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-30"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-31"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-32"}
>     {"violentTheft" : 1, "notGuilty" : 1, "id" : "t17280717-33"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-34"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-35"}
>     {"corporal" : 1, "theft" : 1, "guilty" : 1, "id" : "t17280717-36"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-37"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-38"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-39"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-40"}
>     {"royalOffences" : 1, "notGuilty" : 1, "id" : "t17280717-41"}
>     {"corporal" : 1, "theft" : 1, "guilty" : 1, "id" : "t17280717-42"}
>     {"violentTheft" : 1, "notGuilty" : 1, "id" : "t17280717-43"}
>     {"notGuilty" : 1, "sexual" : 1, "id" : "t17280717-44"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-45"}
>     {"guilty" : 1, "theft" : 1, "notGuilty" : 1, "transport" : 1, "id" : "t17280717-46"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-47"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-48"}
>     {"guilty" : 1, "theft" : 1, "notGuilty" : 1, "transport" : 1, "id" : "t17280717-49"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-50"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-51"}
>     {"theft" : 1, "transport" : 1, "guilty" : 1, "id" : "t17280717-52"}
>     {"theft" : 1, "notGuilty" : 1, "id" : "t17280717-53"}
>     {"theft" : 1, "imprison" : 1, "guilty" : 1, "id" : "t17280717-54"}
>     {"notGuilty" : 1, "sexual" : 1, "id" : "t17280717-55"}
>     {"corporal" : 1, "deception" : 1, "guilty" : 1, "id" : "t17280717-56"}
>     {"deception" : 1, "guilty" : 1, "id" : "t17280717-57"}

  

Step 2: Extract, Transform and Load XML files to get DataFrame of counts
------------------------------------------------------------------------

We have played enough (see **Step 1: Exploring data first: xml parsing
in scala** above first) to understand what to do now with our xml data
in order to get it converted to counts of crimes, verdicts and
punishments.

Let's parse the xml files and turn into Dataframe in one block.

In [ ]:
val rawWTF = sc.wholeTextFiles("dbfs:/datasets/obo/tei/sessionsPapers/*.xml") // has all data on crimes and punishments
val raw = rawWTF.map( x => x._2 )
val trials = raw.flatMap( x => { 
                       val elem = scala.xml.XML.loadString(x);
                       val outJson = (elem \\ "div1").filter(Node => ((Node \ "@type").text == "trialAccount"))
                           .map(Node => {val trialId = (Node \ "@id").text;
                               val trialInterps = (Node \\ "rs" \\ "interp")
                                                                 .filter(n => wantedFields.contains( (n \\ "@type").text))
                                                                 //.map( n => ((n \\ "@type").text, (n \\ "@value").text ));
                                                                 .map( n => ((n \\ "@value").text , 1 ));
                               val trialCounts = reduceByKey(trialInterps).toMap;
                               //(trialId, trialInterps, trialCounts)
                               scala.util.parsing.json.JSONObject(trialCounts updated ("id", trialId)).toString()
                              })
  outJson
})

  

>     rawWTF: org.apache.spark.rdd.RDD[(String, String)] = dbfs:/datasets/obo/tei/sessionsPapers/*.xml MapPartitionsRDD[197] at wholeTextFiles at command-753740454082178:1
>     raw: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[198] at map at command-753740454082178:2
>     trials: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[199] at flatMap at command-753740454082178:3

In [ ]:
dbutils.fs.rm("dbfs:/datasets/obo/processed/trialCounts",recurse=true) // let's remove the files from the previous analysis
trials.saveAsTextFile("dbfs:/datasets/obo/processed/trialCounts") // now let's save the trial counts - aboout 220 seconds to pars all data and get counts

In [ ]:
display(dbutils.fs.ls("dbfs:/datasets/obo/processed/trialCounts"))

  

[TABLE]

In [ ]:
val trialCountsDF = sqlContext.read.json("dbfs:/datasets/obo/processed/trialCounts")

  

>     trialCountsDF: org.apache.spark.sql.DataFrame = [breakingPeace: bigint, corporal: bigint ... 18 more fields]

In [ ]:
trialCountsDF.printSchema

  

>     root
>      |-- breakingPeace: long (nullable = true)
>      |-- corporal: long (nullable = true)
>      |-- damage: long (nullable = true)
>      |-- death: long (nullable = true)
>      |-- deception: long (nullable = true)
>      |-- guilty: long (nullable = true)
>      |-- id: string (nullable = true)
>      |-- imprison: long (nullable = true)
>      |-- kill: long (nullable = true)
>      |-- miscPunish: long (nullable = true)
>      |-- miscVerdict: long (nullable = true)
>      |-- miscellaneous: long (nullable = true)
>      |-- noPunish: long (nullable = true)
>      |-- notGuilty: long (nullable = true)
>      |-- royalOffences: long (nullable = true)
>      |-- sexual: long (nullable = true)
>      |-- specialVerdict: long (nullable = true)
>      |-- theft: long (nullable = true)
>      |-- transport: long (nullable = true)
>      |-- violentTheft: long (nullable = true)

In [ ]:
trialCountsDF.count // total number of trials = 197751

  

>     res14: Long = 197751

In [ ]:
display(trialCountsDF)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

In [ ]:
val trDF = trialCountsDF.na.fill(0) // filling nulls with 0

  

>     trDF: org.apache.spark.sql.DataFrame = [breakingPeace: bigint, corporal: bigint ... 18 more fields]

In [ ]:
display(trDF)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

  

This is already available as the following csv file:

-   <http://lamastex.org/datasets/public/OldBailey/oboOffencePunnishmentCountsFrom-sds-2-2-ApacheSparkScalaProcessingOfOBOXMLDoneByRaazOn20180405.csv>

Please cite this URL if you use this data or the Apache licensed codes
in the databricks notebook above for your own non-commerical analysis:

-   <http://lamastex.org/datasets/public/OldBailey/>

Raazesh Sainudiin generated this header **Old bailey Processing in
Apache Spark** on Thu Apr 5 18:22:43 CEST 2018 in Uppsala, Sweden.

Step 0: Dowloading and Loading Data (The Full Dataset)
------------------------------------------------------

First we will be downloading data from
<http://lamastex.org/datasets/public/OldBailey/index.html>.

The steps below need to be done once for a give shard!

You can download the tiny dataset
`obo-tiny/OB-tiny_tei_7-2_CC-BY-NC.zip` **to save time and space in db
CE**

**Optional TODOs:**

-   one could just read the zip files directly (see week 10 on Beijing
    taxi trajectories example from the scalable-data-science course in
    2016 or read 'importing zip files' in the Guide).
-   one could just download from s3 directly

In [ ]:
# if you want to download the tiny dataset
wget https://raw.githubusercontent.com/raazesh-sainudiin/scalable-data-science/master/datasets/obo-tiny/OB-tiny_tei_7-2_CC-BY-NC.zip

  

>     --2019-05-10 05:24:57--  https://raw.githubusercontent.com/raazesh-sainudiin/scalable-data-science/master/datasets/obo-tiny/OB-tiny_tei_7-2_CC-BY-NC.zip
>     Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.40.133
>     Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.40.133|:443... connected.
>     HTTP request sent, awaiting response... 200 OK
>     Length: 1209487 (1.2M) [application/zip]
>     Saving to: ‘OB-tiny_tei_7-2_CC-BY-NC.zip’
>
>          0K .......... .......... .......... .......... ..........  4% 19.1M 0s
>         50K .......... .......... .......... .......... ..........  8% 25.6M 0s
>        100K .......... .......... .......... .......... .......... 12% 29.0M 0s
>        150K .......... .......... .......... .......... .......... 16% 23.5M 0s
>        200K .......... .......... .......... .......... .......... 21% 33.8M 0s
>        250K .......... .......... .......... .......... .......... 25% 42.2M 0s
>        300K .......... .......... .......... .......... .......... 29% 39.9M 0s
>        350K .......... .......... .......... .......... .......... 33% 27.8M 0s
>        400K .......... .......... .......... .......... .......... 38% 37.9M 0s
>        450K .......... .......... .......... .......... .......... 42% 34.6M 0s
>        500K .......... .......... .......... .......... .......... 46% 40.6M 0s
>        550K .......... .......... .......... .......... .......... 50% 71.0M 0s
>        600K .......... .......... .......... .......... .......... 55% 48.7M 0s
>        650K .......... .......... .......... .......... .......... 59% 41.5M 0s
>        700K .......... .......... .......... .......... .......... 63% 65.3M 0s
>        750K .......... .......... .......... .......... .......... 67% 42.7M 0s
>        800K .......... .......... .......... .......... .......... 71% 44.9M 0s
>        850K .......... .......... .......... .......... .......... 76% 84.9M 0s
>        900K .......... .......... .......... .......... .......... 80% 42.5M 0s
>        950K .......... .......... .......... .......... .......... 84% 82.1M 0s
>       1000K .......... .......... .......... .......... .......... 88% 63.5M 0s
>       1050K .......... .......... .......... .......... .......... 93% 79.9M 0s
>       1100K .......... .......... .......... .......... .......... 97% 60.8M 0s
>       1150K .......... .......... .......... .                    100% 78.7M=0.03s
>
>     2019-05-10 05:24:58 (40.5 MB/s) - ‘OB-tiny_tei_7-2_CC-BY-NC.zip’ saved [1209487/1209487]

In [ ]:
# this is the full dataset - necessary for a project on this dataset
wget http://lamastex.org/datasets/public/OldBailey/OB_tei_7-2_CC-BY-NC.zip

  

>     --2019-05-10 05:25:11--  http://lamastex.org/datasets/public/OldBailey/OB_tei_7-2_CC-BY-NC.zip
>     Resolving lamastex.org (lamastex.org)... 166.62.28.100
>     Connecting to lamastex.org (lamastex.org)|166.62.28.100|:80... connected.
>     HTTP request sent, awaiting response... 200 OK
>     Length: 324723900 (310M) [application/zip]
>     Saving to: ‘OB_tei_7-2_CC-BY-NC.zip’
>
>          0K .......... .......... .......... .......... ..........  0%  149K 35m32s
>         50K .......... .......... .......... .......... ..........  0%  297K 26m39s
>        100K .......... .......... .......... .......... ..........  0%  298K 23m40s
>        150K .......... .......... .......... .......... ..........  0% 31.4M 17m47s
>        200K .......... .......... .......... .......... ..........  0%  300K 17m45s
>        250K .......... .......... .......... .......... ..........  0% 47.4M 14m48s
>        300K .......... .......... .......... .......... ..........  0% 53.4M 12m42s
>        350K .......... .......... .......... .......... ..........  0% 67.0M 11m7s
>        400K .......... .......... .......... .......... ..........  0%  302K 11m50s
>        450K .......... .......... .......... .......... ..........  0% 26.2M 10m40s
>        500K .......... .......... .......... .......... ..........  0% 28.0M 9m43s
>        550K .......... .......... .......... .......... ..........  0% 45.9M 8m55s
>        600K .......... .......... .......... .......... ..........  0%  268M 8m13s
>        650K .......... .......... .......... .......... ..........  0%  307K 8m52s
>        700K .......... .......... .......... .......... ..........  0% 45.5M 8m17s
>        750K .......... .......... .......... .......... ..........  0% 62.8M 7m46s
>        800K .......... .......... .......... .......... ..........  0% 27.8M 7m19s
>        850K .......... .......... .......... .......... ..........  0% 41.8M 6m55s
>        900K .......... .......... .......... .......... ..........  0% 45.3M 6m33s
>        950K .......... .......... .......... .......... ..........  0% 52.0M 6m14s
>       1000K .......... .......... .......... .......... ..........  0% 74.1M 5m56s
>       1050K .......... .......... .......... .......... ..........  0% 45.2M 5m40s
>       1100K .......... .......... .......... .......... ..........  0%  307K 6m10s
>       1150K .......... .......... .......... .......... ..........  0% 79.6M 5m55s
>       1200K .......... .......... .......... .......... ..........  0%  200M 5m41s
>       1250K .......... .......... .......... .......... ..........  0%  125M 5m28s
>       1300K .......... .......... .......... .......... ..........  0%  231M 5m16s
>       1350K .......... .......... .......... .......... ..........  0% 4.52M 5m7s
>       1400K .......... .......... .......... .......... ..........  0% 27.5M 4m56s
>       1450K .......... .......... .......... .......... ..........  0% 41.5M 4m47s
>       1500K .......... .......... .......... .......... ..........  0%  223M 4m38s
>       1550K .......... .......... .......... .......... ..........  0%  300M 4m29s
>       1600K .......... .......... .......... .......... ..........  0%  254M 4m21s
>       1650K .......... .......... .......... .......... ..........  0%  338K 4m40s
>       1700K .......... .......... .......... .......... ..........  0% 50.9M 4m33s
>       1750K .......... .......... .......... .......... ..........  0% 46.8M 4m25s
>       1800K .......... .......... .......... .......... ..........  0% 55.2M 4m18s
>       1850K .......... .......... .......... .......... ..........  0% 56.2M 4m11s
>       1900K .......... .......... .......... .......... ..........  0% 53.8M 4m5s
>       1950K .......... .......... .......... .......... ..........  0% 54.7M 3m59s
>       2000K .......... .......... .......... .......... ..........  0% 57.0M 3m53s
>       2050K .......... .......... .......... .......... ..........  0% 46.9M 3m48s
>       2100K .......... .......... .......... .......... ..........  0% 6.17M 3m44s
>       2150K .......... .......... .......... .......... ..........  0% 50.7M 3m39s
>       2200K .......... .......... .......... .......... ..........  0% 45.4M 3m34s
>       2250K .......... .......... .......... .......... ..........  0% 48.9M 3m29s
>       2300K .......... .......... .......... .......... ..........  0% 37.3M 3m25s
>       2350K .......... .......... .......... .......... ..........  0% 62.7M 3m21s
>       2400K .......... .......... .......... .......... ..........  0% 69.0M 3m17s
>       2450K .......... .......... .......... .......... ..........  0% 53.1M 3m13s
>       2500K .......... .......... .......... .......... ..........  0%  342K 3m27s
>       2550K .......... .......... .......... .......... ..........  0% 45.3M 3m23s
>       2600K .......... .......... .......... .......... ..........  0% 29.2M 3m20s
>       2650K .......... .......... .......... .......... ..........  0% 30.3M 3m16s
>       2700K .......... .......... .......... .......... ..........  0% 70.2M 3m13s
>       2750K .......... .......... .......... .......... ..........  0% 44.9M 3m9s
>       2800K .......... .......... .......... .......... ..........  0%  147M 3m6s
>       2850K .......... .......... .......... .......... ..........  0% 23.4M 3m3s
>       2900K .......... .......... .......... .......... ..........  0% 51.7M 3m0s
>       2950K .......... .......... .......... .......... ..........  0% 41.1M 2m57s
>       3000K .......... .......... .......... .......... ..........  0% 73.1M 2m54s
>       3050K .......... .......... .......... .......... ..........  0% 34.4M 2m51s
>       3100K .......... .......... .......... .......... ..........  0% 29.0M 2m49s
>       3150K .......... .......... .......... .......... ..........  1% 26.5M 2m46s
>       3200K .......... .......... .......... .......... ..........  1% 31.4M 2m44s
>       3250K .......... .......... .......... .......... ..........  1% 58.3M 2m42s
>       3300K .......... .......... .......... .......... ..........  1% 33.3M 2m39s
>       3350K .......... .......... .......... .......... ..........  1% 32.6M 2m37s
>       3400K .......... .......... .......... .......... ..........  1% 33.4M 2m35s
>       3450K .......... .......... .......... .......... ..........  1% 43.6M 2m33s
>       3500K .......... .......... .......... .......... ..........  1% 55.1M 2m31s
>       3550K .......... .......... .......... .......... ..........  1%  261M 2m29s
>       3600K .......... .......... .......... .......... ..........  1%  227M 2m26s
>       3650K .......... .......... .......... .......... ..........  1%  180M 2m24s
>       3700K .......... .......... .......... .......... ..........  1%  350K 2m34s
>       3750K .......... .......... .......... .......... ..........  1% 37.1M 2m33s
>       3800K .......... .......... .......... .......... ..........  1% 51.3M 2m31s
>       3850K .......... .......... .......... .......... ..........  1% 78.8M 2m29s
>       3900K .......... .......... .......... .......... ..........  1% 31.1M 2m27s
>       3950K .......... .......... .......... .......... ..........  1% 52.8M 2m25s
>       4000K .......... .......... .......... .......... ..........  1% 49.4M 2m23s
>       4050K .......... .......... .......... .......... ..........  1% 39.2M 2m22s
>       4100K .......... .......... .......... .......... ..........  1% 32.9M 2m20s
>       4150K .......... .......... .......... .......... ..........  1% 23.7M 2m19s
>       4200K .......... .......... .......... .......... ..........  1% 38.7M 2m17s
>       4250K .......... .......... .......... .......... ..........  1% 57.5M 2m15s
>       4300K .......... .......... .......... .......... ..........  1% 64.4M 2m14s
>       4350K .......... .......... .......... .......... ..........  1% 43.5M 2m12s
>       4400K .......... .......... .......... .......... ..........  1% 38.8M 2m11s
>       4450K .......... .......... .......... .......... ..........  1% 32.4M 2m10s
>       4500K .......... .......... .......... .......... ..........  1% 62.9M 2m8s
>       4550K .......... .......... .......... .......... ..........  1% 46.6M 2m7s
>       4600K .......... .......... .......... .......... ..........  1% 38.9M 2m6s
>       4650K .......... .......... .......... .......... ..........  1% 51.5M 2m4s
>       4700K .......... .......... .......... .......... ..........  1% 38.6M 2m3s
>       4750K .......... .......... .......... .......... ..........  1% 42.1M 2m2s
>       4800K .......... .......... .......... .......... ..........  1% 74.8M 2m1s
>       4850K .......... .......... .......... .......... ..........  1% 52.5M 1m59s
>       4900K .......... .......... .......... .......... ..........  1% 42.4M 1m58s
>       4950K .......... .......... .......... .......... ..........  1% 52.1M 1m57s
>       5000K .......... .......... .......... .......... ..........  1% 58.0M 1m56s
>       5050K .......... .......... .......... .......... ..........  1% 43.9M 1m55s
>       5100K .......... .......... .......... .......... ..........  1% 63.0M 1m54s
>       5150K .......... .......... .......... .......... ..........  1% 46.1M 1m53s
>       5200K .......... .......... .......... .......... ..........  1% 48.0M 1m52s
>       5250K .......... .......... .......... .......... ..........  1%  371K 1m59s
>       5300K .......... .......... .......... .......... ..........  1% 45.3M 1m58s
>       5350K .......... .......... .......... .......... ..........  1% 45.3M 1m56s
>       5400K .......... .......... .......... .......... ..........  1% 49.0M 1m55s
>       5450K .......... .......... .......... .......... ..........  1% 32.4M 1m54s
>       5500K .......... .......... .......... .......... ..........  1% 38.8M 1m54s
>       5550K .......... .......... .......... .......... ..........  1% 41.7M 1m53s
>       5600K .......... .......... .......... .......... ..........  1% 46.5M 1m52s
>       5650K .......... .......... .......... .......... ..........  1% 39.9M 1m51s
>       5700K .......... .......... .......... .......... ..........  1% 29.0M 1m50s
>       5750K .......... .......... .......... .......... ..........  1% 41.1M 1m49s
>       5800K .......... .......... .......... .......... ..........  1% 44.5M 1m48s
>       5850K .......... .......... .......... .......... ..........  1% 51.3M 1m47s
>       5900K .......... .......... .......... .......... ..........  1% 50.3M 1m46s
>       5950K .......... .......... .......... .......... ..........  1% 37.4M 1m45s
>       6000K .......... .......... .......... .......... ..........  1% 44.4M 1m45s
>       6050K .......... .......... .......... .......... ..........  1% 43.0M 1m44s
>       6100K .......... .......... .......... .......... ..........  1% 39.5M 1m43s
>       6150K .......... .......... .......... .......... ..........  1% 48.6M 1m42s
>       6200K .......... .......... .......... .......... ..........  1% 48.7M 1m41s
>       6250K .......... .......... .......... .......... ..........  1% 35.7M 1m41s
>       6300K .......... .......... .......... .......... ..........  2% 50.0M 1m40s
>       6350K .......... .......... .......... .......... ..........  2% 24.8M 99s
>       6400K .......... .......... .......... .......... ..........  2% 55.7M 98s
>       6450K .......... .......... .......... .......... ..........  2% 42.8M 98s
>       6500K .......... .......... .......... .......... ..........  2% 2.37M 98s
>       6550K .......... .......... .......... .......... ..........  2% 44.0M 97s
>       6600K .......... .......... .......... .......... ..........  2% 35.1M 97s
>       6650K .......... .......... .......... .......... ..........  2% 59.5M 96s
>       6700K .......... .......... .......... .......... ..........  2% 81.8M 95s
>       6750K .......... .......... .......... .......... ..........  2% 50.6M 94s
>       6800K .......... .......... .......... .......... ..........  2%  439K 99s
>       6850K .......... .......... .......... .......... ..........  2% 48.7M 98s
>       6900K .......... .......... .......... .......... ..........  2% 43.1M 98s
>       6950K .......... .......... .......... .......... ..........  2% 44.7M 97s
>       7000K .......... .......... .......... .......... ..........  2% 43.4M 96s
>       7050K .......... .......... .......... .......... ..........  2% 32.6M 96s
>       7100K .......... .......... .......... .......... ..........  2% 40.4M 95s
>       7150K .......... .......... .......... .......... ..........  2% 42.8M 94s
>       7200K .......... .......... .......... .......... ..........  2% 49.5M 94s
>       7250K .......... .......... .......... .......... ..........  2% 28.9M 93s
>       7300K .......... .......... .......... .......... ..........  2% 37.7M 93s
>       7350K .......... .......... .......... .......... ..........  2% 46.5M 92s
>       7400K .......... .......... .......... .......... ..........  2% 48.2M 91s
>       7450K .......... .......... .......... .......... ..........  2% 49.7M 91s
>       7500K .......... .......... .......... .......... ..........  2% 26.7M 90s
>       7550K .......... .......... .......... .......... ..........  2% 64.0M 90s
>       7600K .......... .......... .......... .......... ..........  2% 51.9M 89s
>       7650K .......... .......... .......... .......... ..........  2% 42.4M 89s
>       7700K .......... .......... .......... .......... ..........  2% 46.6M 88s
>       7750K .......... .......... .......... .......... ..........  2% 21.0M 88s
>       7800K .......... .......... .......... .......... ..........  2% 27.7M 87s
>       7850K .......... .......... .......... .......... ..........  2%  160M 86s
>       7900K .......... .......... .......... .......... ..........  2% 17.2M 86s
>       7950K .......... .......... .......... .......... ..........  2% 86.2M 85s
>       8000K .......... .......... .......... .......... ..........  2%  251M 85s
>       8050K .......... .......... .......... .......... ..........  2% 2.42M 85s
>       8100K .......... .......... .......... .......... ..........  2% 41.5M 85s
>       8150K .......... .......... .......... .......... ..........  2% 8.64M 84s
>       8200K .......... .......... .......... .......... ..........  2% 46.6M 84s
>       8250K .......... .......... .......... .......... ..........  2% 49.1M 83s
>       8300K .......... .......... .......... .......... ..........  2% 48.3M 83s
>       8350K .......... .......... .......... .......... ..........  2%  460K 86s
>       8400K .......... .......... .......... .......... ..........  2% 39.9M 86s
>       8450K .......... .......... .......... .......... ..........  2% 44.6M 85s
>       8500K .......... .......... .......... .......... ..........  2% 36.8M 85s
>       8550K .......... .......... .......... .......... ..........  2% 57.2M 85s
>       8600K .......... .......... .......... .......... ..........  2% 31.5M 84s
>       8650K .......... .......... .......... .......... ..........  2% 24.5M 84s
>       8700K .......... .......... .......... .......... ..........  2% 38.6M 83s
>       8750K .......... .......... .......... .......... ..........  2% 76.6M 83s
>       8800K .......... .......... .......... .......... ..........  2% 48.3M 82s
>       8850K .......... .......... .......... .......... ..........  2% 35.6M 82s
>       8900K .......... .......... .......... .......... ..........  2% 32.8M 81s
>       8950K .......... .......... .......... .......... ..........  2% 79.3M 81s
>       9000K .......... .......... .......... .......... ..........  2% 43.8M 81s
>       9050K .......... .......... .......... .......... ..........  2% 34.4M 80s
>       9100K .......... .......... .......... .......... ..........  2% 33.6M 80s
>       9150K .......... .......... .......... .......... ..........  2% 56.9M 79s
>       9200K .......... .......... .......... .......... ..........  2% 52.1M 79s
>       9250K .......... .......... .......... .......... ..........  2% 32.5M 79s
>       9300K .......... .......... .......... .......... ..........  2% 69.0M 78s
>       9350K .......... .......... .......... .......... ..........  2% 46.8M 78s
>       9400K .......... .......... .......... .......... ..........  2% 30.6M 77s
>       9450K .......... .......... .......... .......... ..........  2% 49.5M 77s
>       9500K .......... .......... .......... .......... ..........  3% 32.0M 77s
>       9550K .......... .......... .......... .......... ..........  3% 40.6M 76s
>       9600K .......... .......... .......... .......... ..........  3% 1.99M 77s
>       9650K .......... .......... .......... .......... ..........  3% 66.7M 76s
>       9700K .......... .......... .......... .......... ..........  3% 18.8M 76s
>       9750K .......... .......... .......... .......... ..........  3% 38.7M 76s
>       9800K .......... .......... .......... .......... ..........  3% 88.4M 75s
>       9850K .......... .......... .......... .......... ..........  3% 40.3M 75s
>       9900K .......... .......... .......... .......... ..........  3% 1.48M 75s
>       9950K .......... .......... .......... .......... ..........  3%  652K 77s
>      10000K .......... .......... .......... .......... ..........  3% 48.0M 77s
>      10050K .......... .......... .......... .......... ..........  3% 37.5M 77s
>      10100K .......... .......... .......... .......... ..........  3% 45.7M 76s
>      10150K .......... .......... .......... .......... ..........  3% 45.6M 76s
>      10200K .......... .......... .......... .......... ..........  3% 40.4M 76s
>      10250K .......... .......... .......... .......... ..........  3% 33.5M 75s
>      10300K .......... .......... .......... .......... ..........  3% 58.3M 75s
>      10350K .......... .......... .......... .......... ..........  3% 39.0M 75s
>      10400K .......... .......... .......... .......... ..........  3% 33.8M 74s
>      10450K .......... .......... .......... .......... ..........  3% 30.1M 74s
>      10500K .......... .......... .......... .......... ..........  3% 42.9M 74s
>      10550K .......... .......... .......... .......... ..........  3% 69.1M 73s
>      10600K .......... .......... .......... .......... ..........  3% 35.9M 73s
>      10650K .......... .......... .......... .......... ..........  3% 38.0M 73s
>      10700K .......... .......... .......... .......... ..........  3% 47.5M 72s
>      10750K .......... .......... .......... .......... ..........  3% 45.7M 72s
>      10800K .......... .......... .......... .......... ..........  3% 34.8M 72s
>      10850K .......... .......... .......... .......... ..........  3% 54.0M 71s
>      10900K .......... .......... .......... .......... ..........  3% 50.1M 71s
>      10950K .......... .......... .......... .......... ..........  3% 49.2M 71s
>      11000K .......... .......... .......... .......... ..........  3% 10.5M 71s
>      11050K .......... .......... .......... .......... ..........  3% 48.7M 70s
>      11100K .......... .......... .......... .......... ..........  3% 38.7M 70s
>      11150K .......... .......... .......... .......... ..........  3% 2.26M 70s
>      11200K .......... .......... .......... .......... ..........  3% 31.3M 70s
>      11250K .......... .......... .......... .......... ..........  3% 14.9M 70s
>      11300K .......... .......... .......... .......... ..........  3% 16.1M 70s
>      11350K .......... .......... .......... .......... ..........  3%  231M 69s
>      11400K .......... .......... .......... .......... ..........  3%  119M 69s
>      11450K .......... .......... .......... .......... ..........  3% 1.57M 69s
>      11500K .......... .......... .......... .......... ..........  3%  653K 71s
>      11550K .......... .......... .......... .......... ..........  3% 44.4M 71s
>      11600K .......... .......... .......... .......... ..........  3% 41.7M 71s
>      11650K .......... .......... .......... .......... ..........  3% 50.4M 70s
>      11700K .......... .......... .......... .......... ..........  3% 40.8M 70s
>      11750K .......... .......... .......... .......... ..........  3% 34.8M 70s
>      11800K .......... .......... .......... .......... ..........  3% 46.2M 69s
>      11850K .......... .......... .......... .......... ..........  3% 44.0M 69s
>      11900K .......... .......... .......... .......... ..........  3% 38.0M 69s
>      11950K .......... .......... .......... .......... ..........  3% 34.0M 69s
>      12000K .......... .......... .......... .......... ..........  3% 31.7M 68s
>      12050K .......... .......... .......... .......... ..........  3% 27.5M 68s
>      12100K .......... .......... .......... .......... ..........  3%  138M 68s
>      12150K .......... .......... .......... .......... ..........  3% 48.2M 68s
>      12200K .......... .......... .......... .......... ..........  3% 35.2M 67s
>      12250K .......... .......... .......... .......... ..........  3% 53.7M 67s
>      12300K .......... .......... .......... .......... ..........  3% 45.9M 67s
>      12350K .......... .......... .......... .......... ..........  3% 40.2M 67s
>      12400K .......... .......... .......... .......... ..........  3% 37.4M 66s
>      12450K .......... .......... .......... .......... ..........  3% 10.3M 66s
>      12500K .......... .......... .......... .......... ..........  3% 61.7M 66s
>      12550K .......... .......... .......... .......... ..........  3% 46.1M 66s
>      12600K .......... .......... .......... .......... ..........  3% 47.1M 65s
>      12650K .......... .......... .......... .......... ..........  4% 18.3M 65s
>      12700K .......... .......... .......... .......... ..........  4% 3.00M 65s
>      12750K .......... .......... .......... .......... ..........  4% 9.88M 65s
>      12800K .......... .......... .......... .......... ..........  4% 19.6M 65s
>      12850K .......... .......... .......... .......... ..........  4% 40.5M 65s
>      12900K .......... .......... .......... .......... ..........  4% 47.0M 64s
>      12950K .......... .......... .......... .......... ..........  4% 20.8M 64s
>      13000K .......... .......... .......... .......... ..........  4% 1.55M 65s
>      13050K .......... .......... .......... .......... ..........  4%  652K 66s
>      13100K .......... .......... .......... .......... ..........  4% 56.9M 66s
>      13150K .......... .......... .......... .......... ..........  4% 37.0M 66s
>      13200K .......... .......... .......... .......... ..........  4% 55.5M 66s
>      13250K .......... .......... .......... .......... ..........  4% 29.4M 65s
>      13300K .......... .......... .......... .......... ..........  4% 53.4M 65s
>      13350K .......... .......... .......... .......... ..........  4% 46.7M 65s
>      13400K .......... .......... .......... .......... ..........  4% 25.6M 65s
>      13450K .......... .......... .......... .......... ..........  4% 27.2M 64s
>      13500K .......... .......... .......... .......... ..........  4% 67.6M 64s
>      13550K .......... .......... .......... .......... ..........  4% 37.6M 64s
>      13600K .......... .......... .......... .......... ..........  4% 13.8M 64s
>      13650K .......... .......... .......... .......... ..........  4% 63.1M 64s
>      13700K .......... .......... .......... .......... ..........  4% 47.6M 63s
>      13750K .......... .......... .......... .......... ..........  4% 64.0M 63s
>      13800K .......... .......... .......... .......... ..........  4%  193M 63s
>      13850K .......... .......... .......... .......... ..........  4% 10.2M 63s
>      13900K .......... .......... .......... .......... ..........  4% 8.77M 63s
>      13950K .......... .......... .......... .......... ..........  4% 24.0M 63s
>      14000K .......... .......... .......... .......... ..........  4%  226M 62s
>      14050K .......... .......... .......... .......... ..........  4% 66.6M 62s
>      14100K .......... .......... .......... .......... ..........  4%  261M 62s
>      14150K .......... .......... .......... .......... ..........  4%  275M 62s
>      14200K .......... .......... .......... .......... ..........  4% 70.5M 61s
>      14250K .......... .......... .......... .......... ..........  4%  229M 61s
>      14300K .......... .......... .......... .......... ..........  4% 2.56M 61s
>      14350K .......... .......... .......... .......... ..........  4% 19.5M 61s
>      14400K .......... .......... .......... .......... ..........  4% 39.9M 61s
>      14450K .......... .......... .......... .......... ..........  4% 36.6M 61s
>      14500K .......... .......... .......... .......... ..........  4% 30.8M 61s
>      14550K .......... .......... .......... .......... ..........  4% 1.52M 61s
>      14600K .......... .......... .......... .......... ..........  4%  655K 62s
>      14650K .......... .......... .......... .......... ..........  4% 25.9M 62s
>      14700K .......... .......... .......... .......... ..........  4%  123M 62s
>      14750K .......... .......... .......... .......... ..........  4% 48.1M 62s
>      14800K .......... .......... .......... .......... ..........  4% 25.4M 62s
>      14850K .......... .......... .......... .......... ..........  4% 68.0M 61s
>      14900K .......... .......... .......... .......... ..........  4% 41.0M 61s
>      14950K .......... .......... .......... .......... ..........  4% 46.0M 61s
>      15000K .......... .......... .......... .......... ..........  4% 26.8M 61s
>      15050K .......... .......... .......... .......... ..........  4% 71.2M 61s
>      15100K .......... .......... .......... .......... ..........  4% 26.2M 61s
>      15150K .......... .......... .......... .......... ..........  4% 32.8M 60s
>      15200K .......... .......... .......... .......... ..........  4% 75.0M 60s
>      15250K .......... .......... .......... .......... ..........  4% 41.8M 60s
>      15300K .......... .......... .......... .......... ..........  4% 1.68M 60s
>      15350K .......... .......... .......... .......... ..........  4% 46.4M 60s
>      15400K .......... .......... .......... .......... ..........  4% 79.1M 60s
>      15450K .......... .......... .......... .......... ..........  4% 27.7M 60s
>      15500K .......... .......... .......... .......... ..........  4%  270M 60s
>      15550K .......... .......... .......... .......... ..........  4% 52.5M 59s
>      15600K .......... .......... .......... .......... ..........  4% 35.9M 59s
>      15650K .......... .......... .......... .......... ..........  4% 70.1M 59s
>      15700K .......... .......... .......... .......... ..........  4% 40.6M 59s
>      15750K .......... .......... .......... .......... ..........  4% 37.6M 59s
>      15800K .......... .......... .......... .......... ..........  4% 64.5M 59s
>
>     *** WARNING: skipped 437490 bytes of output ***
>
>     300800K .......... .......... .......... .......... .......... 94% 35.4M 2s
>     300850K .......... .......... .......... .......... .......... 94% 39.3M 2s
>     300900K .......... .......... .......... .......... .......... 94% 23.2M 2s
>     300950K .......... .......... .......... .......... .......... 94% 51.1M 2s
>     301000K .......... .......... .......... .......... .......... 94%  287M 2s
>     301050K .......... .......... .......... .......... .......... 94% 10.2M 2s
>     301100K .......... .......... .......... .......... .......... 94% 37.4M 2s
>     301150K .......... .......... .......... .......... .......... 94% 34.0M 2s
>     301200K .......... .......... .......... .......... .......... 94% 52.9M 2s
>     301250K .......... .......... .......... .......... .......... 95% 19.3M 2s
>     301300K .......... .......... .......... .......... .......... 95% 58.0M 2s
>     301350K .......... .......... .......... .......... .......... 95% 13.9M 2s
>     301400K .......... .......... .......... .......... .......... 95% 13.7M 2s
>     301450K .......... .......... .......... .......... .......... 95% 32.7M 2s
>     301500K .......... .......... .......... .......... .......... 95% 58.2M 2s
>     301550K .......... .......... .......... .......... .......... 95% 56.1M 2s
>     301600K .......... .......... .......... .......... .......... 95% 3.90M 2s
>     301650K .......... .......... .......... .......... .......... 95% 4.97M 2s
>     301700K .......... .......... .......... .......... .......... 95% 33.6M 2s
>     301750K .......... .......... .......... .......... .......... 95% 21.2M 2s
>     301800K .......... .......... .......... .......... .......... 95% 79.4M 2s
>     301850K .......... .......... .......... .......... .......... 95% 1.45M 2s
>     301900K .......... .......... .......... .......... .......... 95% 2.77M 2s
>     301950K .......... .......... .......... .......... .......... 95% 32.0M 2s
>     302000K .......... .......... .......... .......... .......... 95% 48.9M 2s
>     302050K .......... .......... .......... .......... .......... 95% 51.2M 2s
>     302100K .......... .......... .......... .......... .......... 95% 27.6M 2s
>     302150K .......... .......... .......... .......... .......... 95%  969K 2s
>     302200K .......... .......... .......... .......... .......... 95% 26.0M 2s
>     302250K .......... .......... .......... .......... .......... 95% 22.4M 2s
>     302300K .......... .......... .......... .......... .......... 95% 26.2M 2s
>     302350K .......... .......... .......... .......... .......... 95% 8.69M 2s
>     302400K .......... .......... .......... .......... .......... 95% 25.8M 2s
>     302450K .......... .......... .......... .......... .......... 95% 37.4M 2s
>     302500K .......... .......... .......... .......... .......... 95% 16.0M 2s
>     302550K .......... .......... .......... .......... .......... 95% 44.6M 2s
>     302600K .......... .......... .......... .......... .......... 95%  112M 2s
>     302650K .......... .......... .......... .......... .......... 95% 92.7M 2s
>     302700K .......... .......... .......... .......... .......... 95% 50.7M 2s
>     302750K .......... .......... .......... .......... .......... 95% 68.7M 2s
>     302800K .......... .......... .......... .......... .......... 95% 52.3M 2s
>     302850K .......... .......... .......... .......... .......... 95%  176M 2s
>     302900K .......... .......... .......... .......... .......... 95% 14.2M 2s
>     302950K .......... .......... .......... .......... .......... 95% 61.1M 2s
>     303000K .......... .......... .......... .......... .......... 95% 48.5M 2s
>     303050K .......... .......... .......... .......... .......... 95% 39.2M 2s
>     303100K .......... .......... .......... .......... .......... 95% 31.6M 2s
>     303150K .......... .......... .......... .......... .......... 95% 3.40M 2s
>     303200K .......... .......... .......... .......... .......... 95% 4.79M 2s
>     303250K .......... .......... .......... .......... .......... 95% 43.1M 2s
>     303300K .......... .......... .......... .......... .......... 95% 39.8M 2s
>     303350K .......... .......... .......... .......... .......... 95% 55.9M 2s
>     303400K .......... .......... .......... .......... .......... 95% 39.9M 2s
>     303450K .......... .......... .......... .......... .......... 95%  981K 2s
>     303500K .......... .......... .......... .......... .......... 95% 48.7M 2s
>     303550K .......... .......... .......... .......... .......... 95% 37.0M 2s
>     303600K .......... .......... .......... .......... .......... 95% 32.2M 2s
>     303650K .......... .......... .......... .......... .......... 95% 27.3M 2s
>     303700K .......... .......... .......... .......... .......... 95%  958K 2s
>     303750K .......... .......... .......... .......... .......... 95% 32.4M 2s
>     303800K .......... .......... .......... .......... .......... 95% 40.1M 2s
>     303850K .......... .......... .......... .......... .......... 95% 24.4M 1s
>     303900K .......... .......... .......... .......... .......... 95% 8.55M 1s
>     303950K .......... .......... .......... .......... .......... 95% 21.2M 1s
>     304000K .......... .......... .......... .......... .......... 95% 82.2M 1s
>     304050K .......... .......... .......... .......... .......... 95% 58.8M 1s
>     304100K .......... .......... .......... .......... .......... 95% 37.1M 1s
>     304150K .......... .......... .......... .......... .......... 95% 51.9M 1s
>     304200K .......... .......... .......... .......... .......... 95% 26.0M 1s
>     304250K .......... .......... .......... .......... .......... 95% 33.1M 1s
>     304300K .......... .......... .......... .......... .......... 95% 47.8M 1s
>     304350K .......... .......... .......... .......... .......... 95% 72.3M 1s
>     304400K .......... .......... .......... .......... .......... 96% 72.8M 1s
>     304450K .......... .......... .......... .......... .......... 96% 14.0M 1s
>     304500K .......... .......... .......... .......... .......... 96% 51.5M 1s
>     304550K .......... .......... .......... .......... .......... 96% 28.7M 1s
>     304600K .......... .......... .......... .......... .......... 96% 26.8M 1s
>     304650K .......... .......... .......... .......... .......... 96% 29.3M 1s
>     304700K .......... .......... .......... .......... .......... 96% 3.83M 1s
>     304750K .......... .......... .......... .......... .......... 96% 5.01M 1s
>     304800K .......... .......... .......... .......... .......... 96% 30.2M 1s
>     304850K .......... .......... .......... .......... .......... 96% 36.2M 1s
>     304900K .......... .......... .......... .......... .......... 96% 29.4M 1s
>     304950K .......... .......... .......... .......... .......... 96% 60.8M 1s
>     305000K .......... .......... .......... .......... .......... 96%  990K 1s
>     305050K .......... .......... .......... .......... .......... 96% 32.3M 1s
>     305100K .......... .......... .......... .......... .......... 96% 28.3M 1s
>     305150K .......... .......... .......... .......... .......... 96% 60.3M 1s
>     305200K .......... .......... .......... .......... .......... 96% 9.72M 1s
>     305250K .......... .......... .......... .......... .......... 96% 1.02M 1s
>     305300K .......... .......... .......... .......... .......... 96% 47.4M 1s
>     305350K .......... .......... .......... .......... .......... 96% 25.3M 1s
>     305400K .......... .......... .......... .......... .......... 96% 20.0M 1s
>     305450K .......... .......... .......... .......... .......... 96% 8.74M 1s
>     305500K .......... .......... .......... .......... .......... 96% 38.1M 1s
>     305550K .......... .......... .......... .......... .......... 96% 41.6M 1s
>     305600K .......... .......... .......... .......... .......... 96% 41.6M 1s
>     305650K .......... .......... .......... .......... .......... 96% 39.6M 1s
>     305700K .......... .......... .......... .......... .......... 96% 56.2M 1s
>     305750K .......... .......... .......... .......... .......... 96% 33.8M 1s
>     305800K .......... .......... .......... .......... .......... 96% 40.9M 1s
>     305850K .......... .......... .......... .......... .......... 96% 26.6M 1s
>     305900K .......... .......... .......... .......... .......... 96% 93.4M 1s
>     305950K .......... .......... .......... .......... .......... 96% 35.0M 1s
>     306000K .......... .......... .......... .......... .......... 96% 16.2M 1s
>     306050K .......... .......... .......... .......... .......... 96% 50.1M 1s
>     306100K .......... .......... .......... .......... .......... 96% 27.6M 1s
>     306150K .......... .......... .......... .......... .......... 96% 39.4M 1s
>     306200K .......... .......... .......... .......... .......... 96% 27.9M 1s
>     306250K .......... .......... .......... .......... .......... 96% 3.71M 1s
>     306300K .......... .......... .......... .......... .......... 96% 5.03M 1s
>     306350K .......... .......... .......... .......... .......... 96% 24.6M 1s
>     306400K .......... .......... .......... .......... .......... 96% 42.8M 1s
>     306450K .......... .......... .......... .......... .......... 96% 59.0M 1s
>     306500K .......... .......... .......... .......... .......... 96% 34.6M 1s
>     306550K .......... .......... .......... .......... .......... 96%  986K 1s
>     306600K .......... .......... .......... .......... .......... 96% 16.5M 1s
>     306650K .......... .......... .......... .......... .......... 96% 51.3M 1s
>     306700K .......... .......... .......... .......... .......... 96% 45.1M 1s
>     306750K .......... .......... .......... .......... .......... 96% 12.3M 1s
>     306800K .......... .......... .......... .......... .......... 96% 1.01M 1s
>     306850K .......... .......... .......... .......... .......... 96% 48.9M 1s
>     306900K .......... .......... .......... .......... .......... 96% 29.3M 1s
>     306950K .......... .......... .......... .......... .......... 96% 20.4M 1s
>     307000K .......... .......... .......... .......... .......... 96% 8.58M 1s
>     307050K .......... .......... .......... .......... .......... 96% 45.3M 1s
>     307100K .......... .......... .......... .......... .......... 96% 23.8M 1s
>     307150K .......... .......... .......... .......... .......... 96% 50.9M 1s
>     307200K .......... .......... .......... .......... .......... 96% 46.0M 1s
>     307250K .......... .......... .......... .......... .......... 96% 37.0M 1s
>     307300K .......... .......... .......... .......... .......... 96% 62.4M 1s
>     307350K .......... .......... .......... .......... .......... 96% 32.4M 1s
>     307400K .......... .......... .......... .......... .......... 96% 42.2M 1s
>     307450K .......... .......... .......... .......... .......... 96% 45.6M 1s
>     307500K .......... .......... .......... .......... .......... 96% 44.3M 1s
>     307550K .......... .......... .......... .......... .......... 97% 16.7M 1s
>     307600K .......... .......... .......... .......... .......... 97% 12.4M 1s
>     307650K .......... .......... .......... .......... .......... 97% 95.4M 1s
>     307700K .......... .......... .......... .......... .......... 97%  256M 1s
>     307750K .......... .......... .......... .......... .......... 97% 41.1M 1s
>     307800K .......... .......... .......... .......... .......... 97% 44.6M 1s
>     307850K .......... .......... .......... .......... .......... 97% 2.23M 1s
>     307900K .......... .......... .......... .......... .......... 97% 20.2M 1s
>     307950K .......... .......... .......... .......... .......... 97% 55.0M 1s
>     308000K .......... .......... .......... .......... .......... 97% 71.1M 1s
>     308050K .......... .......... .......... .......... .......... 97% 43.4M 1s
>     308100K .......... .......... .......... .......... .......... 97% 1.43M 1s
>     308150K .......... .......... .......... .......... .......... 97% 2.54M 1s
>     308200K .......... .......... .......... .......... .......... 97% 26.5M 1s
>     308250K .......... .......... .......... .......... .......... 97% 58.8M 1s
>     308300K .......... .......... .......... .......... .......... 97% 11.9M 1s
>     308350K .......... .......... .......... .......... .......... 97% 45.1M 1s
>     308400K .......... .......... .......... .......... .......... 97% 1.02M 1s
>     308450K .......... .......... .......... .......... .......... 97% 30.6M 1s
>     308500K .......... .......... .......... .......... .......... 97% 18.3M 1s
>     308550K .......... .......... .......... .......... .......... 97% 9.12M 1s
>     308600K .......... .......... .......... .......... .......... 97% 38.6M 1s
>     308650K .......... .......... .......... .......... .......... 97% 26.5M 1s
>     308700K .......... .......... .......... .......... .......... 97% 92.4M 1s
>     308750K .......... .......... .......... .......... .......... 97% 36.7M 1s
>     308800K .......... .......... .......... .......... .......... 97% 33.1M 1s
>     308850K .......... .......... .......... .......... .......... 97% 51.0M 1s
>     308900K .......... .......... .......... .......... .......... 97% 30.4M 1s
>     308950K .......... .......... .......... .......... .......... 97% 24.7M 1s
>     309000K .......... .......... .......... .......... .......... 97% 93.6M 1s
>     309050K .......... .......... .......... .......... .......... 97%  150M 1s
>     309100K .......... .......... .......... .......... .......... 97% 27.8M 1s
>     309150K .......... .......... .......... .......... .......... 97% 3.34M 1s
>     309200K .......... .......... .......... .......... .......... 97% 27.5M 1s
>     309250K .......... .......... .......... .......... .......... 97% 23.4M 1s
>     309300K .......... .......... .......... .......... .......... 97%  257M 1s
>     309350K .......... .......... .......... .......... .......... 97%  260M 1s
>     309400K .......... .......... .......... .......... .......... 97% 5.70M 1s
>     309450K .......... .......... .......... .......... .......... 97% 9.49M 1s
>     309500K .......... .......... .......... .......... .......... 97% 35.0M 1s
>     309550K .......... .......... .......... .......... .......... 97% 40.6M 1s
>     309600K .......... .......... .......... .......... .......... 97% 95.9M 1s
>     309650K .......... .......... .......... .......... .......... 97% 1.43M 1s
>     309700K .......... .......... .......... .......... .......... 97% 2.59M 1s
>     309750K .......... .......... .......... .......... .......... 97% 45.9M 1s
>     309800K .......... .......... .......... .......... .......... 97% 49.8M 1s
>     309850K .......... .......... .......... .......... .......... 97% 10.6M 1s
>     309900K .......... .......... .......... .......... .......... 97% 42.8M 1s
>     309950K .......... .......... .......... .......... .......... 97% 1.02M 1s
>     310000K .......... .......... .......... .......... .......... 97% 26.9M 1s
>     310050K .......... .......... .......... .......... .......... 97% 17.1M 1s
>     310100K .......... .......... .......... .......... .......... 97% 9.39M 1s
>     310150K .......... .......... .......... .......... .......... 97% 34.7M 1s
>     310200K .......... .......... .......... .......... .......... 97% 48.9M 1s
>     310250K .......... .......... .......... .......... .......... 97% 45.4M 1s
>     310300K .......... .......... .......... .......... .......... 97% 31.5M 1s
>     310350K .......... .......... .......... .......... .......... 97% 44.5M 1s
>     310400K .......... .......... .......... .......... .......... 97% 42.9M 1s
>     310450K .......... .......... .......... .......... .......... 97% 35.4M 1s
>     310500K .......... .......... .......... .......... .......... 97% 42.5M 1s
>     310550K .......... .......... .......... .......... .......... 97% 45.8M 1s
>     310600K .......... .......... .......... .......... .......... 97% 53.6M 1s
>     310650K .......... .......... .......... .......... .......... 97% 27.6M 1s
>     310700K .......... .......... .......... .......... .......... 97% 18.0M 1s
>     310750K .......... .......... .......... .......... .......... 98% 3.73M 1s
>     310800K .......... .......... .......... .......... .......... 98% 57.3M 1s
>     310850K .......... .......... .......... .......... .......... 98% 41.4M 1s
>     310900K .......... .......... .......... .......... .......... 98% 51.8M 1s
>     310950K .......... .......... .......... .......... .......... 98% 5.56M 1s
>     311000K .......... .......... .......... .......... .......... 98% 9.97M 1s
>     311050K .......... .......... .......... .......... .......... 98% 46.4M 1s
>     311100K .......... .......... .......... .......... .......... 98% 40.7M 1s
>     311150K .......... .......... .......... .......... .......... 98% 45.4M 1s
>     311200K .......... .......... .......... .......... .......... 98% 1.43M 1s
>     311250K .......... .......... .......... .......... .......... 98% 2.58M 1s
>     311300K .......... .......... .......... .......... .......... 98% 39.9M 1s
>     311350K .......... .......... .......... .......... .......... 98% 51.2M 1s
>     311400K .......... .......... .......... .......... .......... 98% 40.9M 1s
>     311450K .......... .......... .......... .......... .......... 98% 10.9M 1s
>     311500K .......... .......... .......... .......... .......... 98% 1.02M 1s
>     311550K .......... .......... .......... .......... .......... 98% 28.6M 1s
>     311600K .......... .......... .......... .......... .......... 98% 16.7M 1s
>     311650K .......... .......... .......... .......... .......... 98% 9.48M 1s
>     311700K .......... .......... .......... .......... .......... 98% 35.1M 1s
>     311750K .......... .......... .......... .......... .......... 98% 45.7M 1s
>     311800K .......... .......... .......... .......... .......... 98% 47.1M 1s
>     311850K .......... .......... .......... .......... .......... 98% 51.0M 1s
>     311900K .......... .......... .......... .......... .......... 98% 30.7M 1s
>     311950K .......... .......... .......... .......... .......... 98% 48.4M 1s
>     312000K .......... .......... .......... .......... .......... 98% 8.57M 1s
>     312050K .......... .......... .......... .......... .......... 98% 71.1M 1s
>     312100K .......... .......... .......... .......... .......... 98% 51.1M 1s
>     312150K .......... .......... .......... .......... .......... 98% 37.0M 1s
>     312200K .......... .......... .......... .......... .......... 98% 63.5M 1s
>     312250K .......... .......... .......... .......... .......... 98% 39.8M 1s
>     312300K .......... .......... .......... .......... .......... 98% 4.12M 1s
>     312350K .......... .......... .......... .......... .......... 98% 51.5M 1s
>     312400K .......... .......... .......... .......... .......... 98% 52.8M 1s
>     312450K .......... .......... .......... .......... .......... 98% 54.1M 1s
>     312500K .......... .......... .......... .......... .......... 98% 5.51M 1s
>     312550K .......... .......... .......... .......... .......... 98% 10.1M 1s
>     312600K .......... .......... .......... .......... .......... 98% 44.0M 1s
>     312650K .......... .......... .......... .......... .......... 98% 43.4M 0s
>     312700K .......... .......... .......... .......... .......... 98% 44.0M 0s
>     312750K .......... .......... .......... .......... .......... 98% 44.8M 0s
>     312800K .......... .......... .......... .......... .......... 98%  984K 0s
>     312850K .......... .......... .......... .......... .......... 98% 21.6M 0s
>     312900K .......... .......... .......... .......... .......... 98% 46.2M 0s
>     312950K .......... .......... .......... .......... .......... 98% 29.5M 0s
>     313000K .......... .......... .......... .......... .......... 98% 12.1M 0s
>     313050K .......... .......... .......... .......... .......... 98% 1.02M 0s
>     313100K .......... .......... .......... .......... .......... 98% 27.4M 0s
>     313150K .......... .......... .......... .......... .......... 98% 20.3M 0s
>     313200K .......... .......... .......... .......... .......... 98% 27.3M 0s
>     313250K .......... .......... .......... .......... .......... 98% 9.28M 0s
>     313300K .......... .......... .......... .......... .......... 98% 35.4M 0s
>     313350K .......... .......... .......... .......... .......... 98% 42.8M 0s
>     313400K .......... .......... .......... .......... .......... 98% 59.4M 0s
>     313450K .......... .......... .......... .......... .......... 98% 35.7M 0s
>     313500K .......... .......... .......... .......... .......... 98% 34.3M 0s
>     313550K .......... .......... .......... .......... .......... 98% 7.98M 0s
>     313600K .......... .......... .......... .......... .......... 98% 17.4M 0s
>     313650K .......... .......... .......... .......... .......... 98% 28.5M 0s
>     313700K .......... .......... .......... .......... .......... 98% 44.8M 0s
>     313750K .......... .......... .......... .......... .......... 98% 45.4M 0s
>     313800K .......... .......... .......... .......... .......... 98%  153M 0s
>     313850K .......... .......... .......... .......... .......... 98% 5.46M 0s
>     313900K .......... .......... .......... .......... .......... 99% 19.9M 0s
>     313950K .......... .......... .......... .......... .......... 99% 44.3M 0s
>     314000K .......... .......... .......... .......... .......... 99% 77.7M 0s
>     314050K .......... .......... .......... .......... .......... 99% 6.61M 0s
>     314100K .......... .......... .......... .......... .......... 99% 10.2M 0s
>     314150K .......... .......... .......... .......... .......... 99% 45.6M 0s
>     314200K .......... .......... .......... .......... .......... 99% 23.6M 0s
>     314250K .......... .......... .......... .......... .......... 99% 69.4M 0s
>     314300K .......... .......... .......... .......... .......... 99% 52.4M 0s
>     314350K .......... .......... .......... .......... .......... 99%  988K 0s
>     314400K .......... .......... .......... .......... .......... 99% 23.1M 0s
>     314450K .......... .......... .......... .......... .......... 99% 38.1M 0s
>     314500K .......... .......... .......... .......... .......... 99% 31.4M 0s
>     314550K .......... .......... .......... .......... .......... 99% 12.3M 0s
>     314600K .......... .......... .......... .......... .......... 99% 1.02M 0s
>     314650K .......... .......... .......... .......... .......... 99% 48.8M 0s
>     314700K .......... .......... .......... .......... .......... 99% 28.0M 0s
>     314750K .......... .......... .......... .......... .......... 99% 16.0M 0s
>     314800K .......... .......... .......... .......... .......... 99% 9.35M 0s
>     314850K .......... .......... .......... .......... .......... 99% 32.3M 0s
>     314900K .......... .......... .......... .......... .......... 99% 51.0M 0s
>     314950K .......... .......... .......... .......... .......... 99% 48.9M 0s
>     315000K .......... .......... .......... .......... .......... 99% 34.8M 0s
>     315050K .......... .......... .......... .......... .......... 99% 40.7M 0s
>     315100K .......... .......... .......... .......... .......... 99% 41.5M 0s
>     315150K .......... .......... .......... .......... .......... 99% 6.18M 0s
>     315200K .......... .......... .......... .......... .......... 99% 42.0M 0s
>     315250K .......... .......... .......... .......... .......... 99% 22.5M 0s
>     315300K .......... .......... .......... .......... .......... 99% 39.1M 0s
>     315350K .......... .......... .......... .......... .......... 99% 35.3M 0s
>     315400K .......... .......... .......... .......... .......... 99% 6.76M 0s
>     315450K .......... .......... .......... .......... .......... 99% 29.6M 0s
>     315500K .......... .......... .......... .......... .......... 99% 39.5M 0s
>     315550K .......... .......... .......... .......... .......... 99% 73.4M 0s
>     315600K .......... .......... .......... .......... .......... 99% 73.2M 0s
>     315650K .......... .......... .......... .......... .......... 99% 3.94M 0s
>     315700K .......... .......... .......... .......... .......... 99% 32.0M 0s
>     315750K .......... .......... .......... .......... .......... 99% 66.6M 0s
>     315800K .......... .......... .......... .......... .......... 99% 15.7M 0s
>     315850K .......... .......... .......... .......... .......... 99%  144M 0s
>     315900K .......... .......... .......... .......... .......... 99%  999K 0s
>     315950K .......... .......... .......... .......... .......... 99% 14.3M 0s
>     316000K .......... .......... .......... .......... .......... 99%  144M 0s
>     316050K .......... .......... .......... .......... .......... 99% 29.3M 0s
>     316100K .......... .......... .......... .......... .......... 99% 12.3M 0s
>     316150K .......... .......... .......... .......... .......... 99%  101M 0s
>     316200K .......... .......... .......... .......... .......... 99% 1.02M 0s
>     316250K .......... .......... .......... .......... .......... 99% 28.2M 0s
>     316300K .......... .......... .......... .......... .......... 99% 17.8M 0s
>     316350K .......... .......... .......... .......... .......... 99% 9.62M 0s
>     316400K .......... .......... .......... .......... .......... 99% 37.1M 0s
>     316450K .......... .......... .......... .......... .......... 99% 35.1M 0s
>     316500K .......... .......... .......... .......... .......... 99% 38.6M 0s
>     316550K .......... .......... .......... .......... .......... 99% 48.5M 0s
>     316600K .......... .......... .......... .......... .......... 99% 42.9M 0s
>     316650K .......... .......... .......... .......... .......... 99% 45.0M 0s
>     316700K .......... .......... .......... .......... .......... 99% 5.88M 0s
>     316750K .......... .......... .......... .......... .......... 99% 32.4M 0s
>     316800K .......... .......... .......... .......... .......... 99% 38.1M 0s
>     316850K .......... .......... .......... .......... .......... 99% 53.5M 0s
>     316900K .......... .......... .......... .......... .......... 99% 34.9M 0s
>     316950K .......... .......... .......... .......... .......... 99% 6.17M 0s
>     317000K .......... .......... .......... .......... .......... 99% 32.9M 0s
>     317050K .......... .......... .......... .......... .......... 99% 42.3M 0s
>     317100K .......... ...                                        100% 33.7M=36s
>
>     2019-05-10 05:25:47 (8.64 MB/s) - ‘OB_tei_7-2_CC-BY-NC.zip’ saved [324723900/324723900]

In [ ]:
pwd && ls -al

  

>     /databricks/driver
>     total 329248
>     drwxr-xr-x 1 root   root        4096 May 10 05:25 .
>     drwxr-xr-x 1 root   root        4096 Nov 10 11:07 ..
>     drwxr-xr-x 2 root   root        4096 Apr  9 22:35 books
>     drwxr-xr-x 2 root   root        4096 Jan  1  1970 conf
>     -rw-r--r-- 1 root   root         731 Nov 10 11:07 derby.log
>     drwxr-xr-x 3 root   root        4096 Nov 10 11:06 eventlogs
>     drwxr-xr-x 2 root   root      155648 May 10 05:00 ganglia
>     drwxr-xr-x 2 root   root        4096 Nov 10 11:07 library-install-logs
>     drwxr-xr-x 2 root   root       32768 May 10 05:01 logs
>     -rw-r--r-- 1 root   root   324723900 Sep  2  2016 OB_tei_7-2_CC-BY-NC.zip
>     -rw-r--r-- 1 root   root     1209487 May 10 05:24 OB-tiny_tei_7-2_CC-BY-NC.zip
>     -rw-r--r-- 1 root   root     3652403 Feb 18  2016 old_sou.tar.gz
>     drwxrwxr-x 2 ubuntu ubuntu     12288 Aug 31  2017 sou
>     -rw-r--r-- 1 root   root     3652403 Feb 18  2016 sou.tar.gz
>     -rw-r--r-- 1 root   root     3666765 Aug 31  2017 sou.tgz

  

Make sure you comment/uncomment the right files depending on wheter you
have downloaded the tiny dataset or the big one.

In [ ]:
unzip OB-tiny_tei_7-2_CC-BY-NC.zip
#unzip OB_tei_7-2_CC-BY-NC.zip


  

>     Archive:  OB_tei_7-2_CC-BY-NC.zip
>        creating: tei/
>        creating: tei/ordinarysAccounts/
>       inflating: tei/ordinarysAccounts/OA16930726.xml  
>       inflating: tei/ordinarysAccounts/OA17480622.xml  
>       inflating: tei/ordinarysAccounts/OA17150520.xml  
>       inflating: tei/ordinarysAccounts/OA17310514.xml  
>       inflating: tei/ordinarysAccounts/OA17160713.xml  
>       inflating: tei/ordinarysAccounts/OA17450709.xml  
>       inflating: tei/ordinarysAccounts/OA17070124.xml  
>       inflating: tei/ordinarysAccounts/OA16760830.xml  
>       inflating: tei/ordinarysAccounts/OA17261103.xml  
>       inflating: tei/ordinarysAccounts/OA17230909.xml  
>       inflating: tei/ordinarysAccounts/OA17500326.xml  
>       inflating: tei/ordinarysAccounts/OA16911218.xml  
>       inflating: tei/ordinarysAccounts/OA17280626.xml  
>       inflating: tei/ordinarysAccounts/OA16930127.xml  
>       inflating: tei/ordinarysAccounts/OA17250430.xml  
>       inflating: tei/ordinarysAccounts/OA17141222.xml  
>       inflating: tei/ordinarysAccounts/OA17511023.xml  
>       inflating: tei/ordinarysAccounts/OA17131223.xml  
>       inflating: tei/ordinarysAccounts/OA16860917.xml  
>       inflating: tei/ordinarysAccounts/OA17630504.xml  
>       inflating: tei/ordinarysAccounts/OA17280212.xml  
>       inflating: tei/ordinarysAccounts/OA16970915.xml  
>       inflating: tei/ordinarysAccounts/OA17370303.xml  
>       inflating: tei/ordinarysAccounts/OA17320605.xml  
>       inflating: tei/ordinarysAccounts/OA17150202.xml  
>       inflating: tei/ordinarysAccounts/OA17550728.xml  
>       inflating: tei/ordinarysAccounts/OA16900613.xml  
>       inflating: tei/ordinarysAccounts/OA17170626.xml  
>       inflating: tei/ordinarysAccounts/OA17060515.xml  
>       inflating: tei/ordinarysAccounts/OA16970528.xml  
>       inflating: tei/ordinarysAccounts/OA16840526.xml  
>       inflating: tei/ordinarysAccounts/OA17320809.xml  
>       inflating: tei/ordinarysAccounts/OA16971103.xml  
>       inflating: tei/ordinarysAccounts/OA17080428.xml  
>       inflating: tei/ordinarysAccounts/OA17081027.xml  
>       inflating: tei/ordinarysAccounts/OA16970423.xml  
>       inflating: tei/ordinarysAccounts/OA16911023.xml  
>       inflating: tei/ordinarysAccounts/OA17520713.xml  
>       inflating: tei/ordinarysAccounts/OA16870127.xml  
>       inflating: tei/ordinarysAccounts/OA16971222.xml  
>       inflating: tei/ordinarysAccounts/OA17120527a.xml  
>       inflating: tei/ordinarysAccounts/OA17581002.xml  
>       inflating: tei/ordinarysAccounts/OA16791219.xml  
>       inflating: tei/ordinarysAccounts/OA17121223.xml  
>       inflating: tei/ordinarysAccounts/OA17200413.xml  
>       inflating: tei/ordinarysAccounts/OA16910928.xml  
>       inflating: tei/ordinarysAccounts/OA17061213.xml  
>       inflating: tei/ordinarysAccounts/OA17301116.xml  
>       inflating: tei/ordinarysAccounts/OA17130313.xml  
>       inflating: tei/ordinarysAccounts/OA16931023.xml  
>       inflating: tei/ordinarysAccounts/OA17000720.xml  
>       inflating: tei/ordinarysAccounts/OA17470121.xml  
>       inflating: tei/ordinarysAccounts/OA17420712.xml  
>       inflating: tei/ordinarysAccounts/OA17290207.xml  
>       inflating: tei/ordinarysAccounts/OA17170520.xml  
>       inflating: tei/ordinarysAccounts/OA17180531.xml  
>       inflating: tei/ordinarysAccounts/OA17540626.xml  
>       inflating: tei/ordinarysAccounts/OA17121031.xml  
>       inflating: tei/ordinarysAccounts/OA17111024.xml  
>       inflating: tei/ordinarysAccounts/OA17321009.xml  
>       inflating: tei/ordinarysAccounts/OA17460425.xml  
>       inflating: tei/ordinarysAccounts/OA17521011.xml  
>       inflating: tei/ordinarysAccounts/OA17530528.xml  
>       inflating: tei/ordinarysAccounts/OA17340211.xml  
>       inflating: tei/ordinarysAccounts/OA17531210.xml  
>       inflating: tei/ordinarysAccounts/OA17380118.xml  
>       inflating: tei/ordinarysAccounts/OA17080924.xml  
>       inflating: tei/ordinarysAccounts/OA17320306.xml  
>       inflating: tei/ordinarysAccounts/OA17560920.xml  
>       inflating: tei/ordinarysAccounts/OA17230208.xml  
>       inflating: tei/ordinarysAccounts/OA17000419.xml  
>       inflating: tei/ordinarysAccounts/OA17230617.xml  
>       inflating: tei/ordinarysAccounts/OA16840710.xml  
>       inflating: tei/ordinarysAccounts/OA17560223.xml  
>       inflating: tei/ordinarysAccounts/OA17550512.xml  
>       inflating: tei/ordinarysAccounts/OA16950417.xml  
>       inflating: tei/ordinarysAccounts/OA16921026.xml  
>       inflating: tei/ordinarysAccounts/OA16850304.xml  
>       inflating: tei/ordinarysAccounts/OA17640611.xml  
>       inflating: tei/ordinarysAccounts/OA16920909.xml  
>       inflating: tei/ordinarysAccounts/OA17190722.xml  
>       inflating: tei/ordinarysAccounts/OA16891023.xml  
>       inflating: tei/ordinarysAccounts/OA17440217.xml  
>       inflating: tei/ordinarysAccounts/OA17420407.xml  
>       inflating: tei/ordinarysAccounts/OA17440315.xml  
>       inflating: tei/ordinarysAccounts/OA17530416.xml  
>       inflating: tei/ordinarysAccounts/OA17520601.xml  
>       inflating: tei/ordinarysAccounts/OA16841017.xml  
>       inflating: tei/ordinarysAccounts/OA16840917.xml  
>       inflating: tei/ordinarysAccounts/OA17190525.xml  
>       inflating: tei/ordinarysAccounts/OA17331219.xml  
>       inflating: tei/ordinarysAccounts/OA16780906.xml  
>       inflating: tei/ordinarysAccounts/OA17480318.xml  
>       inflating: tei/ordinarysAccounts/OA16870413.xml  
>       inflating: tei/ordinarysAccounts/OA17331006.xml  
>       inflating: tei/ordinarysAccounts/OA17020529.xml  
>       inflating: tei/ordinarysAccounts/OA17600428.xml  
>       inflating: tei/ordinarysAccounts/OA17420507.xml  
>       inflating: tei/ordinarysAccounts/OA17440608.xml  
>       inflating: tei/ordinarysAccounts/OA17251103.xml  
>       inflating: tei/ordinarysAccounts/OA17640711.xml  
>       inflating: tei/ordinarysAccounts/OA17500706.xml  
>       inflating: tei/ordinarysAccounts/OA17180317.xml  
>       inflating: tei/ordinarysAccounts/OA17310616.xml  
>       inflating: tei/ordinarysAccounts/OA17211222.xml  
>       inflating: tei/ordinarysAccounts/OA16790121.xml  
>       inflating: tei/ordinarysAccounts/OA16980126.xml  
>       inflating: tei/ordinarysAccounts/OA17050309.xml  
>       inflating: tei/ordinarysAccounts/OA17241207.xml  
>       inflating: tei/ordinarysAccounts/OA17000719.xml  
>       inflating: tei/ordinarysAccounts/OA17070718.xml  
>       inflating: tei/ordinarysAccounts/OA16940228.xml  
>       inflating: tei/ordinarysAccounts/OA17181031.xml  
>       inflating: tei/ordinarysAccounts/OA16901024.xml  
>       inflating: tei/ordinarysAccounts/OA17200919.xml  
>       inflating: tei/ordinarysAccounts/OA17360705.xml  
>       inflating: tei/ordinarysAccounts/OA16850724.xml  
>       inflating: tei/ordinarysAccounts/OA16970716.xml  
>       inflating: tei/ordinarysAccounts/OA17270811.xml  
>       inflating: tei/ordinarysAccounts/OA16771219.xml  
>       inflating: tei/ordinarysAccounts/OA17280911.xml  
>       inflating: tei/ordinarysAccounts/OA17210512.xml  
>       inflating: tei/ordinarysAccounts/OA17391221.xml  
>       inflating: tei/ordinarysAccounts/OA17250201.xml  
>       inflating: tei/ordinarysAccounts/OA17220208.xml  
>       inflating: tei/ordinarysAccounts/OA17090518.xml  
>       inflating: tei/ordinarysAccounts/OA17210403.xml  
>       inflating: tei/ordinarysAccounts/OA17140310.xml  
>       inflating: tei/ordinarysAccounts/OA17270918.xml  
>       inflating: tei/ordinarysAccounts/OA17520922.xml  
>       inflating: tei/ordinarysAccounts/OA17441005.xml  
>       inflating: tei/ordinarysAccounts/OA17460404.xml  
>       inflating: tei/ordinarysAccounts/OA17200815.xml  
>       inflating: tei/ordinarysAccounts/OA17000301.xml  
>       inflating: tei/ordinarysAccounts/OA17260803.xml  
>       inflating: tei/ordinarysAccounts/OA16790307.xml  
>       inflating: tei/ordinarysAccounts/OA17290324.xml  
>       inflating: tei/ordinarysAccounts/OA17050207.xml  
>       inflating: tei/ordinarysAccounts/OA17621013.xml  
>       inflating: tei/ordinarysAccounts/OA17170201.xml  
>       inflating: tei/ordinarysAccounts/OA17230408.xml  
>       inflating: tei/ordinarysAccounts/OA17420113.xml  
>       inflating: tei/ordinarysAccounts/OA17061025.xml  
>       inflating: tei/ordinarysAccounts/OA17481028.xml  
>       inflating: tei/ordinarysAccounts/OA17470731.xml  
>       inflating: tei/ordinarysAccounts/OA17591119.xml  
>       inflating: tei/ordinarysAccounts/OA16950712.xml  
>       inflating: tei/ordinarysAccounts/OA17390314.xml  
>       inflating: tei/ordinarysAccounts/OA17150511.xml  
>       inflating: tei/ordinarysAccounts/OA17130925.xml  
>       inflating: tei/ordinarysAccounts/OA16790122.xml  
>       inflating: tei/ordinarysAccounts/OA17050504.xml  
>       inflating: tei/ordinarysAccounts/OA17160608.xml  
>       inflating: tei/ordinarysAccounts/OA17110421.xml  
>       inflating: tei/ordinarysAccounts/OA16840523.xml  
>       inflating: tei/ordinarysAccounts/OA17030721.xml  
>       inflating: tei/ordinarysAccounts/OA17070912.xml  
>       inflating: tei/ordinarysAccounts/OA17240828.xml  
>       inflating: tei/ordinarysAccounts/OA17360726.xml  
>       inflating: tei/ordinarysAccounts/OA16900912.xml  
>       inflating: tei/ordinarysAccounts/OA17110811.xml  
>       inflating: tei/ordinarysAccounts/OA17260218.xml  
>       inflating: tei/ordinarysAccounts/OA17400507.xml  
>       inflating: tei/ordinarysAccounts/OA16790509.xml  
>       inflating: tei/ordinarysAccounts/OA17470617.xml  
>       inflating: tei/ordinarysAccounts/OA16801027.xml  
>       inflating: tei/ordinarysAccounts/OA17001220.xml  
>       inflating: tei/ordinarysAccounts/OA17640307.xml  
>       inflating: tei/ordinarysAccounts/OA17190223.xml  
>       inflating: tei/ordinarysAccounts/OA17291222.xml  
>       inflating: tei/ordinarysAccounts/OA17571123.xml  
>       inflating: tei/ordinarysAccounts/OA17360524.xml  
>       inflating: tei/ordinarysAccounts/OA17381222.xml  
>       inflating: tei/ordinarysAccounts/OA17031103.xml  
>       inflating: tei/ordinarysAccounts/OA17210705.xml  
>       inflating: tei/ordinarysAccounts/OA17120527.xml  
>       inflating: tei/ordinarysAccounts/OA17251222.xml  
>       inflating: tei/ordinarysAccounts/OA17220924.xml  
>       inflating: tei/ordinarysAccounts/OA17410914.xml  
>       inflating: tei/ordinarysAccounts/OA16910603.xml  
>       inflating: tei/ordinarysAccounts/OA17410612.xml  
>       inflating: tei/ordinarysAccounts/OA17270213.xml  
>       inflating: tei/ordinarysAccounts/OA17300512.xml  
>       inflating: tei/ordinarysAccounts/OA17260912.xml  
>       inflating: tei/ordinarysAccounts/OA17120919.xml  
>       inflating: tei/ordinarysAccounts/OA17040621.xml  
>       inflating: tei/ordinarysAccounts/OA16770316.xml  
>       inflating: tei/ordinarysAccounts/OA17260509.xml  
>       inflating: tei/ordinarysAccounts/OA17400806.xml  
>       inflating: tei/ordinarysAccounts/OA17140421.xml  
>       inflating: tei/ordinarysAccounts/OA17410731.xml  
>       inflating: tei/ordinarysAccounts/OA16940124.xml  
>       inflating: tei/ordinarysAccounts/OA17511111.xml  
>       inflating: tei/ordinarysAccounts/OA17160919.xml  
>       inflating: tei/ordinarysAccounts/OA17470729.xml  
>       inflating: tei/ordinarysAccounts/OA17290725.xml  
>       inflating: tei/ordinarysAccounts/OA17341002.xml  
>       inflating: tei/ordinarysAccounts/OA16860602.xml  
>       inflating: tei/ordinarysAccounts/OA17301007.xml  
>       inflating: tei/ordinarysAccounts/OA16780123.xml  
>       inflating: tei/ordinarysAccounts/OA17510325.xml  
>       inflating: tei/ordinarysAccounts/OA16920302.xml  
>       inflating: tei/ordinarysAccounts/OA17601208.xml  
>       inflating: tei/ordinarysAccounts/OA17380526.xml  
>       inflating: tei/ordinarysAccounts/OA17460801.xml  
>       inflating: tei/ordinarysAccounts/OA17140129.xml  
>       inflating: tei/ordinarysAccounts/OA16980309.xml  
>       inflating: tei/ordinarysAccounts/OA17501107.xml  
>       inflating: tei/ordinarysAccounts/OA17201026.xml  
>       inflating: tei/ordinarysAccounts/OA17630615.xml  
>       inflating: tei/ordinarysAccounts/OA17030811.xml  
>       inflating: tei/ordinarysAccounts/OA16980803.xml  
>       inflating: tei/ordinarysAccounts/OA17060719.xml  
>       inflating: tei/ordinarysAccounts/OA16851023.xml  
>       inflating: tei/ordinarysAccounts/OA17211023.xml  
>       inflating: tei/ordinarysAccounts/OA17171220.xml  
>       inflating: tei/ordinarysAccounts/OA17101215.xml  
>       inflating: tei/ordinarysAccounts/OA17591003.xml  
>       inflating: tei/ordinarysAccounts/OA17491018.xml  
>       inflating: tei/ordinarysAccounts/OA17520323.xml  
>       inflating: tei/ordinarysAccounts/OA17080128.xml  
>       inflating: tei/ordinarysAccounts/OA17330425.xml  
>       inflating: tei/ordinarysAccounts/OA17700604.xml  
>       inflating: tei/ordinarysAccounts/OA16800121.xml  
>       inflating: tei/ordinarysAccounts/OA17611005.xml  
>       inflating: tei/ordinarysAccounts/OA17371005.xml  
>       inflating: tei/ordinarysAccounts/OA17120801.xml  
>       inflating: tei/ordinarysAccounts/OA17570518.xml  
>       inflating: tei/ordinarysAccounts/OA17500207.xml  
>       inflating: tei/ordinarysAccounts/OA16910918.xml  
>       inflating: tei/ordinarysAccounts/OA16910717.xml  
>       inflating: tei/ordinarysAccounts/OA17540805.xml  
>       inflating: tei/ordinarysAccounts/OA16850904.xml  
>       inflating: tei/ordinarysAccounts/OA16930616.xml  
>       inflating: tei/ordinarysAccounts/OA17430412.xml  
>       inflating: tei/ordinarysAccounts/OA17110808.xml  
>       inflating: tei/ordinarysAccounts/OA16851216.xml  
>       inflating: tei/ordinarysAccounts/OA17600915.xml  
>       inflating: tei/ordinarysAccounts/OA17180317a.xml  
>       inflating: tei/ordinarysAccounts/OA17510211.xml  
>       inflating: tei/ordinarysAccounts/OA17340709.xml  
>       inflating: tei/ordinarysAccounts/OA17510729.xml  
>       inflating: tei/ordinarysAccounts/OA17430518.xml  
>       inflating: tei/ordinarysAccounts/OA16870525.xml  
>       inflating: tei/ordinarysAccounts/OA16910501.xml  
>       inflating: tei/ordinarysAccounts/OA17070912A.xml  
>       inflating: tei/ordinarysAccounts/OA17330528.xml  
>       inflating: tei/ordinarysAccounts/OA17370629.xml  
>       inflating: tei/ordinarysAccounts/OA17310726.xml  
>       inflating: tei/ordinarysAccounts/OA16861025.xml  
>       inflating: tei/ordinarysAccounts/OA16770504.xml  
>       inflating: tei/ordinarysAccounts/OA17091216.xml  
>       inflating: tei/ordinarysAccounts/OA17321016.xml  
>       inflating: tei/ordinarysAccounts/OA17521113.xml  
>       inflating: tei/ordinarysAccounts/OA17500808.xml  
>       inflating: tei/ordinarysAccounts/OA16760517.xml  
>       inflating: tei/ordinarysAccounts/OA16871216.xml  
>       inflating: tei/ordinarysAccounts/OA17110718.xml  
>       inflating: tei/ordinarysAccounts/OA17381108.xml  
>       inflating: tei/ordinarysAccounts/OA17291121.xml  
>       inflating: tei/ordinarysAccounts/OA17180127.xml  
>       inflating: tei/ordinarysAccounts/OA17580501.xml  
>       inflating: tei/ordinarysAccounts/OA17100726.xml  
>       inflating: tei/ordinarysAccounts/OA17141027.xml  
>       inflating: tei/ordinarysAccounts/OA17140922.xml  
>       inflating: tei/ordinarysAccounts/OA17650417.xml  
>       inflating: tei/ordinarysAccounts/OA17240904.xml  
>       inflating: tei/ordinarysAccounts/OA16901222.xml  
>       inflating: tei/ordinarysAccounts/OA17520113.xml  
>       inflating: tei/ordinarysAccounts/OA17020128.xml  
>       inflating: tei/ordinarysAccounts/OA17151223.xml  
>       inflating: tei/ordinarysAccounts/OA17431121.xml  
>       inflating: tei/ordinarysAccounts/OA17130429.xml  
>       inflating: tei/ordinarysAccounts/OA17490220.xml  
>       inflating: tei/ordinarysAccounts/OA17720527.xml  
>       inflating: tei/ordinarysAccounts/OA17501231.xml  
>       inflating: tei/ordinarysAccounts/OA16850506.xml  
>       inflating: tei/ordinarysAccounts/OA17330305.xml  
>       inflating: tei/ordinarysAccounts/OA16950918.xml  
>       inflating: tei/ordinarysAccounts/OA17180527.xml  
>       inflating: tei/ordinarysAccounts/OA17040922.xml  
>       inflating: tei/ordinarysAccounts/OA17200627.xml  
>       inflating: tei/ordinarysAccounts/OA17250524.xml  
>       inflating: tei/ordinarysAccounts/OA17180806.xml  
>       inflating: tei/ordinarysAccounts/OA17380308.xml  
>       inflating: tei/ordinarysAccounts/OA16950524.xml  
>       inflating: tei/ordinarysAccounts/OA16891221.xml  
>       inflating: tei/ordinarysAccounts/OA16791024.xml  
>       inflating: tei/ordinarysAccounts/OA17191106.xml  
>       inflating: tei/ordinarysAccounts/OA17081217.xml  
>       inflating: tei/ordinarysAccounts/OA17220521.xml  
>       inflating: tei/ordinarysAccounts/OA17621110.xml  
>       inflating: tei/ordinarysAccounts/OA17530212.xml  
>       inflating: tei/ordinarysAccounts/OA17090916.xml  
>       inflating: tei/ordinarysAccounts/OA17190213.xml  
>       inflating: tei/ordinarysAccounts/OA16900723.xml  
>       inflating: tei/ordinarysAccounts/OA17490426.xml  
>       inflating: tei/ordinarysAccounts/OA17100317.xml  
>       inflating: tei/ordinarysAccounts/OA17110302.xml  
>       inflating: tei/ordinarysAccounts/OA17450607.xml  
>       inflating: tei/ordinarysAccounts/OA17280327.xml  
>       inflating: tei/ordinarysAccounts/OA17100915.xml  
>       inflating: tei/ordinarysAccounts/OA17520427.xml  
>       inflating: tei/ordinarysAccounts/OA17230805.xml  
>       inflating: tei/ordinarysAccounts/OA17360204.xml  
>       inflating: tei/ordinarysAccounts/OA16910126.xml  
>       inflating: tei/ordinarysAccounts/OA16850506A.xml  
>       inflating: tei/ordinarysAccounts/OA17340308.xml  
>       inflating: tei/ordinarysAccounts/OA17151028.xml  
>       inflating: tei/ordinarysAccounts/OA16951213.xml  
>       inflating: tei/ordinarysAccounts/OA17540204.xml  
>       inflating: tei/ordinarysAccounts/OA17161219.xml  
>       inflating: tei/ordinarysAccounts/OA17280520.xml  
>       inflating: tei/ordinarysAccounts/OA17240203.xml  
>       inflating: tei/ordinarysAccounts/OA17040510.xml  
>       inflating: tei/ordinarysAccounts/OA17041220.xml  
>       inflating: tei/ordinarysAccounts/OA17120618.xml  
>       inflating: tei/ordinarysAccounts/OA16761025.xml  
>       inflating: tei/ordinarysAccounts/OA17721014.xml  
>       inflating: tei/ordinarysAccounts/OA17311220.xml  
>       inflating: tei/ordinarysAccounts/OA17631228.xml  
>       inflating: tei/ordinarysAccounts/OA17351110.xml  
>       inflating: tei/ordinarysAccounts/OA17260627.xml  
>       inflating: tei/ordinarysAccounts/OA17220718.xml  
>       inflating: tei/ordinarysAccounts/OA16910226.xml  
>       inflating: tei/ordinarysAccounts/OA17310308.xml  
>       inflating: tei/ordinarysAccounts/OA16860528.xml  
>       inflating: tei/ordinarysAccounts/OA17601025.xml  
>       inflating: tei/ordinarysAccounts/OA17630824.xml  
>       inflating: tei/ordinarysAccounts/OA17240429.xml  
>       inflating: tei/ordinarysAccounts/OA17531029.xml  
>       inflating: tei/ordinarysAccounts/OA17700419.xml  
>       inflating: tei/ordinarysAccounts/OA17700214.xml  
>       inflating: tei/ordinarysAccounts/OA17000524.xml  
>       inflating: tei/ordinarysAccounts/OA17390702.xml  
>       inflating: tei/ordinarysAccounts/OA16920520.xml  
>       inflating: tei/ordinarysAccounts/OA16781216.xml  
>       inflating: tei/ordinarysAccounts/OA17550317.xml  
>       inflating: tei/ordinarysAccounts/OA16960129.xml  
>       inflating: tei/ordinarysAccounts/OA16890715.xml  
>       inflating: tei/ordinarysAccounts/OA17500516.xml  
>       inflating: tei/ordinarysAccounts/OA17490317.xml  
>       inflating: tei/ordinarysAccounts/OA17030310.xml  
>       inflating: tei/ordinarysAccounts/OA16990802.xml  
>       inflating: tei/ordinarysAccounts/OA17410916.xml  
>       inflating: tei/ordinarysAccounts/OA17650213.xml  
>       inflating: tei/ordinarysAccounts/OA16900509.xml  
>       inflating: tei/ordinarysAccounts/OA17000906.xml  
>       inflating: tei/ordinarysAccounts/OA17110525.xml  
>       inflating: tei/ordinarysAccounts/OA17131024.xml  
>       inflating: tei/ordinarysAccounts/OA16850610.xml  
>       inflating: tei/ordinarysAccounts/OA17271120.xml  
>       inflating: tei/ordinarysAccounts/OA16920415.xml  
>       inflating: tei/ordinarysAccounts/OA16780306.xml  
>       inflating: tei/ordinarysAccounts/OA17390803.xml  
>       inflating: tei/ordinarysAccounts/OA17320726.xml  
>       inflating: tei/ordinarysAccounts/OA16760705.xml  
>       inflating: tei/ordinarysAccounts/OA17490703.xml  
>       inflating: tei/ordinarysAccounts/OA17361102.xml  
>       inflating: tei/ordinarysAccounts/OA17450315.xml  
>       inflating: tei/ordinarysAccounts/OA17111222.xml  
>       inflating: tei/ordinarysAccounts/OA17431021.xml  
>       inflating: tei/ordinarysAccounts/OA17210728.xml  
>       inflating: tei/ordinarysAccounts/OA17200129.xml  
>       inflating: tei/ordinarysAccounts/OA17151102.xml  
>       inflating: tei/ordinarysAccounts/OA17421118.xml  
>       inflating: tei/ordinarysAccounts/OA17241111.xml  
>       inflating: tei/ordinarysAccounts/OA16841217.xml  
>       inflating: tei/ordinarysAccounts/OA17541209.xml  
>       inflating: tei/ordinarysAccounts/OA17530806.xml  
>       inflating: tei/ordinarysAccounts/OA16771017.xml  
>       inflating: tei/ordinarysAccounts/OA17360927.xml  
>       inflating: tei/ordinarysAccounts/OA16860120.xml  
>       inflating: tei/ordinarysAccounts/OA17190608.xml  
>       inflating: tei/ordinarysAccounts/OA17631123.xml  
>       inflating: tei/ordinarysAccounts/OA17460620.xml  
>       inflating: tei/ordinarysAccounts/OA16780522.xml  
>       inflating: tei/ordinarysAccounts/OA17270322.xml  
>       inflating: tei/ordinarysAccounts/OA17600211.xml  
>       inflating: tei/ordinarysAccounts/OA17580701.xml  
>       inflating: tei/ordinarysAccounts/OA17300417.xml  
>       inflating: tei/ordinarysAccounts/OA17240615.xml  
>       inflating: tei/ordinarysAccounts/OA17611111.xml  
>       inflating: tei/ordinarysAccounts/OA17130131.xml  
>       inflating: tei/ordinarysAccounts/OA17210911.xml  
>       inflating: tei/ordinarysAccounts/OA17471116.xml  
>       inflating: tei/ordinarysAccounts/OA17400213.xml  
>       inflating: tei/ordinarysAccounts/OA17620719.xml  
>       inflating: tei/ordinarysAccounts/OA17140528.xml  
>       inflating: tei/ordinarysAccounts/OA17140716.xml  
>       inflating: tei/ordinarysAccounts/OA17520702.xml  
>       inflating: tei/ordinarysAccounts/OA17231106.xml  
>       inflating: tei/ordinarysAccounts/OA17450726.xml  
>       inflating: tei/ordinarysAccounts/OA17540605.xml  
>       inflating: tei/ordinarysAccounts/OA17720708.xml  
>       inflating: tei/ordinarysAccounts/OA17320214.xml  
>       inflating: tei/ordinarysAccounts/OA17631012.xml  
>       inflating: tei/ordinarysAccounts/OA17421122.xml  
>       inflating: tei/ordinarysAccounts/OA17630117.xml  
>       inflating: tei/ordinarysAccounts/OA17290822.xml  
>       inflating: tei/ordinarysAccounts/OA17010129.xml  
>       inflating: tei/ordinarysAccounts/OA17250104.xml  
>       inflating: tei/ordinarysAccounts/OA17021230.xml  
>       inflating: tei/ordinarysAccounts/OA16930201.xml  
>       inflating: tei/ordinarysAccounts/OA17300601.xml  
>       inflating: tei/ordinarysAccounts/OA17220504.xml  
>       inflating: tei/ordinarysAccounts/OA17330129.xml  
>       inflating: tei/ordinarysAccounts/OA16870715.xml  
>       inflating: tei/ordinarysAccounts/OA17471221.xml  
>       inflating: tei/ordinarysAccounts/OA17071024.xml  
>       inflating: tei/ordinarysAccounts/OA17571005.xml  
>       inflating: tei/ordinarysAccounts/OA17441107.xml  
>       inflating: tei/ordinarysAccounts/OA17310924.xml  
>       inflating: tei/ordinarysAccounts/OA17090624.xml  
>       inflating: tei/ordinarysAccounts/OA16930308.xml  
>       inflating: tei/ordinarysAccounts/OA17090323.xml  
>       inflating: tei/ordinarysAccounts/OA17010516.xml  
>       inflating: tei/ordinarysAccounts/OA17221109.xml  
>       inflating: tei/ordinarysAccounts/OA17350922.xml  
>       inflating: tei/ordinarysAccounts/OA17210208.xml  
>       inflating: tei/ordinarysAccounts/OA17040322.xml  
>       inflating: tei/ordinarysAccounts/OA16930517.xml  
>       inflating: tei/ordinarysAccounts/OA17610404.xml  
>       inflating: tei/ordinarysAccounts/OA17070606.xml  
>       inflating: tei/ordinarysAccounts/OA16930920.xml  
>       inflating: tei/ordinarysAccounts/OA17220924a.xml  
>       inflating: tei/ordinarysAccounts/OA16921221.xml  
>       inflating: tei/ordinarysAccounts/OA17150622.xml  
>       inflating: tei/ordinarysAccounts/OA17480511.xml  
>       inflating: tei/ordinarysAccounts/OA17510617.xml  
>       inflating: tei/ordinarysAccounts/OA17401124.xml  
>       inflating: tei/ordinarysAccounts/OA17080303b.xml  
>       inflating: tei/ordinarysAccounts/OA17230525.xml  
>       inflating: tei/ordinarysAccounts/OA17301223.xml  
>       inflating: tei/ordinarysAccounts/OA17160127.xml  
>       inflating: tei/ordinarysAccounts/OA17390530.xml  
>       inflating: tei/ordinarysAccounts/OA17320522.xml  
>       inflating: tei/ordinarysAccounts/OA17380719.xml  
>       inflating: tei/ordinarysAccounts/OA17070502.xml  
>       inflating: tei/ordinarysAccounts/OA16931220.xml  
>       inflating: tei/ordinarysAccounts/OA17221231.xml  
>       inflating: tei/ordinarysAccounts/OA17160921.xml  
>       inflating: tei/ordinarysAccounts/OA17220314.xml  
>       inflating: tei/ordinarysAccounts/OA16780417.xml  
>       inflating: tei/ordinarysAccounts/OA17150803.xml  
>       inflating: tei/ordinarysAccounts/OA16890531.xml  
>       inflating: tei/ordinarysAccounts/OA17530723.xml  
>       inflating: tei/ordinarysAccounts/OA17550804.xml  
>       inflating: tei/ordinarysAccounts/OA17410504.xml  
>       inflating: tei/ordinarysAccounts/OA17410318.xml  
>       inflating: tei/ordinarysAccounts/OA17170320.xml  
>       inflating: tei/ordinarysAccounts/OA17540401.xml  
>       inflating: tei/ordinarysAccounts/OA17150921.xml  
>       inflating: tei/ordinarysAccounts/OA17281111.xml  
>       inflating: tei/ordinarysAccounts/OA17260314.xml  
>       inflating: tei/ordinarysAccounts/OA17490804.xml  
>       inflating: tei/ordinarysAccounts/OA17001106.xml  
>       inflating: tei/ordinarysAccounts/OA17560628.xml  
>       inflating: tei/ordinarysAccounts/OA17041025.xml  
>       inflating: tei/ordinarysAccounts/OA17531001.xml  
>       inflating: tei/ordinarysAccounts/OA17670914.xml  
>       inflating: tei/ordinarysAccounts/OA17501003.xml  
>       inflating: tei/ordinarysAccounts/OA17090803.xml  
>       inflating: tei/ordinarysAccounts/OA17071217.xml  
>       inflating: tei/ordinarysAccounts/OA17080303a.xml  
>       inflating: tei/ordinarysAccounts/OA16841219.xml  
>       inflating: tei/ordinarysAccounts/OA17641217.xml  
>       inflating: tei/ordinarysAccounts/OA16980622.xml  
>       inflating: tei/ordinarysAccounts/OA17551112.xml  
>       inflating: tei/ordinarysAccounts/OA16861217.xml  
>       inflating: tei/ordinarysAccounts/OA17010319.xml  
>       inflating: tei/ordinarysAccounts/OA17231223.xml  
>       inflating: tei/ordinarysAccounts/OA17610420.xml  
>       inflating: tei/ordinarysAccounts/OA17171002.xml  
>       inflating: tei/ordinarysAccounts/OA17441224.xml  
>        creating: tei/sessionsPapers/
>       inflating: tei/sessionsPapers/18800322.xml  
>       inflating: tei/sessionsPapers/18791124.xml  
>       inflating: tei/sessionsPapers/18061029.xml  
>       inflating: tei/sessionsPapers/17790915.xml  
>
>     *** WARNING: skipped 76527 bytes of output ***
>
>       inflating: tei/sessionsPapers/17590530.xml  
>       inflating: tei/sessionsPapers/19101011.xml  
>       inflating: tei/sessionsPapers/18120115.xml  
>       inflating: tei/sessionsPapers/18850622.xml  
>       inflating: tei/sessionsPapers/18840225.xml  
>       inflating: tei/sessionsPapers/19080526.xml  
>       inflating: tei/sessionsPapers/18511124.xml  
>       inflating: tei/sessionsPapers/18730407.xml  
>       inflating: tei/sessionsPapers/18861025.xml  
>       inflating: tei/sessionsPapers/17821204.xml  
>       inflating: tei/sessionsPapers/18191201.xml  
>       inflating: tei/sessionsPapers/17870418.xml  
>       inflating: tei/sessionsPapers/18681123.xml  
>       inflating: tei/sessionsPapers/18670506.xml  
>       inflating: tei/sessionsPapers/18350615.xml  
>       inflating: tei/sessionsPapers/18960420.xml  
>       inflating: tei/sessionsPapers/19010204.xml  
>       inflating: tei/sessionsPapers/17350522.xml  
>       inflating: tei/sessionsPapers/18140914.xml  
>       inflating: tei/sessionsPapers/17600521.xml  
>       inflating: tei/sessionsPapers/17070903.xml  
>       inflating: tei/sessionsPapers/17950701.xml  
>       inflating: tei/sessionsPapers/17340424.xml  
>       inflating: tei/sessionsPapers/17760522.xml  
>       inflating: tei/sessionsPapers/17220907.xml  
>       inflating: tei/sessionsPapers/19020909.xml  
>       inflating: tei/sessionsPapers/18180909.xml  
>       inflating: tei/sessionsPapers/17690510.xml  
>       inflating: tei/sessionsPapers/17970920.xml  
>       inflating: tei/sessionsPapers/18910727.xml  
>       inflating: tei/sessionsPapers/17970426.xml  
>       inflating: tei/sessionsPapers/18510915.xml  
>       inflating: tei/sessionsPapers/16770601.xml  
>       inflating: tei/sessionsPapers/17210301.xml  
>       inflating: tei/sessionsPapers/17531024.xml  
>       inflating: tei/sessionsPapers/17890603.xml  
>       inflating: tei/sessionsPapers/19121008.xml  
>       inflating: tei/sessionsPapers/18371127.xml  
>       inflating: tei/sessionsPapers/18181028.xml  
>       inflating: tei/sessionsPapers/18210411.xml  
>       inflating: tei/sessionsPapers/18380129.xml  
>       inflating: tei/sessionsPapers/17991204.xml  
>       inflating: tei/sessionsPapers/18991023.xml  
>       inflating: tei/sessionsPapers/18391216.xml  
>       inflating: tei/sessionsPapers/18370918.xml  
>       inflating: tei/sessionsPapers/19020113.xml  
>       inflating: tei/sessionsPapers/17420428.xml  
>       inflating: tei/sessionsPapers/16840702.xml  
>       inflating: tei/sessionsPapers/18720506.xml  
>       inflating: tei/sessionsPapers/18100411.xml  
>       inflating: tei/sessionsPapers/16751013.xml  
>       inflating: tei/sessionsPapers/18990626.xml  
>       inflating: tei/sessionsPapers/17761204.xml  
>       inflating: tei/sessionsPapers/18930109.xml  
>       inflating: tei/sessionsPapers/16810413.xml  
>       inflating: tei/sessionsPapers/17641017.xml  
>       inflating: tei/sessionsPapers/18120701.xml  
>       inflating: tei/sessionsPapers/17850914.xml  
>       inflating: tei/sessionsPapers/16930531.xml  
>       inflating: tei/sessionsPapers/19040620.xml  
>       inflating: tei/sessionsPapers/18520105.xml  
>       inflating: tei/sessionsPapers/17190408.xml  
>       inflating: tei/sessionsPapers/17860531.xml  
>       inflating: tei/sessionsPapers/18680706.xml  
>       inflating: tei/sessionsPapers/18980425.xml  
>       inflating: tei/sessionsPapers/18580222.xml  
>       inflating: tei/sessionsPapers/16890516.xml  
>       inflating: tei/sessionsPapers/17390502.xml  
>       inflating: tei/sessionsPapers/18040215.xml  
>       inflating: tei/sessionsPapers/17330404.xml  
>       inflating: tei/sessionsPapers/18520510.xml  
>       inflating: tei/sessionsPapers/17780218.xml  
>       inflating: tei/sessionsPapers/19121201.xml  
>       inflating: tei/sessionsPapers/19090518.xml  
>       inflating: tei/sessionsPapers/18601022.xml  
>       inflating: tei/sessionsPapers/18580201.xml  
>       inflating: tei/sessionsPapers/18310217.xml  
>       inflating: tei/sessionsPapers/18780527.xml  
>       inflating: tei/sessionsPapers/18231203.xml  
>       inflating: tei/sessionsPapers/18200918.xml  
>       inflating: tei/sessionsPapers/17260420.xml  
>       inflating: tei/sessionsPapers/17971206.xml  
>       inflating: tei/sessionsPapers/18840128.xml  
>       inflating: tei/sessionsPapers/18430508.xml  
>       inflating: tei/sessionsPapers/18770528.xml  
>       inflating: tei/sessionsPapers/18530509.xml  
>       inflating: tei/sessionsPapers/18890107.xml  
>       inflating: tei/sessionsPapers/17760417.xml  
>       inflating: tei/sessionsPapers/17591024.xml  
>       inflating: tei/sessionsPapers/17540227.xml  
>       inflating: tei/sessionsPapers/17171016.xml  
>       inflating: tei/sessionsPapers/18780408.xml  
>       inflating: tei/sessionsPapers/17291203.xml  
>       inflating: tei/sessionsPapers/16900903.xml  
>       inflating: tei/sessionsPapers/17710911.xml  
>       inflating: tei/sessionsPapers/17040601.xml  
>       inflating: tei/sessionsPapers/17430114.xml  
>       inflating: tei/sessionsPapers/18680817.xml  
>       inflating: tei/sessionsPapers/18680127.xml  
>       inflating: tei/sessionsPapers/17710515.xml  
>       inflating: tei/sessionsPapers/17350911.xml  
>       inflating: tei/sessionsPapers/19120109.xml  
>       inflating: tei/sessionsPapers/18841117.xml  
>       inflating: tei/sessionsPapers/18871024.xml  
>       inflating: tei/sessionsPapers/18940528.xml  
>       inflating: tei/sessionsPapers/17280501.xml  
>       inflating: tei/sessionsPapers/17150223.xml  
>       inflating: tei/sessionsPapers/16821206.xml  
>       inflating: tei/sessionsPapers/17361208.xml  
>       inflating: tei/sessionsPapers/18590704.xml  
>       inflating: tei/sessionsPapers/16910909.xml  
>       inflating: tei/sessionsPapers/18920725.xml  
>       inflating: tei/sessionsPapers/19110110.xml  
>       inflating: tei/sessionsPapers/17521206.xml  
>       inflating: tei/sessionsPapers/18350921.xml  
>       inflating: tei/sessionsPapers/18370227.xml  
>       inflating: tei/sessionsPapers/18810228.xml  
>       inflating: tei/sessionsPapers/18210912.xml  
>       inflating: tei/sessionsPapers/18660409.xml  
>       inflating: tei/sessionsPapers/16910422.xml  
>       inflating: tei/sessionsPapers/18170219.xml  
>       inflating: tei/sessionsPapers/18770507.xml  
>       inflating: tei/sessionsPapers/19071021.xml  
>       inflating: tei/sessionsPapers/16840515.xml  
>       inflating: tei/sessionsPapers/19020210.xml  
>       inflating: tei/sessionsPapers/18111030.xml  
>       inflating: tei/sessionsPapers/17200907.xml  
>       inflating: tei/sessionsPapers/16820223.xml  
>       inflating: tei/sessionsPapers/16820906A.xml  
>       inflating: tei/sessionsPapers/19090202.xml  
>       inflating: tei/sessionsPapers/17180110.xml  
>       inflating: tei/sessionsPapers/18550226.xml  
>       inflating: tei/sessionsPapers/17930220.xml  
>       inflating: tei/sessionsPapers/18890729.xml  
>       inflating: tei/sessionsPapers/18700711.xml  
>       inflating: tei/sessionsPapers/17181205.xml  
>       inflating: tei/sessionsPapers/18410510.xml  
>       inflating: tei/sessionsPapers/19011021.xml  
>       inflating: tei/sessionsPapers/18180218.xml  
>       inflating: tei/sessionsPapers/16750707.xml  
>       inflating: tei/sessionsPapers/17160906.xml  
>       inflating: tei/sessionsPapers/19080303.xml  
>       inflating: tei/sessionsPapers/17580405.xml  
>       inflating: tei/sessionsPapers/17510523.xml  
>       inflating: tei/sessionsPapers/18430102.xml  
>       inflating: tei/sessionsPapers/17940716.xml  
>       inflating: tei/sessionsPapers/19101206.xml  
>       inflating: tei/sessionsPapers/17520625.xml  
>       inflating: tei/sessionsPapers/17820220.xml  
>       inflating: tei/sessionsPapers/16800117.xml  
>       inflating: tei/sessionsPapers/17720909.xml  
>       inflating: tei/sessionsPapers/18080713.xml  
>       inflating: tei/sessionsPapers/18460105.xml  
>       inflating: tei/sessionsPapers/18061203.xml  
>       inflating: tei/sessionsPapers/17270113.xml  
>       inflating: tei/sessionsPapers/18520816.xml  
>       inflating: tei/sessionsPapers/17210830.xml  
>       inflating: tei/sessionsPapers/17870115.xml  
>       inflating: tei/sessionsPapers/18291203.xml  
>       inflating: tei/sessionsPapers/18670107.xml  
>       inflating: tei/sessionsPapers/17211206.xml  
>       inflating: tei/sessionsPapers/18841215.xml  
>       inflating: tei/sessionsPapers/17690112.xml  
>       inflating: tei/sessionsPapers/18400106.xml  
>       inflating: tei/sessionsPapers/18100919.xml  
>       inflating: tei/sessionsPapers/17190514.xml  
>       inflating: tei/sessionsPapers/17120111.xml  
>       inflating: tei/sessionsPapers/17510703.xml  
>       inflating: tei/sessionsPapers/18460921.xml  
>       inflating: tei/sessionsPapers/18390204.xml  
>       inflating: tei/sessionsPapers/17500912.xml  
>       inflating: tei/sessionsPapers/17440113.xml  
>       inflating: tei/sessionsPapers/18550917.xml  
>       inflating: tei/sessionsPapers/18791020.xml  
>       inflating: tei/sessionsPapers/18290910.xml  
>       inflating: tei/sessionsPapers/18311201.xml  
>       inflating: tei/sessionsPapers/17640607.xml  
>       inflating: tei/sessionsPapers/18900303.xml  
>       inflating: tei/sessionsPapers/17700711.xml  
>       inflating: tei/sessionsPapers/19060305.xml  
>       inflating: tei/sessionsPapers/18560204.xml  
>       inflating: tei/sessionsPapers/18880528.xml  
>       inflating: tei/sessionsPapers/18000402.xml  
>       inflating: tei/sessionsPapers/18990912.xml  
>       inflating: tei/sessionsPapers/16830829.xml  
>       inflating: tei/sessionsPapers/19061022.xml  
>       inflating: tei/sessionsPapers/17470715.xml  
>       inflating: tei/sessionsPapers/18960323.xml  
>       inflating: tei/sessionsPapers/17830604.xml  
>       inflating: tei/sessionsPapers/18491029.xml  
>       inflating: tei/sessionsPapers/18951118.xml  
>       inflating: tei/sessionsPapers/18260406.xml  
>       inflating: tei/sessionsPapers/18030112.xml  
>       inflating: tei/sessionsPapers/19080908.xml  
>       inflating: tei/sessionsPapers/18560616.xml  
>       inflating: tei/sessionsPapers/18970628.xml  
>       inflating: tei/sessionsPapers/16761213.xml  
>       inflating: tei/sessionsPapers/17500117.xml  
>       inflating: tei/sessionsPapers/17601022.xml  
>       inflating: tei/sessionsPapers/18420103.xml  
>       inflating: tei/sessionsPapers/18240715.xml  
>       inflating: tei/sessionsPapers/18410823.xml  
>       inflating: tei/sessionsPapers/18831210.xml  
>       inflating: tei/sessionsPapers/18441125.xml  
>       inflating: tei/sessionsPapers/17960217.xml  
>       inflating: tei/sessionsPapers/16921207.xml  
>       inflating: tei/sessionsPapers/18970503.xml  
>       inflating: tei/sessionsPapers/19041017.xml  
>       inflating: tei/sessionsPapers/17120227.xml  
>       inflating: tei/sessionsPapers/16820712.xml  
>       inflating: tei/sessionsPapers/18360815.xml  
>       inflating: tei/sessionsPapers/17760221.xml  
>       inflating: tei/sessionsPapers/18351026.xml  
>       inflating: tei/sessionsPapers/18221204.xml  
>       inflating: tei/sessionsPapers/17900526.xml  
>       inflating: tei/sessionsPapers/16791015.xml  
>       inflating: tei/sessionsPapers/18420131.xml  
>       inflating: tei/sessionsPapers/19091012.xml  
>       inflating: tei/sessionsPapers/19100111.xml  
>       inflating: tei/sessionsPapers/16830418A.xml  
>       inflating: tei/sessionsPapers/17570420.xml  
>       inflating: tei/sessionsPapers/16871012.xml  
>       inflating: tei/sessionsPapers/16770425.xml  
>       inflating: tei/sessionsPapers/18230910.xml  
>       inflating: tei/sessionsPapers/17600709.xml  
>       inflating: tei/sessionsPapers/17640113.xml  
>       inflating: tei/sessionsPapers/17290226.xml  
>       inflating: tei/sessionsPapers/18360229.xml  
>       inflating: tei/sessionsPapers/17571207.xml  
>       inflating: tei/sessionsPapers/17200712.xml  
>       inflating: tei/sessionsPapers/17731020.xml  
>       inflating: tei/sessionsPapers/17560528.xml  
>       inflating: tei/sessionsPapers/19110207.xml  
>       inflating: tei/sessionsPapers/18650508.xml  
>       inflating: tei/sessionsPapers/19081208.xml  
>       inflating: tei/sessionsPapers/18470510.xml  
>       inflating: tei/sessionsPapers/17340630.xml  
>       inflating: tei/sessionsPapers/16991213A.xml  
>       inflating: tei/sessionsPapers/18631214.xml  
>       inflating: tei/sessionsPapers/18400511.xml  
>       inflating: tei/sessionsPapers/18680921.xml  
>       inflating: tei/sessionsPapers/16940524.xml  
>       inflating: tei/sessionsPapers/18440205.xml  
>       inflating: tei/sessionsPapers/17980523.xml  
>       inflating: tei/sessionsPapers/18591024.xml  
>       inflating: tei/sessionsPapers/19111010.xml  
>       inflating: tei/sessionsPapers/17470909.xml  
>       inflating: tei/sessionsPapers/17490405.xml  
>       inflating: tei/sessionsPapers/18690816.xml  
>       inflating: tei/sessionsPapers/17310115.xml  
>       inflating: tei/sessionsPapers/18190707.xml  
>       inflating: tei/sessionsPapers/17410225.xml  
>       inflating: tei/sessionsPapers/17720603.xml  
>       inflating: tei/sessionsPapers/18740202.xml  
>       inflating: tei/sessionsPapers/18761120.xml  
>       inflating: tei/sessionsPapers/16820116.xml  
>       inflating: tei/sessionsPapers/18390617.xml  
>       inflating: tei/sessionsPapers/18670610.xml  
>       inflating: tei/sessionsPapers/17161105.xml  
>       inflating: tei/sessionsPapers/18411213.xml  
>       inflating: tei/sessionsPapers/18570615.xml  
>       inflating: tei/sessionsPapers/18210214.xml  
>       inflating: tei/sessionsPapers/18660129.xml  
>       inflating: tei/sessionsPapers/18320906.xml  
>       inflating: tei/sessionsPapers/19080721.xml  
>       inflating: tei/sessionsPapers/16860414.xml  
>       inflating: tei/sessionsPapers/18610408.xml  
>       inflating: tei/sessionsPapers/17080707.xml  
>       inflating: tei/sessionsPapers/17890422.xml  
>       inflating: tei/sessionsPapers/17830726.xml  
>       inflating: tei/sessionsPapers/17750111.xml  
>       inflating: tei/sessionsPapers/18820626.xml  
>       inflating: tei/sessionsPapers/16750219.xml  
>       inflating: tei/sessionsPapers/18980913.xml  
>       inflating: tei/sessionsPapers/18440610.xml  
>       inflating: tei/sessionsPapers/18530103.xml  
>       inflating: tei/sessionsPapers/18930724.xml  
>       inflating: tei/sessionsPapers/18480821.xml  
>       inflating: tei/sessionsPapers/18790526.xml  
>       inflating: tei/sessionsPapers/18370130.xml  
>       inflating: tei/sessionsPapers/18421128.xml  
>       inflating: tei/sessionsPapers/19130401.xml  
>       inflating: tei/sessionsPapers/18140706.xml  
>       inflating: tei/sessionsPapers/18820227.xml  
>       inflating: tei/sessionsPapers/17751206.xml  
>       inflating: tei/sessionsPapers/17680413.xml  
>       inflating: tei/sessionsPapers/17810912.xml  
>       inflating: tei/sessionsPapers/18720108.xml  
>       inflating: tei/sessionsPapers/17461015.xml  
>       inflating: tei/sessionsPapers/18910504.xml  
>       inflating: tei/sessionsPapers/18930626.xml  
>       inflating: tei/sessionsPapers/18821016.xml  
>       inflating: tei/sessionsPapers/18891118.xml  
>       inflating: tei/sessionsPapers/17520914.xml  
>       inflating: tei/sessionsPapers/17400227.xml  
>       inflating: tei/sessionsPapers/17241204.xml  
>       inflating: tei/sessionsPapers/16811207.xml  
>       inflating: tei/sessionsPapers/18570406.xml  
>       inflating: tei/sessionsPapers/17450710.xml  
>       inflating: tei/sessionsPapers/19120611.xml  
>       inflating: tei/sessionsPapers/17850406.xml  
>       inflating: tei/sessionsPapers/18581213.xml  
>       inflating: tei/sessionsPapers/19050403.xml  
>       inflating: tei/sessionsPapers/18590815.xml  
>       inflating: tei/sessionsPapers/18861213.xml  
>       inflating: tei/sessionsPapers/17740518.xml  
>       inflating: tei/sessionsPapers/17110112.xml  
>       inflating: tei/sessionsPapers/17391017.xml  
>       inflating: tei/sessionsPapers/18201206.xml  
>       inflating: tei/sessionsPapers/17130708.xml  
>       inflating: tei/sessionsPapers/17320525.xml  
>       inflating: tei/sessionsPapers/18561027.xml  
>       inflating: tei/sessionsPapers/18921114.xml  
>       inflating: tei/sessionsPapers/17680518.xml  
>       inflating: tei/sessionsPapers/18540508.xml  
>       inflating: tei/sessionsPapers/19070225.xml  
>       inflating: tei/sessionsPapers/18250915.xml  
>       inflating: tei/sessionsPapers/17660903.xml  
>       inflating: tei/sessionsPapers/17740706.xml  
>       inflating: tei/sessionsPapers/17610116.xml  
>       inflating: tei/sessionsPapers/16780116.xml  
>       inflating: tei/sessionsPapers/18810627.xml  
>       inflating: tei/sessionsPapers/18081130.xml  
>       inflating: tei/sessionsPapers/18860628.xml  
>       inflating: tei/sessionsPapers/17800405.xml  
>       inflating: tei/sessionsPapers/18080914.xml  
>       inflating: tei/sessionsPapers/18710710.xml  
>       inflating: tei/sessionsPapers/18561215.xml  
>       inflating: tei/sessionsPapers/17471014.xml  
>       inflating: tei/sessionsPapers/18381231.xml  
>       inflating: tei/sessionsPapers/19020310.xml  
>       inflating: tei/sessionsPapers/18790805.xml  
>       inflating: tei/sessionsPapers/17240226.xml  
>       inflating: tei/sessionsPapers/18200416.xml  
>       inflating: tei/sessionsPapers/17450911.xml  
>       inflating: tei/sessionsPapers/19050502.xml  
>       inflating: tei/sessionsPapers/17760710.xml  
>       inflating: tei/sessionsPapers/17750913.xml  
>       inflating: tei/sessionsPapers/17920523.xml  
>       inflating: tei/sessionsPapers/18490611.xml  
>       inflating: tei/sessionsPapers/17561020.xml  
>       inflating: tei/sessionsPapers/18990109.xml  
>       inflating: tei/sessionsPapers/18560818.xml  
>       inflating: tei/sessionsPapers/17700221.xml  
>       inflating: tei/sessionsPapers/18960622.xml  
>       inflating: tei/sessionsPapers/18490507.xml  
>       inflating: tei/sessionsPapers/17161010.xml  
>       inflating: tei/sessionsPapers/17860719.xml  
>       inflating: tei/sessionsPapers/18171029.xml  
>       inflating: tei/sessionsPapers/17960511.xml  
>       inflating: tei/sessionsPapers/18750920.xml  
>       inflating: tei/sessionsPapers/17350702.xml  
>       inflating: tei/sessionsPapers/19090907.xml  
>       inflating: tei/sessionsPapers/19090323.xml  
>       inflating: tei/sessionsPapers/18800628.xml  
>       inflating: tei/sessionsPapers/18640919.xml  
>       inflating: tei/sessionsPapers/17980214.xml  
>       inflating: tei/sessionsPapers/17260425.xml  
>       inflating: tei/sessionsPapers/18630608.xml  
>       inflating: tei/sessionsPapers/17841208.xml  
>       inflating: tei/sessionsPapers/18690301.xml  
>       inflating: tei/sessionsPapers/18880702.xml  
>       inflating: tei/sessionsPapers/17710703.xml  
>       inflating: tei/sessionsPapers/17721209.xml  
>       inflating: tei/sessionsPapers/18650814.xml  
>       inflating: tei/sessionsPapers/17230710.xml  
>       inflating: tei/sessionsPapers/18740921.xml  
>       inflating: tei/sessionsPapers/17840601.xml  
>       inflating: tei/sessionsPapers/18511215.xml  
>       inflating: tei/sessionsPapers/16950703.xml  
>       inflating: tei/sessionsPapers/18880730.xml  
>       inflating: tei/sessionsPapers/18370508.xml  
>       inflating: tei/sessionsPapers/17061206.xml  
>       inflating: tei/sessionsPapers/17481012.xml  
>       inflating: tei/sessionsPapers/18091206.xml  
>       inflating: tei/sessionsPapers/18380226.xml  
>       inflating: tei/sessionsPapers/19000723.xml  
>       inflating: tei/sessionsPapers/17241014.xml  
>       inflating: tei/sessionsPapers/17611021.xml  
>       inflating: tei/sessionsPapers/17231204.xml  
>       inflating: tei/sessionsPapers/18940430.xml  
>       inflating: tei/sessionsPapers/18730113.xml  
>       inflating: tei/sessionsPapers/18440304.xml  
>       inflating: tei/sessionsPapers/18900728.xml  
>       inflating: tei/sessionsPapers/17090302.xml  
>       inflating: tei/sessionsPapers/17221205.xml  
>       inflating: tei/sessionsPapers/16861013.xml  
>       inflating: tei/sessionsPapers/16781016.xml  
>       inflating: tei/sessionsPapers/17930911.xml  
>       inflating: tei/sessionsPapers/18400817.xml  
>       inflating: tei/sessionsPapers/18751025.xml  
>       inflating: tei/sessionsPapers/19060625.xml  
>       inflating: tei/sessionsPapers/17860222.xml  
>       inflating: tei/sessionsPapers/17220510.xml  
>       inflating: tei/sessionsPapers/17460903.xml  
>       inflating: tei/sessionsPapers/16791210.xml  
>       inflating: tei/sessionsPapers/17670909.xml  
>       inflating: tei/sessionsPapers/19110905.xml  
>       inflating: tei/sessionsPapers/19130107.xml  
>       inflating: tei/sessionsPapers/17460702.xml  
>       inflating: tei/sessionsPapers/18670819.xml  
>       inflating: tei/sessionsPapers/19110328.xml  
>       inflating: tei/sessionsPapers/19120910.xml  
>       inflating: tei/sessionsPapers/18771210.xml  
>       inflating: tei/sessionsPapers/19001022.xml  
>       inflating: tei/sessionsPapers/18820911.xml  
>       inflating: tei/sessionsPapers/17400416.xml  
>       inflating: tei/sessionsPapers/18331017.xml  
>       inflating: tei/sessionsPapers/17031013.xml  
>       inflating: tei/sessionsPapers/17581025.xml  
>       inflating: tei/sessionsPapers/18000219.xml  
>       inflating: tei/sessionsPapers/18110109.xml  
>       inflating: tei/sessionsPapers/17740907.xml  
>       inflating: tei/sessionsPapers/18501021.xml  
>       inflating: tei/sessionsPapers/18920627.xml  
>       inflating: tei/sessionsPapers/18630817.xml  
>       inflating: tei/sessionsPapers/17421208.xml  
>       inflating: tei/sessionsPapers/17810110.xml  
>       inflating: tei/sessionsPapers/18931016.xml  
>       inflating: tei/sessionsPapers/17420224.xml  
>       inflating: tei/sessionsPapers/16980608.xml  
>       inflating: tei/sessionsPapers/17970712.xml  
>       inflating: tei/sessionsPapers/18961019.xml  
>       inflating: tei/sessionsPapers/18831119.xml  
>       inflating: tei/sessionsPapers/18491126.xml  
>       inflating: tei/sessionsPapers/18811121.xml  
>       inflating: tei/sessionsPapers/17790217.xml  
>       inflating: tei/sessionsPapers/18360613.xml  
>       inflating: tei/sessionsPapers/18820522.xml  
>       inflating: tei/sessionsPapers/18430612.xml  
>       inflating: tei/sessionsPapers/18660813.xml  
>       inflating: tei/sessionsPapers/19010107.xml  
>       inflating: tei/sessionsPapers/19110425.xml  
>       inflating: tei/sessionsPapers/18390304.xml  
>       inflating: tei/sessionsPapers/17720429.xml  
>       inflating: tei/sessionsPapers/17520116.xml  
>       inflating: tei/sessionsPapers/17520514.xml  
>       inflating: tei/sessionsPapers/18720226.xml  
>       inflating: tei/sessionsPapers/17990403.xml  
>       inflating: tei/sessionsPapers/18820109.xml  
>       inflating: tei/sessionsPapers/18460330.xml  
>       inflating: tei/sessionsPapers/18290716.xml  
>       inflating: tei/sessionsPapers/17510417.xml  
>       inflating: tei/sessionsPapers/17710220.xml  
>       inflating: tei/sessionsPapers/18330817.xml  
>       inflating: tei/sessionsPapers/17881210.xml  
>       inflating: tei/sessionsPapers/18660917.xml  
>       inflating: tei/sessionsPapers/18710109.xml  
>       inflating: tei/sessionsPapers/18040516.xml  
>       inflating: tei/sessionsPapers/17520219.xml  
>       inflating: tei/sessionsPapers/19021020.xml  
>       inflating: tei/sessionsPapers/19120423.xml  
>       inflating: tei/sessionsPapers/18440408.xml  
>       inflating: tei/sessionsPapers/17631019.xml  
>       inflating: tei/sessionsPapers/18030914.xml  
>       inflating: tei/sessionsPapers/17740413.xml  
>       inflating: tei/sessionsPapers/17870711.xml  
>       inflating: tei/sessionsPapers/18361212.xml  
>       inflating: tei/sessionsPapers/18520614.xml  
>       inflating: tei/sessionsPapers/17140908.xml  
>       inflating: tei/sessionsPapers/17990220.xml  
>       inflating: tei/sessionsPapers/16780828.xml  
>       inflating: tei/sessionsPapers/18340515.xml  
>       inflating: tei/sessionsPapers/18491217.xml  
>       inflating: tei/sessionsPapers/18540814.xml  
>       inflating: tei/sessionsPapers/17330221.xml  
>       inflating: tei/sessionsPapers/17800223.xml  
>       inflating: tei/sessionsPapers/18790303.xml  
>       inflating: tei/sessionsPapers/17590912.xml  
>       inflating: tei/sessionsPapers/18430918.xml  
>       inflating: tei/sessionsPapers/18570202.xml  
>       inflating: tei/sessionsPapers/18410301.xml  
>       inflating: tei/sessionsPapers/18980207.xml  
>       inflating: tei/sessionsPapers/18350202.xml  
>       inflating: tei/sessionsPapers/18780806.xml  
>       inflating: tei/sessionsPapers/16860707.xml  
>       inflating: tei/sessionsPapers/17160411.xml  
>       inflating: tei/sessionsPapers/18701024.xml  
>       inflating: tei/sessionsPapers/18821211.xml  
>       inflating: tei/sessionsPapers/18211205.xml  
>       inflating: tei/sessionsPapers/16860520.xml  
>       inflating: tei/sessionsPapers/19000625.xml  
>       inflating: tei/sessionsPapers/18411129.xml  
>       inflating: tei/sessionsPapers/18110220.xml  
>       inflating: tei/sessionsPapers/16961209.xml  
>       inflating: tei/sessionsPapers/18270405.xml  
>       inflating: tei/sessionsPapers/17940604.xml  
>       inflating: tei/sessionsPapers/17820515.xml  
>       inflating: tei/sessionsPapers/18301028.xml  
>       inflating: tei/sessionsPapers/18700404.xml  
>       inflating: tei/sessionsPapers/17700530.xml  
>       inflating: tei/sessionsPapers/18210606.xml  
>       inflating: tei/sessionsPapers/18100606.xml  
>       inflating: tei/sessionsPapers/18130602.xml  
>       inflating: tei/sessionsPapers/17230227.xml  
>       inflating: tei/sessionsPapers/17370706.xml  
>       inflating: tei/sessionsPapers/19060911.xml  
>       inflating: tei/sessionsPapers/17920215.xml  
>       inflating: tei/sessionsPapers/18950520.xml  
>       inflating: tei/sessionsPapers/18351214.xml  
>       inflating: tei/sessionsPapers/18761211.xml  
>       inflating: tei/sessionsPapers/18671216.xml  
>       inflating: tei/sessionsPapers/17841210.xml  
>       inflating: tei/sessionsPapers/19090112.xml  
>       inflating: tei/sessionsPapers/18530228.xml  
>       inflating: tei/sessionsPapers/17671021.xml  
>       inflating: tei/sessionsPapers/18510407.xml  
>       inflating: tei/sessionsPapers/18780114.xml  
>       inflating: tei/sessionsPapers/16931206.xml  
>       inflating: tei/sessionsPapers/17590228.xml  
>       inflating: tei/sessionsPapers/16860114.xml  
>       inflating: tei/sessionsPapers/18610923.xml  
>       inflating: tei/sessionsPapers/17560603.xml  
>       inflating: tei/sessionsPapers/17801206.xml  
>       inflating: tei/sessionsPapers/18201028.xml  
>       inflating: tei/sessionsPapers/17150114.xml  
>       inflating: tei/sessionsPapers/18671028.xml  
>       inflating: tei/sessionsPapers/16951014.xml  
>       inflating: tei/sessionsPapers/17611209.xml  
>       inflating: tei/sessionsPapers/16750115.xml  
>       inflating: tei/sessionsPapers/16951203.xml  
>       inflating: tei/sessionsPapers/18620707.xml  
>       inflating: tei/sessionsPapers/18860503.xml  
>       inflating: tei/sessionsPapers/18930911.xml  
>       inflating: tei/sessionsPapers/18140420.xml  
>       inflating: tei/sessionsPapers/17950416.xml  
>       inflating: tei/sessionsPapers/17330112.xml  
>       inflating: tei/sessionsPapers/16980223.xml  
>       inflating: tei/sessionsPapers/17760911.xml  
>       inflating: tei/sessionsPapers/18131027.xml  
>       inflating: tei/sessionsPapers/17300116.xml  

  

Let's put the files in dbfs.

In [ ]:
dbutils.fs.mkdirs("dbfs:/datasets/obo/tei") //need not be done again!

  

>     res1: Boolean = true

In [ ]:
//dbutils.fs.rm("dbfs:/datasets/obo/tei",true)

In [ ]:
ls 
#ls obo-tiny/tei

  

>     books
>     conf
>     derby.log
>     eventlogs
>     ganglia
>     library-install-logs
>     logs
>     OB_tei_7-2_CC-BY-NC.zip
>     OB-tiny_tei_7-2_CC-BY-NC.zip
>     old_sou.tar.gz
>     sou
>     sou.tar.gz
>     sou.tgz
>     tei

In [ ]:
 dbutils.fs.cp("file:/databricks/driver/obo-tiny/tei", "dbfs:/datasets/obo/tei/",recurse=true) // already done and it takes 1500 seconds - a while!
 //dbutils.fs.cp("file:/databricks/driver/tei", "dbfs:/datasets/obo/tei/",recurse=true) // already done and it takes 19 minutes - a while!

  

>     res2: Boolean = true

In [ ]:
//dbutils.fs.rm("dbfs:/datasets/tweets",true) // remove files to make room for the OBO dataset

  

>     res38: Boolean = true

In [ ]:
display(dbutils.fs.ls("dbfs:/datasets/"))

  

[TABLE]

In [ ]:
display(dbutils.fs.ls("dbfs:/datasets/obo/tei/"))

  

[TABLE]

In [ ]:
util.Properties.versionString // check scala version

  

>     res5: String = version 2.11.8